#### version history
-- v5 --
- created 12/06
- taking only prod code update snippet from v4_fromprod
- ....
- Generate config diff for a scope event
- Variant of the Deployed code on staging and 1/12 push
- Generate csv from prod code output sequence file for validation/ manually see the diff
- Unravel json of the ap config in ap-events explode

-- v4_fromprod --
- created 10/26
- copied over from production, to carry on woring in staging

In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
186,application_1642782761878_0188,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-186>

<h4>Imports

In [2]:
import json
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

import numpy as np
import re
import difflib

import time
import datetime
import boto3

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
var_date = 15
var_hr = 14



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
s3locRoot_ee = 's3://mist-aggregated-stats-production/entity_event/'
s3locChild_ee = 'entity_event-production/'
dt_ee = "dt=2021-11-%d/" %var_date
hr_ee = "hr=%d/" % var_hr
s3loc_ee = s3locRoot_ee + s3locChild_ee + dt_ee + hr_ee
print('\n--- s3loc_ee ---')
print(s3loc_ee)

event_rdd_ee = sc.sequenceFile(s3loc_ee).map(lambda r: json.loads(r[1])) #.persist()

event2consider = ['connectivity_failure']

event_rdd_ee_scope = event_rdd_ee.filter(lambda r: (('event_name' in r.keys() ) and (r['event_name'] in event2consider) \
                                                    and ('ap' in r['reason'].keys()) and (r['severity'] > 50) ) )
print('\n',event_rdd_ee_scope.count()) # typically 250403 in 3 days for ['connectivity_failure', 'saturn_anomaly', 'preconnection_anomaly']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2021-11-15/hr=14/

 38

In [5]:
pullSchema = T.StructType([       
    T.StructField('row_key', T.StringType(), True),
    T.StructField('start_time', T.LongType(), True),
    T.StructField('severity', T.FloatType(), True),
    T.StructField('details', T.StringType(), True),
    T.StructField('reason', T.MapType(T.StringType(), T.StringType()), True)
])
df_selectScopeEvents_apscope = event_rdd_ee_scope.map(lambda r: (r['row_key'],r['start_time'], r['severity'], r['details'],r['reason'])).toDF(pullSchema)

apList = [x.ap for x in df_selectScopeEvents_apscope.select('reason.ap').distinct().dropna().collect()]
print(len(apList))
print('Some ap ids: %s', str(apList[:10]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

21
Some ap ids: %s ['5c5b358e4272', '5c5b358eb108', '5c5b355261f6', '5c5b358eb842', '5c5b350ede55', '5c5b358ebcc0', '5c5b358e3304', '5c5b351f1ee0', 'd420b044b2a3', '5c5b358ec076']

In [13]:
s3locRoot_secorAPevents = 's3://mist-secorapp-production/'

s3loc_secorAPevents_6hr = ['ap-events/ap-events-production/'+dt_ee + "hr=%s/" %str(x_).zfill(2) for x_ in np.arange(var_hr,var_hr-7,-1)]
s3loc_secorAPevents_7days = ['ap-events/ap-events-production/'+ "dt=2021-11-%s/" %str(x_).zfill(2) + hr_ee for x_ in np.arange(var_date,var_date-7,-1)]

# print(s3loc_secorAPevents_6hr)
# print(s3loc_secorAPevents_7days)

patternCompile = re.compile('(ap-events/ap-events-production/dt=)(2021-\d\d-\d\d/)(hr=\d\d/)')
dtList_7days = set([patternCompile.search(x_).group(2) for x_ in s3loc_secorAPevents_7days])
print(dtList_7days)


s3locRoot_secorAPevents = 's3://mist-secorapp-production/'
apfileLocations_6hr = [s3locRoot_secorAPevents + x_ for x_ in sorted(s3loc_secorAPevents_6hr)]
print('\nThese are files for up to 6 hours ago')
print(apfileLocations_6hr)

s3locRoot_secorAPevents2 = set([patternCompile.search(x_).group(1) for x_ in s3loc_secorAPevents_7days])
print(s3locRoot_secorAPevents2)

apfileLocations_7days = [s3locRoot_secorAPevents + list(s3locRoot_secorAPevents2)[0]+x_+"hr=*/*/" for x_ in list(dtList_7days)]
# apfileLocations_7days = [s3locRoot_secorAPevents + x_ for x_ in sorted(s3loc_secorAPevents_7days)]
print('\nThese are files for up to 7 days ago')
print(apfileLocations_7days)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'2021-11-11/', '2021-11-14/', '2021-11-15/', '2021-11-10/', '2021-11-09/', '2021-11-13/', '2021-11-12/'}

These are files for up to 6 hours ago
['s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=08/', 's3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=09/', 's3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=10/', 's3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=11/', 's3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=12/', 's3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=13/', 's3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=14/']
{'ap-events/ap-events-production/dt='}

These are files for up to 7 days ago
['s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-11/hr=*/*/', 's3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-14/hr=*/*/', 's3://mis

In [7]:
#pass 1
outerloopdf = None

pullSchema2 = T.StructType([       
    T.StructField('ap', T.StringType(), True),
    T.StructField('ev_name', T.StringType(), True),
    T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),
    T.StructField('timestamp', T.LongType(), True),
])


for readLoc in sorted(apfileLocations_6hr):
    print('Reading from... %s' %readLoc)
    rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
    rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ap'] in apList) and (r['source']['ev_name']=="AP_CONFIGURED") )
    
    # print('\n',event_rdd_apevents_sampledSites.count())
    df_apevents_sample = rdd_apevents_sample.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)
    # df_apevents_sample.printSchema()

    df_apvents_sample_config = df_apevents_sample#.filter('ev_name=="AP_CONFIGURED"')#.persist()
    # df_apvents_sample_config.select('source.ap', 'source.ev_name', 'timestamp').show(10, False)
    loopdf = df_apvents_sample_config#.persist()
    if outerloopdf is None:
        outerloopdf = loopdf
    else:
        outerloopdf = loopdf.unionByName(outerloopdf)

# outerloopdf.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=08/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=09/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=10/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=11/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=12/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=13/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=14/

In [8]:
apList_positive = [x.ap for x in outerloopdf.select('ap').distinct().dropna().collect()]
print(len(apList_positive))
print('Some ap ids: %s', str(apList_positive[:10]))


outerloopdf_pass2 = None

for readLoc in apfileLocations_7days:
    print('Reading from... %s' %readLoc)
    rdd_apevents_pass2 = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
    rdd_apevents_sample_pass2 = rdd_apevents_pass2.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                                             and ('ap' in r['source'].keys()) and (r['source']['ap'] in apList_positive) and (r['source']['ev_name']=="AP_CONFIGURED") )
    
    df_apevents_sample_pass2 = rdd_apevents_sample_pass2.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)
    df_apvents_sample_config_pass2 = df_apevents_sample_pass2
    loopdf_pass2 = df_apvents_sample_config_pass2#.persist()
    if outerloopdf_pass2 is None:
        outerloopdf_pass2 = loopdf_pass2
    else:
        outerloopdf_pass2 = loopdf_pass2.unionByName(outerloopdf_pass2)

# outerloopdf_pass2.persist()
# outerloopdf_pass2.show(10,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

11
Some ap ids: %s ['5c5b358eb108', '5c5b355261f6', '5c5b358ebcc0', '5c5b351f1ee0', '5c5b358ec076', 'd420b0801b76', 'd420b002cb6e', 'd4dc09e4bdcf', 'd4dc09e4bded', '5c5b358e38f9']
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-11/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-14/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-10/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-09/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-13/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-12/hr=*/*/

In [9]:
df_apscope_withconfig = outerloopdf_pass2.withColumn('row_number', F.row_number().over(Window.partitionBy('ap').orderBy(F.col('timestamp').desc() )  )  )
# df_apscope_withconfig.show(3,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
stepWindow = Window.partitionBy('ap').orderBy(F.col("row_number").desc())
df_apscope_withconfig = df_apscope_withconfig.withColumn('config_step', F.lag('config', offset=1).over(stepWindow))
df_apscope_withconfig.persist()
df_apscope_withconfig.show(3,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------+--------------------+----------------+----------+--------------------+
|          ap|      ev_name|              config|       timestamp|row_number|         config_step|
+------------+-------------+--------------------+----------------+----------+--------------------+
|d4dc09e4bdcf|AP_CONFIGURED|{OurOUIs -> [5c-5...|1636447651000000|        14|                null|
|d4dc09e4bdcf|AP_CONFIGURED|{OurOUIs -> [5c-5...|1636484602000000|        13|{OurOUIs -> [5c-5...|
|d4dc09e4bdcf|AP_CONFIGURED|{OurOUIs -> [5c-5...|1636484818000000|        12|{OurOUIs -> [5c-5...|
+------------+-------------+--------------------+----------------+----------+--------------------+
only showing top 3 rows

In [11]:
def getconfigdiff_def(x,y):
    if (x=="") or (y ==""):
        return "'config not found'"
    else:
        try:
            j1 = str(x)#json.loads(x)
            j2 = str(y)#json.loads(y)
            l1 = [x for x in difflib.unified_diff(j1, j2,n=100,lineterm='') if "@@" in x]
            retlist=list()
            for x_ in l1:
                sp = x_.replace("@","").replace(" ","").split("+")[1].split(",")
                s_ = int(sp[0])
                e_ = int(sp[1])
                if sp!=[]:
                    print(s_,e_)
                    retlist.append([{"new":j1[s_: s_+e_-1], "old":j2[s_: s_+e_-1]}])
            retval = str(retlist)
            return retval
        except:
            retval = "error"
        return retval

getconfigdiff = F.udf( getconfigdiff_def, T.StringType() )


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_apscope_withconfigDiff = df_apscope_withconfig .select('ap', 'timestamp', 'row_number', F.to_json(F.col('config')).alias('config'), F.to_json(F.col('config_step')).alias('config_step') )\
                                .withColumn('config_change', getconfigdiff(F.col('config'), F.col('config_step')) )

# df_apscope_withconfigDiff.orderBy('ap', 'row_number').show(10,)

df_m1 = df_selectScopeEvents_apscope.select('row_key', 'start_time', 'reason.ap').join(df_apscope_withconfigDiff, on='ap', how='left')

# df_m1.filter("ap=='5c5b351f1ee0'").orderBy('ap', 'row_number').show(10,)

df_selectScopeEvents_apscope_p2 = df_m1.withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                    .filter('time_delta>0')\
                                    .withColumn('row_number_timedelta', F.row_number().over(Window.partitionBy('ap','row_key','start_time').orderBy(F.col('time_delta').asc() )  )  )
# df_selectScopeEvents_apscope_p2.filter("ap=='5c5b351f1ee0'").orderBy('ap', 'row_number_timedelta').show(20,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# df_selectScopeEvents_apscope_p2.filter("row_number_timedelta=1")\
# .write.mode('overwrite').parquet("s3://mist-staging-jupyterhub-notebooks/jupyter/abhakta@juniper.net/emr-nb/data/scope_with_ap_configs_s1/")

<h4>Change for bulk batch

In [4]:
s3locRoot_ee = 's3://mist-aggregated-stats-production/entity_event/'
s3locChild_ee = 'entity_event-production/'
dt_ee = "dt=2021-11-%d/" %var_date
hr_ee = "hr=*/*/"
s3loc_ee = s3locRoot_ee + s3locChild_ee + dt_ee + hr_ee
print('\n--- s3loc_ee ---')
print(s3loc_ee)

event_rdd_ee = sc.sequenceFile(s3loc_ee).map(lambda r: json.loads(r[1])) #.persist()

event2consider = ['connectivity_failure']

event_rdd_ee_scope = event_rdd_ee.filter(lambda r: (('event_name' in r.keys() ) and (r['event_name'] in event2consider) \
                                                    and ('ap' in r['reason'].keys()) and (r['severity'] > 50) ) )
print('\n',event_rdd_ee_scope.count()) # typically 250403 in 3 days for ['connectivity_failure', 'saturn_anomaly', 'preconnection_anomaly']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2021-11-15/hr=*/*/

 1262

In [6]:
event_rdd_ee_scope.take(1)[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'start_time': 1635793277054, 'end_time': 1636934603522, 'detection_time': 1635796449464, 'modification_time': 1636934726000, 'occurrence': 4802, 'batch_count': 901, 'event_name': 'connectivity_failure', 'event_type': 'failed_scope_authentication', 'mist_only': False, 'enable_action': False, 'category': 'connectivity', 'site_id': '4977c46b-f3b0-4d44-918f-b8b3e116b023', 'org_id': 'a8c903ae-e6d0-42a7-b815-fe395dea8017', 'entity_id': '5c5b358eb842--7f5b5549-a909-4968-9702-0a09a4397691', 'entity_type': 'site', 'display_entity_type': 'site', 'detection_delay': 3172, 'details': {'features': {'clients_list': ['d690ac4e-407e-231f-dd92-88af405edf03', '129cb507-c39c-091e-d490-14198f17c0d1', 'cf5d70c2-7ced-5c47-63da-35eb7ad09070', '1cd3be6a-34c8-b28c-7c35-1cdf6aab28cf', '6b6602d9-5f79-23c4-5e80-e73cac2578eb', 'bcbe30b8-60fd-c7d9-3fa1-29964274a728', '564e19e2-08da-e254-a925-453f4f56e714', 'fdbab779-c520-020c-c120-08e0040aa246', '7cd03708-4718-9446-8983-5fcc351833f4', 'bb55e5d1-04cf-715e-7fb0-86478

In [7]:
a=_

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
for x_ in a.keys():
    print(x_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

start_time
end_time
detection_time
modification_time
occurrence
batch_count
event_name
event_type
mist_only
enable_action
category
site_id
org_id
entity_id
entity_type
display_entity_type
detection_delay
details
row_key
client
scope
reason
severity
event_duration

In [9]:
pullSchema = T.StructType([       
    T.StructField('row_key', T.StringType(), True),
    T.StructField('start_time', T.LongType(), True),
    T.StructField('severity', T.FloatType(), True),
    T.StructField('details', T.StringType(), True),
    T.StructField('reason', T.MapType(T.StringType(), T.StringType()), True)
])
df_selectScopeEvents_apscope = event_rdd_ee_scope.map(lambda r: (r['row_key'],r['start_time'], r['severity'], r['details'],r['reason'])).toDF(pullSchema)

print(pullSchema)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

StructType(List(StructField(row_key,StringType,true),StructField(start_time,LongType,true),StructField(severity,FloatType,true),StructField(details,StringType,true),StructField(reason,MapType(StringType,StringType,true),true)))

In [17]:

apList = [x.ap for x in df_selectScopeEvents_apscope.select('reason.ap').distinct().dropna().collect()]
print(len(apList))
print('Some ap ids: %s', str(apList[:10]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

158
Some ap ids: %s ['5c5b358e4272', '5c5b35f171e8', '5c5b35f17858', 'd4dc0924363e', '5c5b350e164e', '5c5b358eb108', '5c5b35f17030', '5c5b355261f6', '5c5b358eb842', '5c5b35f165b0']

In [18]:
dt_ap = "dt=2021-11-{%d,%d}/" %(var_date-1, var_date)
hr_ap = "hr=*/*/"

readLoc = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap + hr_ap
print(readLoc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-{14,15}/hr=*/*/

In [19]:
pullSchema2 = T.StructType([       
    T.StructField('ap', T.StringType(), True),
    T.StructField('ev_name', T.StringType(), True),
    T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),
    T.StructField('timestamp', T.LongType(), True),
])


print('Reading from... %s' %readLoc)
rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ap'] in apList) and (r['source']['ev_name']=="AP_CONFIGURED") )
    
# print('\n',event_rdd_apevents_sampledSites.count())
df_apevents_sample = rdd_apevents_sample.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)
df_apevents_sample.printSchema()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-{14,15}/hr=*/*/
root
 |-- ap: string (nullable = true)
 |-- ev_name: string (nullable = true)
 |-- config: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- timestamp: long (nullable = true)

In [20]:
df_selectScopeEvents_apscope.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- row_key: string (nullable = true)
 |-- start_time: long (nullable = true)
 |-- severity: float (nullable = true)
 |-- details: string (nullable = true)
 |-- reason: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

In [21]:
dfm1 = df_selectScopeEvents_apscope.select('row_key', 'start_time', 'severity', F.col('reason.ap').alias('ap'))\
            .join(df_apevents_sample, on='ap', how='inner')\
        .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
        .filter('time_delta>0 and time_delta <' + str(6*60*60*1000))

dfm1.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ap: string (nullable = true)
 |-- row_key: string (nullable = true)
 |-- start_time: long (nullable = true)
 |-- severity: float (nullable = true)
 |-- ev_name: string (nullable = true)
 |-- config: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- timestamp: long (nullable = true)
 |-- time_delta: double (nullable = true)

In [22]:
dfm1.show(3,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+-------------+--------+-------------+--------------------+----------------+-----------+
|          ap|             row_key|   start_time|severity|      ev_name|              config|       timestamp| time_delta|
+------------+--------------------+-------------+--------+-------------+--------------------+----------------+-----------+
|5c5b35f17548|5c5b35f17548--6cd...|1637006406675|62.34574|AP_CONFIGURED|{BootConfig -> {O...|1636988147000000|1.8259675E7|
|5c5b35f17548|5c5b35f17548--6cd...|1637006406675|79.80919|AP_CONFIGURED|{BootConfig -> {O...|1636988147000000|1.8259675E7|
|5c5b35f17548|5c5b35f17548--6cd...|1637006406675|85.63146|AP_CONFIGURED|{BootConfig -> {O...|1636988147000000|1.8259675E7|
+------------+--------------------+-------------+--------+-------------+--------------------+----------------+-----------+
only showing top 3 rows

In [25]:
dfm2 = dfm1.withColumn('row_number', F.row_number().over(Window.partitionBy('ap', 'row_key', 'start_time', 'severity').orderBy(F.col('time_delta').desc() )  )  )\
            .filter('row_number==1')
dfm2.show(10,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+-------------+---------+-------------+--------------------+----------------+-----------+----------+
|          ap|             row_key|   start_time| severity|      ev_name|              config|       timestamp| time_delta|row_number|
+------------+--------------------+-------------+---------+-------------+--------------------+----------------+-----------+----------+
|5c5b35f17548|5c5b35f17548--6cd...|1637006406675| 62.34574|AP_CONFIGURED|{BootConfig -> {O...|1636988147000000|1.8259675E7|         1|
|5c5b35f17548|5c5b35f17548--6cd...|1637006406675| 79.80919|AP_CONFIGURED|{BootConfig -> {O...|1636988147000000|1.8259675E7|         1|
|5c5b35f17548|5c5b35f17548--6cd...|1637006406675| 85.63146|AP_CONFIGURED|{BootConfig -> {O...|1636988147000000|1.8259675E7|         1|
|d420b0c121e2|d420b0c121e2--a14...|1636979181835| 63.31937|AP_CONFIGURED|{OurOUIs -> [5c-5...|1636961651000000|1.7530835E7|         1|
|5c5b351f1ee0|5c5b351f1ee0&conn...|1636985440449| 54.28

In [26]:
dfm2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

75

<h5>checks

In [3]:
saveloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2021-12-06/hr=18/FailedScopeConfigDiff_1635418213.seq"
dftemp = sc.sequenceFile(saveloc).map(lambda r: json.loads(r[1]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
dftemp.persist()
dftemp.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

54

In [5]:
a=dftemp.take(1)[0]
for x in a.keys():
    print('--- %s ---' % x)
    print(a[x])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--- start_time ---
1634774435713
--- end_time ---
1635417904040
--- detection_time ---
1634778615627
--- modification_time ---
1635417933000
--- occurrence ---
4882
--- batch_count ---
607
--- event_name ---
connectivity_failure
--- event_type ---
failed_scope_authentication_configdiff
--- mist_only ---
False
--- enable_action ---
False
--- category ---
connectivity
--- site_id ---
9f159831-e7e3-4b26-a481-a40170b0426f
--- org_id ---
5fac827a-edbd-4522-83c6-de8d4a7ea380
--- entity_id ---
d420b0801b76
--- entity_type ---
site
--- display_entity_type ---
site
--- detection_delay ---
4179
--- details ---
{'features': {'clients_list': ['950a83f3-a3c0-b700-e215-247502cdb3e3', 'd49c4f4f-1c13-e066-bb91-eef82cc4dd42', '2b71d5a3-773c-e0c4-3e97-2a31a092981f', '8e4a9bf5-398d-495c-2db5-e25cb7ea4a7a', 'ea2424cc-c66e-6e8b-0068-1dd5d4baae2e', '687d2a68-9495-e026-b035-cc43463165e1', 'ec1d8a5d-4450-8409-2a22-3ddc792cd6f5', '23067ca7-7140-4bcc-e8ce-f1f95cc4bb50', '59609b49-c5f6-0b42-4d63-9c9395b5afc5', '

In [7]:
a['details'].keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['features', 'impacted_subscope', 'symptom', 'event_history'])

In [8]:
a['details']['features'].keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['clients_list', 'roaming', 'tot_fail_events', 'dur', 'num_clients', 'adj_roaming', 'final_features_list'])

In [3]:
df_config = spark.read.parquet("s3://mist-staging-jupyterhub-notebooks/jupyter/abhakta@juniper.net/emr-nb/data/scope_with_ap_configs_1635417631_1635418213/")
df_config.persist()
df_config.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6

In [4]:
df_config.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ap: string (nullable = true)
 |-- row_key: string (nullable = true)
 |-- start_time: long (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- row_number: integer (nullable = true)
 |-- config: string (nullable = true)
 |-- config_step: string (nullable = true)
 |-- config_change: string (nullable = true)
 |-- time_delta: double (nullable = true)
 |-- row_number_timedelta: integer (nullable = true)

In [9]:
rdd_config = df_config.rdd.map(lambda x: x.asDict())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [235]:
rdd_config.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'ap': '5c5b358ec72e', 'row_key': '5c5b358ec72e&connectivity_failure&failed_scope_authentication&1634864461950', 'start_time': 1634864461950, 'timestamp': 1634798932000000, 'row_number': 13, 'config': '{"BootConfig":"{OurOUIs=[5c-5b-35-00-00-00, d4-20-b0-00-00-00, d4-dc-09-00-00-00, ac-23-16-00-00-00], SiteName=12-09408, WIDS=[], L2TPTunnels=[], SVIs=[{dhcp_enabled=true, DHCP_Ena=true, Vlan_ID=1, DHCP={MaxLeaseTime=43200, Hostname=12Y09408M, Enabled=true}, MTU=0}], Radios=[{ProbeControl={ProbeReqSW=true, ProbeRespSW=true}, QOS={Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1}, {DSCP=8, AC=1}, {DSCP=16, AC=1}, {DSCP=9, AC=1}, {DSCP=11, AC=1}, {DSCP=13, AC=1}, {DSCP=15, AC=1}, {DSCP=17, AC=1}, {DSCP=19, AC=1}, {DSCP=21, AC=1}, {DSCP=23, AC=1}]}, PreambleLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=2.4, RRM={Width=20, Power=5, Channel=6}, Width=20, Power=5}, {ProbeControl={ProbeReqSW=true, ProbeResp

In [94]:
from pyspark.sql import Row

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [249]:
def changeformat(x):
    validkeys = ['start_time','ap','config_step']
    retdict = dict()
    for idx in x.keys():
        if idx in validkeys:
            retdict[idx] = str(x[idx])
    return retdict

# d1 = df_config.rdd.map(lambda x: x.asDict()).map(lambda x: changeformat(x)).map(lambda x: (None, json.dumps(x)))

d1 = df_config.rdd.map(lambda x: x.asDict()).map(lambda x: (None, json.dumps(x)))
#.map(lambda x: json.dumps(x))#.map(lambda x: list(x))#.map(lambda x: Row(**x))#

print(d1.take(2))
# dict_keys(['ap', 'row_key', 'start_time', 'timestamp', 'row_number', 'config', 'config_step', 'config_change', 'time_delta', 'row_number_timedelta'])



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(None, '{"ap": "5c5b358ec72e", "row_key": "5c5b358ec72e&connectivity_failure&failed_scope_authentication&1634864461950", "start_time": 1634864461950, "timestamp": 1634798932000000, "row_number": 13, "config": "{\\"BootConfig\\":\\"{OurOUIs=[5c-5b-35-00-00-00, d4-20-b0-00-00-00, d4-dc-09-00-00-00, ac-23-16-00-00-00], SiteName=12-09408, WIDS=[], L2TPTunnels=[], SVIs=[{dhcp_enabled=true, DHCP_Ena=true, Vlan_ID=1, DHCP={MaxLeaseTime=43200, Hostname=12Y09408M, Enabled=true}, MTU=0}], Radios=[{ProbeControl={ProbeReqSW=true, ProbeRespSW=true}, QOS={Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1}, {DSCP=8, AC=1}, {DSCP=16, AC=1}, {DSCP=9, AC=1}, {DSCP=11, AC=1}, {DSCP=13, AC=1}, {DSCP=15, AC=1}, {DSCP=17, AC=1}, {DSCP=19, AC=1}, {DSCP=21, AC=1}, {DSCP=23, AC=1}]}, PreambleLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=2.4, RRM={Width=20, Power=5, Channel=6}, Width=20, Power=5}, {ProbeControl={ProbeReqSW=t

In [250]:
d1.saveAsSequenceFile("s3://mist-staging-jupyterhub-notebooks/jupyter/abhakta@juniper.net/emr-nb/data/scope_with_ap_configs_del12")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [253]:
d2 = sc.sequenceFile("s3://mist-staging-jupyterhub-notebooks/jupyter/abhakta@juniper.net/emr-nb/data/scope_with_ap_configs_del12").map(lambda r: json.loads(r[1]))
print(d2.take(1)[0]['config_change'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[{'new': 'el=44}, Width=40, Power=14}], PoePwrConstrained={EnableRadios=false}, AppDef=[], Timestamp=1.6347989324394932E9, Name=12Y09408M, _errors=[], ProxyArp={Enabled=true, Disable_RespSpoofCheck=true, Disable_ReqSp', 'old': 'el=44}, Width=40, Power=14}], PoePwrConstrained={EnableRadios=false}, AppDef=[], Timestamp=1.63479844371533E9, Name=12Y09408M, _errors=[], ProxyArp={Enabled=true, Disable_RespSpoofCheck=true, Disable_ReqSpoo'}], [{'new': ', Width=40, Power=14}]","PoePwrConstrained":"{EnableRadios=true}","AppDef":"[]","Timestamp":"1.6347989324394932E9","Name":"12Y09408M","_errors":"[]","ProxyArp":"{Enabled=true, Disable_RespSpoofCheck=true, Di', 'old': 'Width=40, Power=14}]","PoePwrConstrained":"{EnableRadios=true}","AppDef":"[]","Timestamp":"1.63479844371533E9","Name":"12Y09408M","_errors":"[]","ProxyArp":"{Enabled=true, Disable_RespSpoofCheck=true, Disabl'}], [{'new': '3PBIMXAQ== ndade2@mistsys.com], Disabled=false}","LED":"{Brightness=255}","time_taken":"{start=1.634798932439

In [17]:
# new check
def pickCols(x):
    validkeys = ['start_time','ap','config']
    retdict = dict()
    for idx in x.keys():
        if idx in validkeys:
            retdict[idx] = x[idx]
    return retdict

d2 = df_config.rdd.map(lambda x: x.asDict()).map(lambda x: pickCols(x)).map(lambda x: (None, json.dumps(x)))
d2.take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(None, '{"ap": "5c5b358ec076", "start_time": 1635001202303, "config": "{\\"BootConfig\\":\\"{OurOUIs=[5c-5b-35-00-00-00, d4-20-b0-00-00-00, d4-dc-09-00-00-00, ac-23-16-00-00-00], SiteName=02-00430, WIDS=[], L2TPTunnels=[], SVIs=[{dhcp_enabled=true, DHCP_Ena=true, Vlan_ID=1, DHCP={MaxLeaseTime=43200, Hostname=02A00430M, Enabled=true}, MTU=0}], Radios=[{ProbeControl={ProbeReqSW=true, ProbeRespSW=true}, QOS={Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1}, {DSCP=8, AC=1}, {DSCP=16, AC=1}, {DSCP=9, AC=1}, {DSCP=11, AC=1}, {DSCP=13, AC=1}, {DSCP=15, AC=1}, {DSCP=17, AC=1}, {DSCP=19, AC=1}, {DSCP=21, AC=1}, {DSCP=23, AC=1}]}, PreambleLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=2.4, RRM={Width=20, Power=5, Channel=1}, Width=20, Power=5}, {ProbeControl={ProbeReqSW=true, ProbeRespSW=true}, QOS={Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1},

In [13]:
d2.saveAsSequenceFile("s3://mist-staging-jupyterhub-notebooks/jupyter/abhakta@juniper.net/emr-nb/data/scope_with_ap_configs_del13")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
d2 = sc.sequenceFile("s3://mist-staging-jupyterhub-notebooks/jupyter/abhakta@juniper.net/emr-nb/data/scope_with_ap_configs_del13").map(lambda r: json.loads(str(r[1])))
print(d2.take(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'ap': '5c5b358ec076', 'start_time': 1635001202303, 'config': '{"BootConfig":"{OurOUIs=[5c-5b-35-00-00-00, d4-20-b0-00-00-00, d4-dc-09-00-00-00, ac-23-16-00-00-00], SiteName=02-00430, WIDS=[], L2TPTunnels=[], SVIs=[{dhcp_enabled=true, DHCP_Ena=true, Vlan_ID=1, DHCP={MaxLeaseTime=43200, Hostname=02A00430M, Enabled=true}, MTU=0}], Radios=[{ProbeControl={ProbeReqSW=true, ProbeRespSW=true}, QOS={Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1}, {DSCP=8, AC=1}, {DSCP=16, AC=1}, {DSCP=9, AC=1}, {DSCP=11, AC=1}, {DSCP=13, AC=1}, {DSCP=15, AC=1}, {DSCP=17, AC=1}, {DSCP=19, AC=1}, {DSCP=21, AC=1}, {DSCP=23, AC=1}]}, PreambleLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=2.4, RRM={Width=20, Power=5, Channel=1}, Width=20, Power=5}, {ProbeControl={ProbeReqSW=true, ProbeRespSW=true}, QOS={Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1}, {DSCP=8, AC=1

In [3]:
d3 = sc.sequenceFile("s3://mist-staging-jupyterhub-notebooks/jupyter/abhakta@juniper.net/emr-nb/data/scope_with_ap_configs_1635417631_1635418213_withConfigMeta/").map(lambda r: json.loads(str(r[1])))
print(d3.take(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'start_time': 1635393746517, 'end_time': 1635415480408, 'detection_time': 1635396813231, 'modification_time': 1635415523000, 'occurrence': 51, 'batch_count': 13, 'event_name': 'connectivity_failure', 'event_type': 'failed_scope_authentication_configdiff', 'mist_only': False, 'enable_action': False, 'category': 'connectivity', 'site_id': '3f7ccc33-e2d9-43d2-b55d-fc0adba83233', 'org_id': '0f2adb1c-8843-448f-8753-91a3d2fdbd14', 'entity_id': '5c5b354e72ce--b4d1b405-14a6-4846-9d16-fcb6ec2b7829', 'entity_type': 'site', 'display_entity_type': 'site', 'detection_delay': 3066, 'details': {'features': {'clients_list': ['8789354a-8668-afe3-f2ac-e6a9e558aa89', 'be925267-877a-310f-e366-9bc3e6f26b7e', 'c4d4bd00-fbad-3207-8d33-70c240046f49', '207f7766-1add-97d8-6724-12fa1b041cab', 'a54b3891-e807-84f2-445b-138274191934', '32d4dd60-1a08-d433-5b7b-797c38858089'], 'roaming': 90, 'tot_fail_events': 1295, 'dur': 362.23151666666666, 'num_clients': 6, 'adj_roaming': 20, 'final_features_list': [7.1662659741

In [20]:
a

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'start_time': 1634774435713, 'end_time': 1635417904040, 'detection_time': 1634778615627, 'modification_time': 1635417933000, 'occurrence': 4882, 'batch_count': 607, 'event_name': 'connectivity_failure', 'event_type': 'failed_scope_authentication_configdiff', 'mist_only': False, 'enable_action': False, 'category': 'connectivity', 'site_id': '9f159831-e7e3-4b26-a481-a40170b0426f', 'org_id': '5fac827a-edbd-4522-83c6-de8d4a7ea380', 'entity_id': 'd420b0801b76', 'entity_type': 'site', 'display_entity_type': 'site', 'detection_delay': 4179, 'details': {'features': {'clients_list': ['950a83f3-a3c0-b700-e215-247502cdb3e3', 'd49c4f4f-1c13-e066-bb91-eef82cc4dd42', '2b71d5a3-773c-e0c4-3e97-2a31a092981f', '8e4a9bf5-398d-495c-2db5-e25cb7ea4a7a', 'ea2424cc-c66e-6e8b-0068-1dd5d4baae2e', '687d2a68-9495-e026-b035-cc43463165e1', 'ec1d8a5d-4450-8409-2a22-3ddc792cd6f5', '23067ca7-7140-4bcc-e8ce-f1f95cc4bb50', '59609b49-c5f6-0b42-4d63-9c9395b5afc5', 'ef8be831-daaf-77df-a885-a177429bb46a', '78a2ed64-40a7-cd

In [32]:
print(rdd_config.take(1)[0].keys())
print('---')
print(a.keys())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['ap', 'row_key', 'start_time', 'timestamp', 'row_number', 'config', 'config_step', 'config_change', 'time_delta', 'row_number_timedelta'])
---
dict_keys(['start_time', 'end_time', 'detection_time', 'modification_time', 'occurrence', 'batch_count', 'event_name', 'event_type', 'mist_only', 'enable_action', 'category', 'site_id', 'org_id', 'entity_id', 'entity_type', 'display_entity_type', 'detection_delay', 'details', 'row_key', 'client', 'scope', 'reason', 'severity', 'event_duration'])

In [35]:
dftemp2 = rdd_config.map(lambda x: (x['row_key'], x)).join(dftemp.map(lambda x: (x['row_key'], x)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
dftemp2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16

In [39]:
b = dftemp2.take(1)[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
len(b[1][0]), len(b[1][1])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(10, 24)

In [50]:
b[1][0]['ap'], b[1][1]['reason']['ap']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('5c5b358ec076', '5c5b358ec076')

In [56]:
b[1][1]['details']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'features': {'clients_list': ['ee2d9e83-9502-440a-653b-67b90bbe833b', '5d804d6b-e73c-4180-48a9-971c61bd588a', '72b52a02-a0f1-61e6-e085-43065d853e38', 'ffd811d9-ce57-d451-5866-493a0187e7d1', 'df1192ed-225c-9ca5-1ed6-c1c3e626fb76', '2e87eb62-ef69-c771-3f09-ea8dcefbe380', 'b6927dd6-647d-e2ec-952c-dee0f43663dd', 'b1f5c162-68d6-1a6a-53a1-f0f0c1305964', 'e5e39cfb-b948-5a0e-1f19-ee1853a02f0e', 'f7fa5298-564a-b4ab-4ee3-0666c6684a84', '2867417b-130e-106a-6aec-0070525761c3', '6f54dbc8-1937-df3d-eb31-9091649f5700', 'fc3be9bd-ba2c-0026-4bf1-879402d1305d', 'a7cbda4b-9def-147b-15d9-bb0783e74226', 'a2a53578-4233-ccd5-0afd-955ad0743844', '2c79e722-0921-ba71-a688-63b3079aa1c8', '1afdd26e-cda3-58d1-c8c4-308a3f53eb60', '1018b9f7-32c1-ca1f-ff7c-4f4e9071ab00', 'd2af0f07-c0c9-fe5f-f9d4-9d0947f9dca1', '9958c31c-28fa-103b-f85c-6e095111c1a7', '3dd2d00e-a8bc-1813-46b2-6fcd00858245', '1452a114-213d-1545-0815-16006b58c507', 'e87f4755-028c-eba3-6ee6-886d0ebf2c3e', '19f53efe-b49e-9864-f589-344bbe0cd04e', '0aefefc9

In [58]:
d1 = b[1][1]['details']
d1['apconfig_history'] = b[1][0]

print(d1.keys())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['features', 'impacted_subscope', 'symptom', 'event_history', 'apconfig_history'])

In [61]:
print(d1['apconfig_history']['config_change'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[[{'new': 'annel=36}, Width=20, Power=11}], PoePwrConstrained={EnableRadios=false}, AppDef=[], Timestamp=1.634980677811475E9, Name=02A00430M, _errors=[], ProxyArp={Enabled=true, Disable_RespSpoofCheck=true, Disable_ReqSpoof', 'old': 'annel=36}, Width=20, Power=11}], PoePwrConstrained={EnableRadios=false}, AppDef=[], Timestamp=1.6348964110093029E9, Name=02A00430M, _errors=[], ProxyArp={Enabled=true, Disable_RespSpoofCheck=true, Disable_ReqSpoo'}], [{'new': ', Width=20, Power=11}]","PoePwrConstrained":"{EnableRadios=true}","AppDef":"[]","Timestamp":"1.634980677811475E9","Name":"02A00430M","_errors":"[]","ProxyArp":"{Enabled=true, Disable_RespSpoofCheck=true, Disable_', 'old': '}, Width=20, Power=11}]","PoePwrConstrained":"{EnableRadios=true}","AppDef":"[]","Timestamp":"1.6348964110093029E9","Name":"02A00430M","_errors":"[]","ProxyArp":"{Enabled=true, Disable_RespSpoofCheck=true, Disabl'}], [{'new': 'IMXAQ== ndade2@mistsys.com], Disabled=false}","LED":"{Brightness=255}","time_taken":"{sta

In [62]:
def add_config_meta(x):
    x_key = x[0]
    x_val = x[1][1]  #original event info
    x_val_details = x_val['details']
    x_val_details['apconfig_history'] = x[1][0]  #info from config diff code
    x_val['details'] = x_val_details
    return (x, x_val)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
dftemp3 = dftemp2.map(lambda x: add_config_meta(x))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
c =dftemp3.take(1)[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
c[1]['details'].keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['features', 'impacted_subscope', 'symptom', 'event_history', 'apconfig_history'])

In [68]:
c[1]['details']['apconfig_history']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ap': '5c5b358ec076', 'row_key': '5c5b358ec076--84281ac7-4321-4db9-a893-bb520b976717&connectivity_failure&failed_scope_authentication&1635001202303', 'start_time': 1635001202303, 'timestamp': 1634980677000000, 'row_number': 8, 'config': '{"BootConfig":"{OurOUIs=[5c-5b-35-00-00-00, d4-20-b0-00-00-00, d4-dc-09-00-00-00, ac-23-16-00-00-00], SiteName=02-00430, WIDS=[], L2TPTunnels=[], SVIs=[{dhcp_enabled=true, DHCP_Ena=true, Vlan_ID=1, DHCP={MaxLeaseTime=43200, Hostname=02A00430M, Enabled=true}, MTU=0}], Radios=[{ProbeControl={ProbeReqSW=true, ProbeRespSW=true}, QOS={Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1}, {DSCP=8, AC=1}, {DSCP=16, AC=1}, {DSCP=9, AC=1}, {DSCP=11, AC=1}, {DSCP=13, AC=1}, {DSCP=15, AC=1}, {DSCP=17, AC=1}, {DSCP=19, AC=1}, {DSCP=21, AC=1}, {DSCP=23, AC=1}]}, PreambleLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=2.4, RRM={Width=20, Power=5, Channel=1}, Width=20, Power=5}, {Prob

In [69]:
print(a)

An error was encountered:
Invalid status code '404' from http://ip-172-31-56-200.ec2.internal:8998/sessions/613 with error payload: {"msg":"Session '613' not found."}


<h5>checks 2

In [37]:
# sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2021-12-08/hr=12/FailedScopeConfigDiff_1635418213.seq"
# sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2021-12-24/hr=08/FailedScopeConfigDiff_1640331519.seq"
sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2021-12-24/hr=09/FailedScopeConfigDiff_1640333996.seq"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
rdd0 = sc.sequenceFile(sloc).map(lambda r: json.loads(r[1])) #.persist()
print('\n',rdd0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 12

In [39]:
r1 = rdd0.take(1)
print(r1[0].keys())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['start_time', 'end_time', 'detection_time', 'modification_time', 'occurrence', 'batch_count', 'event_name', 'event_type', 'mist_only', 'enable_action', 'category', 'site_id', 'org_id', 'entity_id', 'entity_type', 'display_entity_type', 'detection_delay', 'details', 'row_key', 'client', 'scope', 'reason', 'severity', 'event_duration'])

In [42]:
r1[0]['details']['apconfig_history'].keys()#['row_number_timedelta']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['ap', 'row_key', 'start_time', 'timestamp', 'row_number', 'config', 'config_step', 'config_change', 'time_delta', 'row_number_timedelta'])

In [43]:
rdd0.toDF().select('row_key', F.col('details.apconfig_history.row_key').alias('row_key_config')).show(3,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------------------------------------------------+--------------+
|row_key                                                                                                          |row_key_config|
+-----------------------------------------------------------------------------------------------------------------+--------------+
|d420b0015e28--18b678bd-47a5-4f73-af1b-80a42808c470&connectivity_failure&failed_scope_authentication&1640293214871|null          |
|d420b0015e28--18b678bd-47a5-4f73-af1b-80a42808c470&connectivity_failure&failed_scope_authentication&1640293214871|null          |
|d420b0015e28--18b678bd-47a5-4f73-af1b-80a42808c470&connectivity_failure&failed_scope_authentication&1640293214871|null          |
+-----------------------------------------------------------------------------------------------------------------+--------------+
only showing top 3 rows

In [45]:
rdd1 = rdd0.map(lambda r: (r['row_key'], r['scope'], r['reason']['ap'], r['severity'],\
                           r['details']['apconfig_history']['row_key'], r['details']['apconfig_history']['row_number_timedelta'], \
                           r['details']['apconfig_history']['time_delta'],\
                           r['details']['apconfig_history']['config_change']\
                          ))
# rdd1.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
df1 = rdd1.toDF(schema=['row_key','scope', 'ap', 'severity', 'row_key_config','row_number_timedelta','time_delta', 'config_change'])
df1.show(4, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------------------------------------------------+-------+------------+-----------------+-----------------------------------------------------------------------------------------------------------------+--------------------+-----------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [47]:
print(df1.count())
print(df1.distinct().count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12
12

In [31]:
df1.select('row_number_timedelta').distinct().show(50,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|row_number_timedelta|
+--------------------+
|                  21|
|                   5|
|                   6|
|                   9|
|                   1|
|                  10|
|                   3|
|                  12|
|                  11|
|                   2|
|                  13|
|                  14|
|                  15|
|                  24|
|                   7|
|                   8|
|                   4|
+--------------------+

In [48]:
df1.filter('row_number_timedelta<=2').count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [49]:
df1.filter('row_number_timedelta<=2').select('row_key','scope', 'ap', 'severity','row_number_timedelta',F.col('time_delta')/(10000*60*60)).orderBy('row_key','row_number_timedelta').show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------------------------------------------------+-------+------------+------------------+--------------------+-----------------------+
|row_key                                                                                                          |scope  |ap          |severity          |row_number_timedelta|(time_delta / 36000000)|
+-----------------------------------------------------------------------------------------------------------------+-------+------------+------------------+--------------------+-----------------------+
|5c5b353e93f8&connectivity_failure&failed_scope_authentication&1640333416927                                      |ap     |5c5b353e93f8|68.78372980674531 |1                   |2.3688313055555557     |
|d420b0015e28--18b678bd-47a5-4f73-af1b-80a42808c470&connectivity_failure&failed_scope_authentication&1640293214871|ap_wlan|d420b0015e28|99.94352911823641 |1                   |1.8771630833333333  

In [50]:
df1.select('ap').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4

In [53]:
a = df1.select('ap','config_change').collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
for x_ in a:
    print('----')
    print(x_.ap)
    x__ = eval(x_.config_change)
    for e_ in x__:
        print('>>>')
        print(e_[0]['new'])
        print(e_[0]['old'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----
d420b0015e28
>>>
eLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=5, RRM={Width=20, Power=9, Channel=36}, Width=20, Power=9}], PoePwrConstrained={EnableRadios=false}, AppDef=[], Timestamp=1.640225636988513E9, Name=BOI2-10-00037, _errors=[Wlan[f99bd487-69ee-4eb1-a524-cb9ea54a7863] already has tunnel set t
eLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=5, RRM={Width=20, Power=8, Channel=36}, Width=20, Power=8}], PoePwrConstrained={EnableRadios=false}, AppDef=[], Timestamp=1.639882484301457E9, Name=BOI2-10-00037, _errors=[Wlan[f99bd487-69ee-4eb1-a524-cb9ea54a7863] already has tunnel set t
>>>
eLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=5, RRM={Width=20, Power=9, Channel=36}, Width=20, Power=9}]","PoePwrConstrained":"{EnableRadios=true}","AppDef":"[]","Timestamp":"1.640225636988513E9","Name":"BOI2-10-00037","_errors":"[Wlan[f99bd487-69ee-4eb1-a524-cb9ea54a7863] already has tunnel
eLength=1, NonAggrReg

<h5>checks 3

In [4]:
apfileLocations_6hr = ["s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=08/"] 
pullSchema2 = T.StructType([       
    T.StructField('ap', T.StringType(), True),
    T.StructField('ev_name', T.StringType(), True),
    T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),
    T.StructField('timestamp', T.LongType(), True),
])


for readLoc in sorted(apfileLocations_6hr):
    print('Reading from... %s' %readLoc)
    rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
    rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ev_name']=="AP_CONFIGURED") )
    df_apevents_sample = rdd_apevents_sample.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-11-15/hr=08/

In [10]:
[x['source']['config'] for x in rdd_apevents_sample.take(3)]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'Radios': [{'Band': '2.4', 'Width': 20, 'Power': 25, 'PreambleLength': 1, 'QOS': {'Dscp2Ac': [{'DSCP': 10, 'AC': 1}, {'DSCP': 12, 'AC': 1}, {'DSCP': 14, 'AC': 1}, {'DSCP': 18, 'AC': 1}, {'DSCP': 20, 'AC': 1}, {'DSCP': 22, 'AC': 1}, {'DSCP': 8, 'AC': 1}, {'DSCP': 16, 'AC': 1}, {'DSCP': 9, 'AC': 1}, {'DSCP': 11, 'AC': 1}, {'DSCP': 13, 'AC': 1}, {'DSCP': 15, 'AC': 1}, {'DSCP': 17, 'AC': 1}, {'DSCP': 19, 'AC': 1}, {'DSCP': 21, 'AC': 1}, {'DSCP': 23, 'AC': 1}]}, 'Channel': 0, 'RRM': {'Channel': 11, 'Width': 20, 'Power': 25}, 'NonAggrRegulate': {'Enabled': True, 'Disabled': False}, 'ProbeControl': {'ProbeReqSW': True, 'ProbeRespSW': True}}, {'Band': '5', 'Width': 40, 'Power': 25, 'PreambleLength': 1, 'QOS': {'Dscp2Ac': [{'DSCP': 10, 'AC': 1}, {'DSCP': 12, 'AC': 1}, {'DSCP': 14, 'AC': 1}, {'DSCP': 18, 'AC': 1}, {'DSCP': 20, 'AC': 1}, {'DSCP': 22, 'AC': 1}, {'DSCP': 8, 'AC': 1}, {'DSCP': 16, 'AC': 1}, {'DSCP': 9, 'AC': 1}, {'DSCP': 11, 'AC': 1}, {'DSCP': 13, 'AC': 1}, {'DSCP': 15, 'AC': 1}, 

In [4]:
df0 = spark.read.parquet("s3://mist-staging-jupyterhub-notebooks/jupyter/abhakta@juniper.net/emr-nb/data/scope_with_ap_configs_1635417631_1635418213/")
df0.persist()
print(df0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6

In [5]:
df0.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ap', 'row_key', 'start_time', 'timestamp', 'row_number', 'config', 'config_step', 'config_change', 'time_delta', 'row_number_timedelta']

In [6]:
df0.show(200,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+-------------+----------------+----------+--------------------+--------------------+--------------------+-----------+--------------------+
|          ap|             row_key|   start_time|       timestamp|row_number|              config|         config_step|       config_change| time_delta|row_number_timedelta|
+------------+--------------------+-------------+----------------+----------+--------------------+--------------------+--------------------+-----------+--------------------+
|5c5b358ec72e|5c5b358ec72e&conn...|1634864461950|1634798932000000|        13|{"BootConfig":"{O...|{"BootConfig":"{O...|[[{'new': 'el=44}...| 6.552995E7|                   1|
|5c5b354e72ce|5c5b354e72ce--b4d...|1635393746517|1635327037000000|         2|{"OurOUIs":"[5c-5...|{"OurOUIs":"[5c-5...|[[{'new': 'eLengt...|6.6709517E7|                   1|
|5c5b358ec076|5c5b358ec076--842...|1635001202303|1634980677000000|         8|{"BootConfig":"{O...|{"BootConfig":"{O...|[[{'new': '

In [8]:
df0 = sc.sequenceFile("s3://mist-staging-jupyterhub-notebooks/jupyter/abhakta@juniper.net/emr-nb/data/scope_with_ap_configs_1635417631_1635418213_withConfigMeta/")\
        .map(lambda r: json.loads(r[1]))
df0.persist()
print(df0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

214

In [12]:
df0.take(1)[0]['details'].keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['features', 'impacted_subscope', 'symptom', 'event_history', 'apconfig_history'])

In [14]:
r1 = df0.map(lambda r: (r['row_key'], r['details']['apconfig_history']['row_number_timedelta']))
dfr1 = r1.toDF(schema=['row_key', 'row_number_timedelta'])
dfr1.show(10,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|             row_key|row_number_timedelta|
+--------------------+--------------------+
|5c5b354e72ce--b4d...|                   1|
|5c5b354e72ce--b4d...|                   1|
|5c5b354e72ce--b4d...|                   1|
|5c5b354e72ce--b4d...|                   3|
|5c5b354e72ce--b4d...|                   2|
|5c5b354e72ce--b4d...|                   2|
|5c5b354e72ce--b4d...|                   3|
|5c5b354e72ce--b4d...|                   5|
|5c5b354e72ce--b4d...|                   5|
|5c5b354e72ce--b4d...|                   5|
+--------------------+--------------------+
only showing top 10 rows

In [15]:
dfr1.filter("row_number_timedelta==1").count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16

In [16]:
dfr1.filter("row_number_timedelta==1").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6

In [ ]:
# s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-13/hr=08/


<h4>Bulk analyze

In [38]:
var_date = 7
var_hr = 14
start_time = time.time()
print(start_time)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1642160597.3961055

In [39]:
s3locRoot_ee = 's3://mist-aggregated-stats-production/entity_event/'
s3locChild_ee = 'entity_event-production/'
dt_ee = "dt=2022-01-%s/" % str(var_date).zfill(2)
hr_ee = "hr=*/*/" #"hr=14/"
s3loc_ee = s3locRoot_ee + s3locChild_ee + dt_ee + hr_ee
print('\n--- s3loc_ee ---')
print(s3loc_ee)

event_rdd_ee = sc.sequenceFile(s3loc_ee).map(lambda r: json.loads(r[1]))#.persist()

event2consider = ['connectivity_failure']

event_rdd_ee_scope = event_rdd_ee.filter(lambda r: (('event_name' in r.keys() ) and (r['event_name'] in event2consider) \
                                                    and ('ap' in r['reason'].keys()) and (r['severity'] > 50) ) )
# print('\n',event_rdd_ee_scope.count()) 
# typically 250403 in 3 days for ['connectivity_failure', 'saturn_anomaly', 'preconnection_anomaly'], 1262 for 1 day in Dec 2021 for scope events with ap & severity >50

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-07/hr=*/*/

In [40]:
pull_schema = T.StructType([
    T.StructField('row_key', T.StringType(), True),
    T.StructField('start_time', T.LongType(), True),
    T.StructField('ap', T.StringType(), True)
])

df_select_scope_events_apscope = event_rdd_ee_scope.map(
    lambda r: (r['row_key'], r['start_time'], r['reason']['ap'])).distinct().toDF(pull_schema)

df_select_scope_events_apscope.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[row_key: string, start_time: bigint, ap: string]

In [41]:
ap_list = [x.ap for x in df_select_scope_events_apscope.select('ap').distinct().dropna().collect()]
print(len(ap_list))
print('Some ap ids: %s', str(ap_list[:10]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

169
Some ap ids: %s ['d420b04419bf', 'd420b08021fc', '5c5b35ce52ee', 'd420b000ad07', '5c5b35be0c26', '5c5b358e4272', 'd4dc0920d7b9', '5c5b358e40d3', '5c5b35aeec86', '5c5b35ce5f55']

In [42]:
# search for aps that had a config event in within the last 2 days | infact filter to aps within 6 hours of the scope event

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
strval = "dt=2022-01-{"
for x in range(var_date, var_date-2, -1):
    strval= strval + str(x).zfill(2) + ","
print(strval[:-1] +"}/")

dt_ap0 = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
hr_ap0 = "hr=*/*/"

readLoc0 = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap0 + hr_ap0
print(readLoc0)

pullSchema2 = T.StructType([       
    T.StructField('ap', T.StringType(), True),
    T.StructField('ev_name', T.StringType(), True),
    T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),
    T.StructField('timestamp', T.LongType(), True),
])


print('Reading from... %s' %readLoc0)
rdd_apevents0 = sc.sequenceFile(readLoc0).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample0 = rdd_apevents0.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list) and (r['source']['ev_name']=="AP_CONFIGURED") )
    
# print('\n',event_rdd_apevents_sampledSites.count())
df_apevents_sample0 = rdd_apevents_sample0.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)

dfm0 = df_select_scope_events_apscope.select('row_key', 'start_time', 'ap')\
                                        .join(df_apevents_sample0, on='ap', how='inner')\
                                        .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                        .filter('time_delta>0 and time_delta <' + str(6*60*60*1000))

ap_list_positive = [x.ap for x in dfm0.select('ap').distinct().dropna().collect()]
print(len(ap_list_positive))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dt=2022-01-{07,06}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{07,06}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{07,06}/hr=*/*/
18

In [44]:
strval = "dt=2022-01-{"
for x in range(var_date, var_date-8, -1):
    strval= strval + str(x).zfill(2) + ","
print(strval[:-1] +"}/")

dt_ap = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
hr_ap = "hr=*/*/"

readLoc = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap + hr_ap
print(readLoc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dt=2022-01-{07,06,05,04,03,02,01,00}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{07,06,05,04,03,02,01,00}/hr=*/*/

In [45]:
pullSchema2 = T.StructType([       
    T.StructField('ap', T.StringType(), True),
    T.StructField('ev_name', T.StringType(), True),
    T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),
    T.StructField('timestamp', T.LongType(), True),
])


print('Reading from... %s' %readLoc)
rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list_positive) and (r['source']['ev_name']=="AP_CONFIGURED") )
    
# print('\n',event_rdd_apevents_sampledSites.count())
df_apevents_sample = rdd_apevents_sample.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)
df_apevents_sample.printSchema()
df_apevents_sample.persist()
print(df_apevents_sample.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{07,06,05,04,03,02,01,00}/hr=*/*/
root
 |-- ap: string (nullable = true)
 |-- ev_name: string (nullable = true)
 |-- config: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- timestamp: long (nullable = true)

322

In [46]:
step_window = Window.partitionBy('ap').orderBy(F.col("row_number").desc())

df_apscope_withconfig = df_select_scope_events_apscope.join(df_apevents_sample, on ='ap', how='inner')\
                                .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                .filter('time_delta>0')\
                                .withColumn('row_number', F.row_number().over(Window.partitionBy('ap').orderBy(F.col('time_delta').asc())))

df_apscope_withconfig = df_apscope_withconfig.withColumn('config_step',
                                                         F.lag('config', offset=1).over(step_window))

df_apscope_withconfig.printSchema()
# df_apscope_withconfig.persist()
# df_apscope_withconfig.filter("ap=='5c5b35ae94ac'").orderBy('row_number').show(300, )

df_select_scope_events_apscope.unpersist()
df_apevents_sample.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ap: string (nullable = true)
 |-- row_key: string (nullable = true)
 |-- start_time: long (nullable = true)
 |-- ev_name: string (nullable = true)
 |-- config: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- timestamp: long (nullable = true)
 |-- time_delta: double (nullable = true)
 |-- row_number: integer (nullable = true)
 |-- config_step: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

DataFrame[ap: string, ev_name: string, config: map<string,string>, timestamp: bigint]

In [47]:
def getconfigdiff_def(x, y):
    if (x == "") or (y == ""):
        return "'config not found'"
    else:
        try:
            j1 = str(x)  # json.loads(x)
            j2 = str(y)  # json.loads(y)
            l1 = [x for x in difflib.unified_diff(j1, j2, n=100, lineterm='') if "@@" in x]
            retlist = list()
            for x_ in l1:
                sp = x_.replace("@", "").replace(" ", "").split("+")[1].split(",")
                s_ = int(sp[0])
                e_ = int(sp[1])
                if sp != []:
                    print(s_, e_)
                    retlist.append([{"new": j1[s_: s_ + e_ - 1], "old": j2[s_: s_ + e_ - 1]}])
            retval = str(retlist)
            return retval
        except:
            retval = "error"
        return retval

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
getconfigdiff = F.udf(getconfigdiff_def, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
df_apscope_withconfigdiff = df_apscope_withconfig.withColumn('config_json', F.to_json(F.col('config')))\
                                                .withColumn('config_step_json', F.to_json(F.col('config_step')) ) \
                                                .withColumn('config_change', getconfigdiff(F.col('config_json'), F.col('config_step_json')))\
                                                .filter("row_number==1")
df_apscope_withconfigdiff.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ap: string (nullable = true)
 |-- row_key: string (nullable = true)
 |-- start_time: long (nullable = true)
 |-- ev_name: string (nullable = true)
 |-- config: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- timestamp: long (nullable = true)
 |-- time_delta: double (nullable = true)
 |-- row_number: integer (nullable = true)
 |-- config_step: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- config_json: string (nullable = true)
 |-- config_step_json: string (nullable = true)
 |-- config_change: string (nullable = true)

In [50]:
t2 = time.time()
print(t2 - start_time)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

695.3396162986755

In [51]:
rdd_config = df_apscope_withconfigdiff.rdd.map(lambda x: x.asDict())
rdd_config.persist()
# print(rdd_config.count())
out_filename = "s3://mist-data-science-dev/aditya/scope_batch_v1/" + dt_ee
rdd_config.map(lambda x: (None, json.dumps(x))).saveAsSequenceFile(out_filename)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
t2 = time.time()
print(t2 - start_time)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1085.9772465229034

In [53]:
# 1027/60, 637/60 # (17.116666666666667, 10.616666666666667)
# 1034/60, 1635/60 # (17.233333333333334, 27.25)
# 655, 1013
# 740, 1175
695, 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
dt_ee

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'dt=2022-01-07/'

In [175]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [95]:
out_filename

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://mist-data-science-dev/aditya/scope_batch_v1/dt=2022-01-10/'

In [96]:
rdd0 = sc.sequenceFile(out_filename).map(lambda r: json.loads(r[1]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
rdd0.map(lambda r: (r['ap'], r['row_key'], r['config_change'])).take(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('5c5b35ae94ac', '5c5b35ae94ac--7e834870-3fc6-4247-8786-f83b1ac04bf0&connectivity_failure&failed_scope_authentication&1641820685490', '[[{\'new\': \'ained={EnableRadios=false}, AppDef=[{App=themeding.club, Hostname=themeding.club}], Timestamp=1.6418124177801602E9, Name=AP05-HER-RM-301/303-008956, _errors=[], ProxyArp={Enabled=true, Disable_RespSpoofCheck=tru\', \'old\': \'ained={EnableRadios=false}, AppDef=[{App=themeding.club, Hostname=themeding.club}], Timestamp=1.641799444477106E9, Name=AP05-HER-RM-301/303-008956, _errors=[], ProxyArp={Enabled=true, Disable_RespSpoofCheck=true\'}], [{\'new\': \'en=false, ID=ba6a1ead-5178-4400-bd21-3bf7c169005f, SSID=MCS-Guest, DisabledIEs={DisableHeIes=true}, roamMode=none, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=true, Ssdp_allow=false,\', \'old\': \'n=false, ID=ba6a1ead-5178-4400-bd21-3bf7c169005f, SSID=MCS-Guest, DisabledIEs={DisableHeIes=true}, BSS={Disabled=true}, roamMode=none, Enabled=true, BcastFilte

In [28]:
time.time()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1642095277.3158011

<h4>Prep validation

In [3]:
# sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2021-12-08/hr=12/FailedScopeConfigDiff_1635418213.seq"
# sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2021-12-24/hr=08/FailedScopeConfigDiff_1640331519.seq"
# sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2021-12-24/hr=09/FailedScopeConfigDiff_1640333996.seq"
sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2022-01-17/hr=23/FailedScopeConfigDiff_1642456425.seq"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
rdd0 = sc.sequenceFile(sloc).map(lambda r: json.loads(r[1])) #.persist()
print('\n',rdd0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 2

In [5]:
r1 = rdd0.take(1)
print(r1[0].keys())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['start_time', 'end_time', 'detection_time', 'modification_time', 'occurrence', 'batch_count', 'event_name', 'event_type', 'mist_only', 'enable_action', 'category', 'site_id', 'org_id', 'entity_id', 'entity_type', 'display_entity_type', 'detection_delay', 'details', 'row_key', 'client', 'scope', 'reason', 'severity', 'event_duration'])

In [6]:
r1[0]['details']['apconfig_history'].keys()#['row_number_timedelta']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['ap', 'row_key', 'start_time', 'timestamp', 'row_number', 'config', 'config_step', 'config_change', 'time_delta', 'row_number_timedelta'])

In [70]:
pullSchema3 = T.StructType([       
    T.StructField('ap', T.StringType(), True),
    T.StructField('row_key', T.StringType(), True),
    T.StructField('start_time', T.LongType(), True),
    T.StructField('timestamp', T.LongType(), True),
    T.StructField('time_delta', T.FloatType(), True),
    T.StructField('config_change', T.StringType(), True)
])

# T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
df1 = rdd0.map(lambda r: (r['details']['apconfig_history']['ap'],
                         r['details']['apconfig_history']['row_key'],
                         r['details']['apconfig_history']['start_time'],
                         r['details']['apconfig_history']['timestamp'],
                         r['details']['apconfig_history']['time_delta'],
                         r['details']['apconfig_history']['config_change'])).toDF(pullSchema3)
# rdd1.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
df1.show(4, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------------------------------------------------------------------------------------------------------------+-------------+----------------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [55]:
for x_ in a:
    print('----')
    print(x_.ap)
    x__ = eval(x_.config_change)
    for e_ in x__:
        print('>>>')
        print(e_[0]['new'])
        print(e_[0]['old'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----
d420b0015e28
>>>
eLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=5, RRM={Width=20, Power=9, Channel=36}, Width=20, Power=9}], PoePwrConstrained={EnableRadios=false}, AppDef=[], Timestamp=1.640225636988513E9, Name=BOI2-10-00037, _errors=[Wlan[f99bd487-69ee-4eb1-a524-cb9ea54a7863] already has tunnel set t
eLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=5, RRM={Width=20, Power=8, Channel=36}, Width=20, Power=8}], PoePwrConstrained={EnableRadios=false}, AppDef=[], Timestamp=1.639882484301457E9, Name=BOI2-10-00037, _errors=[Wlan[f99bd487-69ee-4eb1-a524-cb9ea54a7863] already has tunnel set t
>>>
eLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=5, RRM={Width=20, Power=9, Channel=36}, Width=20, Power=9}]","PoePwrConstrained":"{EnableRadios=true}","AppDef":"[]","Timestamp":"1.640225636988513E9","Name":"BOI2-10-00037","_errors":"[Wlan[f99bd487-69ee-4eb1-a524-cb9ea54a7863] already has tunnel
eLength=1, NonAggrReg

In [19]:
out_filename = "s3://mist-data-science-dev/aditya/scope_batch_aws_stg_deploy_v1/set1/"
df1.coalesce(1).write.mode("overwrite").option('header', 'True').option('delimiter', '\t').option("quote",'"').option("escape",',').csv(out_filename)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df1.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ap: string, row_key: string, start_time: bigint, timestamp: bigint, time_delta: float, config_change: string]

In [130]:
def i_udf1(x):
    x_ = eval(x)
    new_x_ = list()
    for y_ in x_:
        del_dict = dict()
        del_dict['new'] = y_[0]['new']
        del_dict['old'] = y_[0]['old']
        new_x_.append(del_dict)
    return new_x_


udf1 = F.udf(lambda r: i_udf1(r), T.ArrayType(T.MapType(T.StringType(), T.StringType())))

# df1.select('ap', (F.explode(udf1(F.col('config_change')))).alias('new_col') )\
# .select('ap', 'new_col.new', 'new_col.old')\
# .show(20,False)

df2 = df1.select('ap', (F.explode(udf1(F.col('config_change')))).alias('new_col') )\
.select('ap', F.explode('new_col'))

df2.show(20,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ap          |key|value                                                                                                                                                                                                                                                                                                                                                                                                                                

In [132]:
out_filename = "s3://mist-data-science-dev/aditya/scope_batch_aws_stg_deploy_v1/set1/"
df2.coalesce(1).write.mode("overwrite").option('header', 'True').option('delimiter', '\t').option("quote",'"').option("escape",',').csv(out_filename)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h5> #2

In [171]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2021-12-08/hr=12/FailedScopeConfigDiff_1635418213.seq"
# sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2021-12-24/hr=08/FailedScopeConfigDiff_1640331519.seq"
# sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2021-12-24/hr=09/FailedScopeConfigDiff_1640333996.seq"
sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2022-01-{13,14,15,16,17,18}/hr=*/*FailedScopeConfigDiff_*.seq"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
rdd0 = sc.sequenceFile(sloc).map(lambda r: json.loads(r[1])) #.persist()
print('\n',rdd0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 1834

In [17]:
r1 = rdd0.take(1)
print(r1[0].keys())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['start_time', 'end_time', 'detection_time', 'modification_time', 'occurrence', 'batch_count', 'event_name', 'event_type', 'mist_only', 'enable_action', 'category', 'site_id', 'org_id', 'entity_id', 'entity_type', 'display_entity_type', 'detection_delay', 'details', 'row_key', 'client', 'scope', 'reason', 'severity', 'event_duration'])

In [18]:
r1[0]['details']['apconfig_history'].keys()#['row_number_timedelta']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['ap', 'row_key', 'start_time', 'timestamp', 'row_number', 'config', 'config_step', 'config_change', 'time_delta', 'row_number_timedelta'])

In [176]:
pullSchema3 = T.StructType([       
    T.StructField('ap', T.StringType(), True),
    T.StructField('row_key', T.StringType(), True),
    T.StructField('start_time', T.LongType(), True),
    T.StructField('timestamp', T.LongType(), True),
    T.StructField('time_delta', T.FloatType(), True),
    T.StructField('config_change', T.StringType(), True)
])

# T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [177]:
df1 = rdd0.map(lambda r: (r['details']['apconfig_history']['ap'],
                         r['details']['apconfig_history']['row_key'],
                         r['details']['apconfig_history']['start_time'],
                         r['details']['apconfig_history']['timestamp'],
                         r['details']['apconfig_history']['time_delta'],
                         r['details']['apconfig_history']['config_change'])).toDF(pullSchema3)
# rdd1.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [178]:
df1.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ap: string, row_key: string, start_time: bigint, timestamp: bigint, time_delta: float, config_change: string]

In [179]:
def i_udf1(x):
    x_ = eval(x)
    new_x_ = list()
    for y_ in x_:
        del_dict = dict()
        del_dict['new'] = y_[0]['new']
        del_dict['old'] = y_[0]['old']
        new_x_.append(del_dict)
    return new_x_


udf1 = F.udf(lambda r: i_udf1(r), T.ArrayType(T.MapType(T.StringType(), T.StringType())))

# df1.select('ap', (F.explode(udf1(F.col('config_change')))).alias('new_col') )\
# .select('ap', 'new_col.new', 'new_col.old')\
# .show(20,False)

df2 = df1.select('ap', 'row_key', 'start_time', 'timestamp', 'time_delta', (F.explode(udf1(F.col('config_change')))).alias('new_col') )\
.select('ap', 'row_key', 'start_time', 'timestamp', 'time_delta', F.explode('new_col'))

df2.show(20,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+---------------------------------------------------------------------------+-------------+----------------+------------+---+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ap          |row_key                                                                    |

In [180]:
out_filename = "s3://mist-data-science-dev/aditya/scope_batch_aws_stg_deploy_v1/set3/"
df2.coalesce(1).write.mode("overwrite").option('header', 'True').option('delimiter', '\t').option("quote",'"').option("escape",',').csv(out_filename)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h5>#4

In [3]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2022-01-{13,14,15,16,17,18}/hr=*/*FailedScopeConfigDiff_*.seq"
sloc = "s3://mist-aggregated-stats-staging/entity_event/entity_event-staging/dt=2022-02-02/hr=17/FailedScopeConfigDiff_1643450359.seq"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
rdd0 = sc.sequenceFile(sloc).map(lambda r: json.loads(r[1])) #.persist()
print('\n',rdd0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 7681

In [6]:
r1 = rdd0.take(1)
print(r1[0].keys())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['start_time', 'end_time', 'detection_time', 'modification_time', 'occurrence', 'batch_count', 'event_name', 'event_type', 'mist_only', 'enable_action', 'category', 'site_id', 'org_id', 'entity_id', 'entity_type', 'display_entity_type', 'detection_delay', 'details', 'row_key', 'client', 'scope', 'reason', 'severity', 'event_duration'])

In [7]:
r1[0]['details']['apconfig_history'].keys()#['row_number_timedelta']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['ap', 'row_key', 'start_time', 'timestamp', 'row_number', 'config', 'config_step', 'config_change', 'time_delta', 'row_number_timedelta'])

In [8]:
pullSchema3 = T.StructType([       
    T.StructField('ap', T.StringType(), True),
    T.StructField('row_key', T.StringType(), True),
    T.StructField('start_time', T.LongType(), True),
    T.StructField('timestamp', T.LongType(), True),
    T.StructField('time_delta', T.FloatType(), True),
    T.StructField('config_change', T.StringType(), True)
])

# T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df1 = rdd0.map(lambda r: (r['details']['apconfig_history']['ap'],
                         r['details']['apconfig_history']['row_key'],
                         r['details']['apconfig_history']['start_time'],
                         r['details']['apconfig_history']['timestamp'],
                         r['details']['apconfig_history']['time_delta'],
                         r['details']['apconfig_history']['config_change'])).toDF(pullSchema3)
# rdd1.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df1.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ap: string, row_key: string, start_time: bigint, timestamp: bigint, time_delta: float, config_change: string]

In [11]:
def i_udf1(x):
    x_ = eval(x)
    new_x_ = list()
    for y_ in x_:
        del_dict = dict()
        del_dict['new'] = y_[0]['new']
        del_dict['old'] = y_[0]['old']
        new_x_.append(del_dict)
    return new_x_


udf1 = F.udf(lambda r: i_udf1(r), T.ArrayType(T.MapType(T.StringType(), T.StringType())))

# df1.select('ap', (F.explode(udf1(F.col('config_change')))).alias('new_col') )\
# .select('ap', 'new_col.new', 'new_col.old')\
# .show(20,False)

df2 = df1.select('ap', 'row_key', 'start_time', 'timestamp', 'time_delta', (F.explode(udf1(F.col('config_change')))).alias('new_col') )\
.select('ap', 'row_key', 'start_time', 'timestamp', 'time_delta', F.explode('new_col'))

df2.show(20,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------------------------------------------------------------------------------------------------------------+-------------+----------------+-----------+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ap          |row_key                                                                                                          |start_time   |timestamp       |time_delta |key|value                                                                                                          

In [180]:
# out_filename = "s3://mist-data-science-dev/aditya/scope_batch_aws_stg_deploy_v1/set3/"
# df2.coalesce(1).write.mode("overwrite").option('header', 'True').option('delimiter', '\t').option("quote",'"').option("escape",',').csv(out_filename)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h5> #3

In [19]:
rdd1 = rdd0.filter(lambda r: r['row_key']=="5c5b354ee8c1&connectivity_failure&failed_scope_authentication&1642194174178")
rdd1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3

In [20]:
rdd1.persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[22] at RDD at PythonRDD.scala:53

In [21]:
type((eval(rdd1.take(1)[0]['details']['apconfig_history']['config_step'])['BootConfig']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'str'>

In [22]:
(eval(rdd1.take(1)[0]['details']['apconfig_history']['config_step'])).keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['BootConfig', 'OurOUIs', 'SiteName', 'WIDS', 'L2TPTunnels', 'SVIs', 'Radios', 'PoePwrConstrained', 'AppDef', 'Timestamp', 'Name', '_errors', 'ProxyArp', 'OrgID', 'Monitor', 'CnaHostnames', 'RootAuthorizedSSHKeys', 'ID', 'UUID', 'WLANs', 'Dnsmasq', 'SwitchPorts', 'SiteString', 'PwrMgmt', 'SiteID', 'ExtIO', 'IrqBalance', 'APRecovery', 'Hostname', 'SSH', 'LED', 'time_taken', 'Steer', 'Stats', 'Country', 'HttpRedirector', 'VLANNames', 'AVS'])

In [23]:
v1 = rdd1.take(1)[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
l1 = ['OurOUIs', 'SiteName', 'WIDS', 'L2TPTunnels', 'SVIs', 'Radios', 'PoePwrConstrained', 'AppDef', 'Timestamp', 'Name', '_errors', 'ProxyArp', '\
OrgID', 'Monitor', 'CnaHostnames', 'RootAuthorizedSSHKeys', 'ID', 'UUID', 'WLANs', 'Dnsmasq', 'SwitchPorts', 'SiteString', 'PwrMgmt', 'SiteID', 'ExtIO', \
      'IrqBalance', 'APRecovery', 'Hostname', 'SSH', 'LED', 'time_taken', 'Steer', 'Stats', 'Country', 'HttpRedirector', 'VLANNames', 'AVS']

d0 = eval(v1['details']['apconfig_history']['config'])
d1 = eval(v1['details']['apconfig_history']['config_step'])
for x_ in l1:
    print(x_)
    print(d0[x_])
    print(d1[x_])
    print('---\n')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

OurOUIs
[5c-5b-35-00-00-00, d4-20-b0-00-00-00, d4-dc-09-00-00-00, ac-23-16-00-00-00, a8-53-7d-00-00-00]
[5c-5b-35-00-00-00, d4-20-b0-00-00-00, d4-dc-09-00-00-00, ac-23-16-00-00-00, a8-53-7d-00-00-00]
---

SiteName
Wadsworth High School
Wadsworth High School
---

WIDS
[]
[]
---

L2TPTunnels
[]
[]
---

SVIs
[{dhcp_enabled=true, DHCP_Ena=true, Vlan_ID=1, DHCP={MaxLeaseTime=43200, Hostname=AP111-WCS-HS-2-RM2116-61808, Enabled=true}, MTU=0}]
[{dhcp_enabled=true, DHCP_Ena=true, Vlan_ID=1, DHCP={MaxLeaseTime=43200, Hostname=AP111-WCS-HS-2-RM2116-61808, Enabled=true}, MTU=0}]
---

Radios
[{ProbeControl={ProbeReqSW=true, ProbeRespSW=true}, QOS={Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1}, {DSCP=8, AC=1}, {DSCP=16, AC=1}, {DSCP=9, AC=1}, {DSCP=11, AC=1}, {DSCP=13, AC=1}, {DSCP=15, AC=1}, {DSCP=17, AC=1}, {DSCP=19, AC=1}, {DSCP=21, AC=1}, {DSCP=23, AC=1}]}, PreambleLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0

In [34]:
w1 = json.loads(v1['details']['apconfig_history']['config'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h3>ap-events explode

In [3]:
readLoc0 = "s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{14}/hr={14}"
print(readLoc0)

print('Reading from... %s' %readLoc0)
rdd_apevents0 = sc.sequenceFile(readLoc0).map(lambda r: json.loads(r[1])) #.persist()

rdd_apevents_sample0 = rdd_apevents0.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ev_name']=="AP_CONFIGURED") )

# and (r['ap_id']=="5c-5b-35-4e-e8-c1") 

rdd_apevents_sample0.persist()    
print('\n',rdd_apevents_sample0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{14}/hr={14}
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{14}/hr={14}

 10628

In [4]:
r1 = rdd_apevents_sample0.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
r1[0].keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['event_type', 'site_id', 'org_id', 'timestamp', 'source', 'ap_id', 'device_type'])

In [6]:
r1[0]['source']['config'].keys()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['Radios', 'Country', 'Hostname', 'WLANs', 'SVIs', 'LED', 'SSH', 'RootAuthorizedSSHKeys', 'Stats', 'IrqBalance', 'OurOUIs', 'ID', 'UUID', '_errors', 'Timestamp', 'OrgID', 'SiteID', 'Name', 'SiteName', 'HttpRedirector', 'SiteString', 'ProxyArp', 'Steer', 'WIDS', 'CnaHostnames', 'Dnsmasq', 'AppDef', 'Monitor', 'ExtIO', 'PwrMgmt', 'PoePwrConstrained', 'AVS', 'SwitchPorts', 'L2TPTunnels', 'VLANNames', 'APRecovery', 'BootConfig', 'time_taken'])

In [46]:
for x_ in (r1[0]['source']['config']):
    print("%s \t\t\t\t %s" %(x_, type(r1[0]['source']['config'][x_])))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Radios 				 <class 'list'>
Country 				 <class 'str'>
Hostname 				 <class 'str'>
WLANs 				 <class 'list'>
SVIs 				 <class 'list'>
LED 				 <class 'dict'>
SSH 				 <class 'dict'>
RootAuthorizedSSHKeys 				 <class 'list'>
Stats 				 <class 'dict'>
IrqBalance 				 <class 'dict'>
OurOUIs 				 <class 'list'>
ID 				 <class 'str'>
UUID 				 <class 'str'>
_errors 				 <class 'list'>
Timestamp 				 <class 'float'>
OrgID 				 <class 'str'>
SiteID 				 <class 'str'>
Name 				 <class 'str'>
SiteName 				 <class 'str'>
HttpRedirector 				 <class 'dict'>
SiteString 				 <class 'str'>
ProxyArp 				 <class 'dict'>
Steer 				 <class 'dict'>
WIDS 				 <class 'list'>
CnaHostnames 				 <class 'list'>
Dnsmasq 				 <class 'dict'>
AppDef 				 <class 'list'>
Monitor 				 <class 'dict'>
ExtIO 				 <class 'list'>
PwrMgmt 				 <class 'dict'>
PoePwrConstrained 				 <class 'dict'>
AVS 				 <class 'dict'>
SwitchPorts 				 <class 'list'>
L2TPTunnels 				 <class 'list'>
VLANNames 				 <class 'dict'>
APRecovery 		

In [17]:
def unravel(d, counter=0, tag=''):
    counter+=1
    if type(d) is dict:
        for x_ in d:
            newtag = tag + '["' + str(x_) + '"]'
            print("level %d \t %s \t\t\t\t %s" %(counter, newtag, type(d[x_])))
            unravel(d[x_],counter, newtag)
    elif type(d) is list:
        try:
            unravel(d[0], counter-1, tag)
        except:
            print('not found')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
d1 = {1:{1:2}, 2:[1,2]}
unravel(d1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

level 1 	 ["1"] 				 <class 'dict'>
level 2 	 ["1"]["1"] 				 <class 'int'>
level 1 	 ["2"] 				 <class 'list'>

In [19]:
unravel(r1[0]['source']['config'],0,'["source"]["config"]')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

level 1 	 ["source"]["config"]["Radios"] 				 <class 'list'>
level 2 	 ["source"]["config"]["Radios"]["Band"] 				 <class 'str'>
level 2 	 ["source"]["config"]["Radios"]["Width"] 				 <class 'int'>
level 2 	 ["source"]["config"]["Radios"]["Power"] 				 <class 'int'>
level 2 	 ["source"]["config"]["Radios"]["PreambleLength"] 				 <class 'int'>
level 2 	 ["source"]["config"]["Radios"]["QOS"] 				 <class 'dict'>
level 3 	 ["source"]["config"]["Radios"]["QOS"]["Dscp2Ac"] 				 <class 'list'>
level 4 	 ["source"]["config"]["Radios"]["QOS"]["Dscp2Ac"]["DSCP"] 				 <class 'int'>
level 4 	 ["source"]["config"]["Radios"]["QOS"]["Dscp2Ac"]["AC"] 				 <class 'int'>
level 2 	 ["source"]["config"]["Radios"]["Channel"] 				 <class 'int'>
level 2 	 ["source"]["config"]["Radios"]["RRM"] 				 <class 'dict'>
level 3 	 ["source"]["config"]["Radios"]["RRM"]["Channel"] 				 <class 'int'>
level 3 	 ["source"]["config"]["Radios"]["RRM"]["Width"] 				 <class 'int'>
level 3 	 ["source"]["config"]["Radios"]["RRM

In [82]:
unravel(r1[0]['source']['config']["Radios"][0],0,'["source"]["config"]["Radios"]')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

level 1 	 ["source"]["config"]["Radios"]["Band"] 				 <class 'str'>
level 1 	 ["source"]["config"]["Radios"]["Width"] 				 <class 'int'>
level 1 	 ["source"]["config"]["Radios"]["Power"] 				 <class 'int'>
level 1 	 ["source"]["config"]["Radios"]["PreambleLength"] 				 <class 'int'>
level 1 	 ["source"]["config"]["Radios"]["QOS"] 				 <class 'dict'>
level 2 	 ["source"]["config"]["Radios"]["QOS"]["Dscp2Ac"] 				 <class 'list'>
level 1 	 ["source"]["config"]["Radios"]["Channel"] 				 <class 'int'>
level 1 	 ["source"]["config"]["Radios"]["RRM"] 				 <class 'dict'>
level 2 	 ["source"]["config"]["Radios"]["RRM"]["Channel"] 				 <class 'int'>
level 2 	 ["source"]["config"]["Radios"]["RRM"]["Width"] 				 <class 'int'>
level 2 	 ["source"]["config"]["Radios"]["RRM"]["Power"] 				 <class 'int'>
level 1 	 ["source"]["config"]["Radios"]["NonAggrRegulate"] 				 <class 'dict'>
level 2 	 ["source"]["config"]["Radios"]["NonAggrRegulate"]["Enabled"] 				 <class 'bool'>
level 2 	 ["source"]["config"

In [73]:
for x_ in (r1[0]['source']['config']["Radios"]):
    print(x_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'Band': '2.4', 'Width': 20, 'Power': 8, 'PreambleLength': 1, 'QOS': {'Dscp2Ac': [{'DSCP': 10, 'AC': 1}, {'DSCP': 12, 'AC': 1}, {'DSCP': 14, 'AC': 1}, {'DSCP': 18, 'AC': 1}, {'DSCP': 20, 'AC': 1}, {'DSCP': 22, 'AC': 1}, {'DSCP': 8, 'AC': 1}, {'DSCP': 16, 'AC': 1}, {'DSCP': 9, 'AC': 1}, {'DSCP': 11, 'AC': 1}, {'DSCP': 13, 'AC': 1}, {'DSCP': 15, 'AC': 1}, {'DSCP': 17, 'AC': 1}, {'DSCP': 19, 'AC': 1}, {'DSCP': 21, 'AC': 1}, {'DSCP': 23, 'AC': 1}]}, 'Channel': 0, 'RRM': {'Channel': 11, 'Width': 20, 'Power': 8}, 'NonAggrRegulate': {'Enabled': True, 'Disabled': False}, 'ProbeControl': {'ProbeReqSW': True, 'ProbeRespSW': True}}
{'Band': '5', 'Width': 40, 'Power': 11, 'PreambleLength': 1, 'QOS': {'Dscp2Ac': [{'DSCP': 10, 'AC': 1}, {'DSCP': 12, 'AC': 1}, {'DSCP': 14, 'AC': 1}, {'DSCP': 18, 'AC': 1}, {'DSCP': 20, 'AC': 1}, {'DSCP': 22, 'AC': 1}, {'DSCP': 8, 'AC': 1}, {'DSCP': 16, 'AC': 1}, {'DSCP': 9, 'AC': 1}, {'DSCP': 11, 'AC': 1}, {'DSCP': 13, 'AC': 1}, {'DSCP': 15, 'AC': 1}, {'DSCP': 17, 'AC

In [76]:
for x_ in (r1[0]["source"]["config"]["BootConfig"]["SSH"]["Disabled"]):
    print(x_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Disabled
External
AuthorizedKeys

In [80]:
r1[0]["source"]["config"]["SSH"]["Disabled"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

False

<h4>extract

In [106]:

def returnFields(r):
    fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]','r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]','r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]','r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]','r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']
    retval = list()
    for x_ in fields:
        try:
            retval.append(eval(x_))
        except:
            retval.append(None)
    return retval


# rdd_apevents_sample1 = rdd_apevents_sample0.map(lambda r: (r["source"]["config"]["Radios"][0]["Band"],r["source"]["config"]["Radios"][0]["Width"],r["source"]["config"]["Radios"][0]["Power"],r["source"]["config"]["Radios"][0]["PreambleLength"],r["source"]["config"]["Radios"][0]["Channel"],\
#                                                            r["source"]["config"]["Radios"][1]["Band"],r["source"]["config"]["Radios"][1]["Width"],r["source"]["config"]["Radios"][1]["Power"],r["source"]["config"]["Radios"][1]["PreambleLength"],r["source"]["config"]["Radios"][1]["Channel"],\
#                                                           r["source"]["config"]["Hostname"],\
#                                                           r["source"]["config"]["OurOUIs"],\
#                                                           r["source"]["config"]["Name"],r["source"]["config"]["SiteName"],r["source"]["config"]["HttpRedirector"],\
#                                                            r["source"]["config"]["SiteString"],r["source"]["config"]["ProxyArp"],r["source"]["config"]["Steer"],\
#                                                            r["source"]["config"]["WIDS"],r["source"]["config"]["CnaHostnames"],r["source"]["config"]["Dnsmasq"],\
#                                                            r["source"]["config"]["AppDef"],r["source"]["config"]["Monitor"],r["source"]["config"]["ExtIO"],\
#                                                            r["source"]["config"]["PoePwrConstrained"],r["source"]["config"]["AVS"],r["source"]["config"]["SwitchPorts"],r["source"]["config"]["L2TPTunnels"],r["source"]["config"]["VLANNames"],r["source"]["config"]["APRecovery"],r["source"]["config"]["time_taken"],))


rdd_apevents_sample1 = rdd_apevents_sample0.map(lambda r: returnFields(r))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
# rdd_apevents_sample1.take(1)
pullSchema4 = T.StructType([T.StructField("ap" , T.StringType() , True), T.StructField("timestamp" , T.LongType() , True), \
    T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),T.StructField("source_config_WLANs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_SVIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_LED" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SSH" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Stats" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_IrqBalance" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_OurOUIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_ID" , T.StringType() , True),T.StructField("source_config_UUID" , T.StringType() , True),T.StructField("source_config__errors" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Name" , T.StringType() , True),T.StructField("source_config_SiteName" , T.StringType() , True),T.StructField("source_config_HttpRedirector" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SiteString" , T.StringType() , True),T.StructField("source_config_ProxyArp" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Steer" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_WIDS" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_CnaHostnames" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Dnsmasq" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AppDef" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Monitor" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_ExtIO" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_PwrMgmt" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_PoePwrConstrained" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AVS" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SwitchPorts" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_L2TPTunnels" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_VLANNames" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_APRecovery" , T.MapType(T.StringType(),T.StringType()) , True)
])


df_i1 = rdd_apevents_sample1.toDF(pullSchema4)
df_i1.show(4,)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+--------------------+----------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------------+---------------------+-----------------+--------------------+---------------------+------------------+----------------------+----------------------------+------------------------+----------------------+--------------------+------------------+--------------------------+---------------------+--------------------+---------------------+-------------------+---------------------+-------------------------------+--------------------+-------------------------+-------------------------+-----------------------+------------------------+
|          ap|       timestamp|source_config_Radios|source_config_Hostname| source_config_WLANs|  source_config_SVIs|  source_config_LED|   source_config_SSH| source_config_Stats|source_config_IrqBalance|source_config_OurOUIs| source_config_ID|  source_config_UUID|

In [109]:
step_window = Window.partitionBy('ap').orderBy(F.col("timestamp"))

df_apscope_withconfig = df_i1.withColumn("source_config_Radios_step", F.lag("source_config_Radios", offset=1).over(step_window)).withColumn("source_config_Hostname_step", F.lag("source_config_Hostname", offset=1).over(step_window)).withColumn("source_config_WLANs_step", F.lag("source_config_WLANs", offset=1).over(step_window)).withColumn("source_config_SVIs_step", F.lag("source_config_SVIs", offset=1).over(step_window)).withColumn("source_config_LED_step", F.lag("source_config_LED", offset=1).over(step_window)).withColumn("source_config_SSH_step", F.lag("source_config_SSH", offset=1).over(step_window)).withColumn("source_config_Stats_step", F.lag("source_config_Stats", offset=1).over(step_window)).withColumn("source_config_IrqBalance_step", F.lag("source_config_IrqBalance", offset=1).over(step_window)).withColumn("source_config_OurOUIs_step", F.lag("source_config_OurOUIs", offset=1).over(step_window)).withColumn("source_config_ID_step", F.lag("source_config_ID", offset=1).over(step_window)).withColumn("source_config_UUID_step", F.lag("source_config_UUID", offset=1).over(step_window)).withColumn("source_config__errors_step", F.lag("source_config__errors", offset=1).over(step_window)).withColumn("source_config_Name_step", F.lag("source_config_Name", offset=1).over(step_window)).withColumn("source_config_SiteName_step", F.lag("source_config_SiteName", offset=1).over(step_window)).withColumn("source_config_HttpRedirector_step", F.lag("source_config_HttpRedirector", offset=1).over(step_window)).withColumn("source_config_SiteString_step", F.lag("source_config_SiteString", offset=1).over(step_window)).withColumn("source_config_ProxyArp_step", F.lag("source_config_ProxyArp", offset=1).over(step_window)).withColumn("source_config_Steer_step", F.lag("source_config_Steer", offset=1).over(step_window)).withColumn("source_config_WIDS_step", F.lag("source_config_WIDS", offset=1).over(step_window)).withColumn("source_config_CnaHostnames_step", F.lag("source_config_CnaHostnames", offset=1).over(step_window)).withColumn("source_config_Dnsmasq_step", F.lag("source_config_Dnsmasq", offset=1).over(step_window)).withColumn("source_config_AppDef_step", F.lag("source_config_AppDef", offset=1).over(step_window)).withColumn("source_config_Monitor_step", F.lag("source_config_Monitor", offset=1).over(step_window)).withColumn("source_config_ExtIO_step", F.lag("source_config_ExtIO", offset=1).over(step_window)).withColumn("source_config_PwrMgmt_step", F.lag("source_config_PwrMgmt", offset=1).over(step_window)).withColumn("source_config_PoePwrConstrained_step", F.lag("source_config_PoePwrConstrained", offset=1).over(step_window)).withColumn("source_config_AVS_step", F.lag("source_config_AVS", offset=1).over(step_window)).withColumn("source_config_SwitchPorts_step", F.lag("source_config_SwitchPorts", offset=1).over(step_window)).withColumn("source_config_L2TPTunnels_step", F.lag("source_config_L2TPTunnels", offset=1).over(step_window)).withColumn("source_config_VLANNames_step", F.lag("source_config_VLANNames", offset=1).over(step_window)).withColumn("source_config_APRecovery_step", F.lag("source_config_APRecovery", offset=1).over(step_window))

df_apscope_withconfig.select('ap', 'timestamp', 'source_config_Monitor','source_config_Monitor_step').filter("ap == '5c5b350e1c0c'").orderBy('timestamp').show(1000,)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+---------------------+--------------------------+
|          ap|       timestamp|source_config_Monitor|source_config_Monitor_step|
+------------+----------------+---------------------+--------------------------+
|5c5b350e1c0c|1642169944000000| {NonDFSIdleProbeA...|                      null|
|5c5b350e1c0c|1642170400000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642170415000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642170441000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642170688000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642170899000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642171172000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642171384000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642172126000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
+------------+--------------

In [110]:
def getconfigdiff_def(x, y):
    if (x == "") or (y == ""):
        return "'config not found'"
    else:
        try:
            j1 = str(x)  # json.loads(x)
            j2 = str(y)  # json.loads(y)
            l1 = [x for x in difflib.unified_diff(j1, j2, n=100, lineterm='') if "@@" in x]
            retlist = list()
            for x_ in l1:
                sp = x_.replace("@", "").replace(" ", "").split("+")[1].split(",")
                s_ = int(sp[0])
                e_ = int(sp[1])
                if sp != []:
                    print(s_, e_)
                    retlist.append([{"new": j1[s_: s_ + e_ - 1], "old": j2[s_: s_ + e_ - 1]}])
            retval = str(retlist)
            return retval
        except:
            retval = "error"
        return retval

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [111]:
getconfigDiff = F.udf(getconfigdiff_def, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
df_apscope_withconfigdiff = df_apscope_withconfig.withColumn("source_config_Radios_change", getconfigDiff( F.col("source_config_Radios"), F.col("source_config_Radios_step"))).withColumn("source_config_Hostname_change", getconfigDiff( F.col("source_config_Hostname"), F.col("source_config_Hostname_step"))).withColumn("source_config_WLANs_change", getconfigDiff( F.col("source_config_WLANs"), F.col("source_config_WLANs_step"))).withColumn("source_config_SVIs_change", getconfigDiff( F.col("source_config_SVIs"), F.col("source_config_SVIs_step"))).withColumn("source_config_LED_change", getconfigDiff( F.col("source_config_LED"), F.col("source_config_LED_step"))).withColumn("source_config_SSH_change", getconfigDiff( F.col("source_config_SSH"), F.col("source_config_SSH_step"))).withColumn("source_config_Stats_change", getconfigDiff( F.col("source_config_Stats"), F.col("source_config_Stats_step"))).withColumn("source_config_IrqBalance_change", getconfigDiff( F.col("source_config_IrqBalance"), F.col("source_config_IrqBalance_step"))).withColumn("source_config_OurOUIs_change", getconfigDiff( F.col("source_config_OurOUIs"), F.col("source_config_OurOUIs_step"))).withColumn("source_config_ID_change", getconfigDiff( F.col("source_config_ID"), F.col("source_config_ID_step"))).withColumn("source_config_UUID_change", getconfigDiff( F.col("source_config_UUID"), F.col("source_config_UUID_step"))).withColumn("source_config__errors_change", getconfigDiff( F.col("source_config__errors"), F.col("source_config__errors_step"))).withColumn("source_config_Name_change", getconfigDiff( F.col("source_config_Name"), F.col("source_config_Name_step"))).withColumn("source_config_SiteName_change", getconfigDiff( F.col("source_config_SiteName"), F.col("source_config_SiteName_step"))).withColumn("source_config_HttpRedirector_change", getconfigDiff( F.col("source_config_HttpRedirector"), F.col("source_config_HttpRedirector_step"))).withColumn("source_config_SiteString_change", getconfigDiff( F.col("source_config_SiteString"), F.col("source_config_SiteString_step"))).withColumn("source_config_ProxyArp_change", getconfigDiff( F.col("source_config_ProxyArp"), F.col("source_config_ProxyArp_step"))).withColumn("source_config_Steer_change", getconfigDiff( F.col("source_config_Steer"), F.col("source_config_Steer_step"))).withColumn("source_config_WIDS_change", getconfigDiff( F.col("source_config_WIDS"), F.col("source_config_WIDS_step"))).withColumn("source_config_CnaHostnames_change", getconfigDiff( F.col("source_config_CnaHostnames"), F.col("source_config_CnaHostnames_step"))).withColumn("source_config_Dnsmasq_change", getconfigDiff( F.col("source_config_Dnsmasq"), F.col("source_config_Dnsmasq_step"))).withColumn("source_config_AppDef_change", getconfigDiff( F.col("source_config_AppDef"), F.col("source_config_AppDef_step"))).withColumn("source_config_Monitor_change", getconfigDiff( F.col("source_config_Monitor"), F.col("source_config_Monitor_step"))).withColumn("source_config_ExtIO_change", getconfigDiff( F.col("source_config_ExtIO"), F.col("source_config_ExtIO_step"))).withColumn("source_config_PwrMgmt_change", getconfigDiff( F.col("source_config_PwrMgmt"), F.col("source_config_PwrMgmt_step"))).withColumn("source_config_PoePwrConstrained_change", getconfigDiff( F.col("source_config_PoePwrConstrained"), F.col("source_config_PoePwrConstrained_step"))).withColumn("source_config_AVS_change", getconfigDiff( F.col("source_config_AVS"), F.col("source_config_AVS_step"))).withColumn("source_config_SwitchPorts_change", getconfigDiff( F.col("source_config_SwitchPorts"), F.col("source_config_SwitchPorts_step"))).withColumn("source_config_L2TPTunnels_change", getconfigDiff( F.col("source_config_L2TPTunnels"), F.col("source_config_L2TPTunnels_step"))).withColumn("source_config_VLANNames_change", getconfigDiff( F.col("source_config_VLANNames"), F.col("source_config_VLANNames_step"))).withColumn("source_config_APRecovery_change", getconfigDiff( F.col("source_config_APRecovery"), F.col("source_config_APRecovery_step")))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
df_apscope_withconfigdiff.select('ap', 'timestamp', 'source_config_Radios','source_config_Radios_step','source_config_Radios_change')\
.filter("ap == '5c5b350e1c0c'").orderBy('timestamp').show(1000,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----

<h4>iter1

In [144]:
fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]','r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]','r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]','r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]','r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [154]:
[x_.replace("r[","").replace("\"][\"", "_").replace("]","").replace("\"","") for x_ in fields[2:]]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_config_Radios', 'source_config_Hostname', 'source_config_WLANs', 'source_config_SVIs', 'source_config_LED', 'source_config_SSH', 'source_config_Stats', 'source_config_IrqBalance', 'source_config_OurOUIs', 'source_config_ID', 'source_config_UUID', 'source_config__errors', 'source_config_Name', 'source_config_SiteName', 'source_config_HttpRedirector', 'source_config_SiteString', 'source_config_ProxyArp', 'source_config_Steer', 'source_config_WIDS', 'source_config_CnaHostnames', 'source_config_Dnsmasq', 'source_config_AppDef', 'source_config_Monitor', 'source_config_ExtIO', 'source_config_PwrMgmt', 'source_config_PoePwrConstrained', 'source_config_AVS', 'source_config_SwitchPorts', 'source_config_L2TPTunnels', 'source_config_VLANNames', 'source_config_APRecovery']

In [158]:
L1 = [x_.replace("r[","").replace("\"][\"", "_").replace("]","").replace("\"","")+ '_step' for x_ in fields[2:]]
L1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_config_Radios_step', 'source_config_Hostname_step', 'source_config_WLANs_step', 'source_config_SVIs_step', 'source_config_LED_step', 'source_config_SSH_step', 'source_config_Stats_step', 'source_config_IrqBalance_step', 'source_config_OurOUIs_step', 'source_config_ID_step', 'source_config_UUID_step', 'source_config__errors_step', 'source_config_Name_step', 'source_config_SiteName_step', 'source_config_HttpRedirector_step', 'source_config_SiteString_step', 'source_config_ProxyArp_step', 'source_config_Steer_step', 'source_config_WIDS_step', 'source_config_CnaHostnames_step', 'source_config_Dnsmasq_step', 'source_config_AppDef_step', 'source_config_Monitor_step', 'source_config_ExtIO_step', 'source_config_PwrMgmt_step', 'source_config_PoePwrConstrained_step', 'source_config_AVS_step', 'source_config_SwitchPorts_step', 'source_config_L2TPTunnels_step', 'source_config_VLANNames_step', 'source_config_APRecovery_step']

In [159]:
[c.replace("_step", "").replace("\"", "") for c in L1]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_config_Radios', 'source_config_Hostname', 'source_config_WLANs', 'source_config_SVIs', 'source_config_LED', 'source_config_SSH', 'source_config_Stats', 'source_config_IrqBalance', 'source_config_OurOUIs', 'source_config_ID', 'source_config_UUID', 'source_config__errors', 'source_config_Name', 'source_config_SiteName', 'source_config_HttpRedirector', 'source_config_SiteString', 'source_config_ProxyArp', 'source_config_Steer', 'source_config_WIDS', 'source_config_CnaHostnames', 'source_config_Dnsmasq', 'source_config_AppDef', 'source_config_Monitor', 'source_config_ExtIO', 'source_config_PwrMgmt', 'source_config_PoePwrConstrained', 'source_config_AVS', 'source_config_SwitchPorts', 'source_config_L2TPTunnels', 'source_config_VLANNames', 'source_config_APRecovery']

In [165]:
df_apscope_withconfig = df_i1.select('ap', 'timestamp', *[c.replace("_step", "").replace("\"", "") for c in L1], *[F.lag((c.replace("_step", "").replace("\"", "")), offset=1).over(step_window).alias(c) for c in L1] )

df_apscope_withconfig.select('ap', 'timestamp', 'source_config_Monitor','source_config_Monitor_step').filter("ap == '5c5b350e1c0c'").orderBy('timestamp').show(1000,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+---------------------+--------------------------+
|          ap|       timestamp|source_config_Monitor|source_config_Monitor_step|
+------------+----------------+---------------------+--------------------------+
|5c5b350e1c0c|1642169944000000| {NonDFSIdleProbeA...|                      null|
|5c5b350e1c0c|1642170400000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642170415000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642170441000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642170688000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642170899000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642171172000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642171384000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
|5c5b350e1c0c|1642172126000000| {NonDFSIdleProbeA...|      {NonDFSIdleProbeA...|
+------------+--------------

In [ ]:
df_apscope_withconfigdiff = df_apscope_withconfig.withColumn("source_config_Radios_change", getconfigDiff( F.col("source_config_Radios"), F.col("source_config_Radios_step")))

In [172]:
L0 = df_apscope_withconfig.columns
print(L0)

print('--')
print(L1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ap', 'timestamp', 'source_config_Radios', 'source_config_Hostname', 'source_config_WLANs', 'source_config_SVIs', 'source_config_LED', 'source_config_SSH', 'source_config_Stats', 'source_config_IrqBalance', 'source_config_OurOUIs', 'source_config_ID', 'source_config_UUID', 'source_config__errors', 'source_config_Name', 'source_config_SiteName', 'source_config_HttpRedirector', 'source_config_SiteString', 'source_config_ProxyArp', 'source_config_Steer', 'source_config_WIDS', 'source_config_CnaHostnames', 'source_config_Dnsmasq', 'source_config_AppDef', 'source_config_Monitor', 'source_config_ExtIO', 'source_config_PwrMgmt', 'source_config_PoePwrConstrained', 'source_config_AVS', 'source_config_SwitchPorts', 'source_config_L2TPTunnels', 'source_config_VLANNames', 'source_config_APRecovery', 'source_config_Radios_step', 'source_config_Hostname_step', 'source_config_WLANs_step', 'source_config_SVIs_step', 'source_config_LED_step', 'source_config_SSH_step', 'source_config_Stats_step', 'sour

In [168]:
[c.replace("_step", "_change") for c in L1]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_config_Radios_change', 'source_config_Hostname_change', 'source_config_WLANs_change', 'source_config_SVIs_change', 'source_config_LED_change', 'source_config_SSH_change', 'source_config_Stats_change', 'source_config_IrqBalance_change', 'source_config_OurOUIs_change', 'source_config_ID_change', 'source_config_UUID_change', 'source_config__errors_change', 'source_config_Name_change', 'source_config_SiteName_change', 'source_config_HttpRedirector_change', 'source_config_SiteString_change', 'source_config_ProxyArp_change', 'source_config_Steer_change', 'source_config_WIDS_change', 'source_config_CnaHostnames_change', 'source_config_Dnsmasq_change', 'source_config_AppDef_change', 'source_config_Monitor_change', 'source_config_ExtIO_change', 'source_config_PwrMgmt_change', 'source_config_PoePwrConstrained_change', 'source_config_AVS_change', 'source_config_SwitchPorts_change', 'source_config_L2TPTunnels_change', 'source_config_VLANNames_change', 'source_config_APRecovery_change']

In [174]:
df_apscope_withconfigdiff = df_apscope_withconfig.select(*[c for c in L0], *[getconfigDiff(F.col(c.replace("_step","")), F.col(c) ).alias(c.replace("_step","_change")) for c in L1] )


df_apscope_withconfigdiff.select('ap', 'timestamp', 'source_config_Radios','source_config_Radios_step','source_config_Radios_change')\
.filter("ap == '5c5b350e1c0c'").orderBy('timestamp').show(1000,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----

<h3>Bulk #2

In [3]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
var_date = 7
var_hr = 14
start_time = time.time()
print(start_time)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1643104894.943473

In [5]:
s3locRoot_ee = 's3://mist-aggregated-stats-production/entity_event/'
s3locChild_ee = 'entity_event-production/'
dt_ee = "dt=2022-01-%s/" % str(var_date).zfill(2)
hr_ee = "hr=14/" # "hr=*/*/" 
s3loc_ee = s3locRoot_ee + s3locChild_ee + dt_ee + hr_ee
print('\n--- s3loc_ee ---')
print(s3loc_ee)

event_rdd_ee = sc.sequenceFile(s3loc_ee).map(lambda r: json.loads(r[1]))#.persist()

event2consider = ['connectivity_failure']

event_rdd_ee_scope = event_rdd_ee.filter(lambda r: (('event_name' in r.keys() ) and (r['event_name'] in event2consider) \
                                                    and ('ap' in r['reason'].keys()) and (r['severity'] > 50) ) )
# print('\n',event_rdd_ee_scope.count()) 
# typically 250403 in 3 days for ['connectivity_failure', 'saturn_anomaly', 'preconnection_anomaly'], 1262 for 1 day in Dec 2021 for scope events with ap & severity >50

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-07/hr=14/

In [6]:
pull_schema = T.StructType([
    T.StructField('row_key', T.StringType(), True),
    T.StructField('start_time', T.LongType(), True),
    T.StructField('ap', T.StringType(), True)
])

df_select_scope_events_apscope = event_rdd_ee_scope.map(
    lambda r: (r['row_key'], r['start_time'], r['reason']['ap'])).distinct().toDF(pull_schema)

df_select_scope_events_apscope.persist()
print(df_select_scope_events_apscope.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

69

In [7]:
ap_list = [x.ap for x in df_select_scope_events_apscope.select('ap').distinct().dropna().collect()]
print(len(ap_list))
print('Some ap ids: %s', str(ap_list[:10]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

69
Some ap ids: %s ['5c5b35ce52ee', 'd420b08021fc', 'd420b04419bf', '5c5b35aeec86', 'd4dc09212b24', '5c5b35ce5f55', '5c5b358e7eea', '5c5b358e40d3', '5c5b358e730a', '5c5b35aec2ce']

In [8]:
strval = "dt=2022-01-{"
for x in range(var_date, var_date-2, -1):
    strval= strval + str(x).zfill(2) + ","
print(strval[:-1] +"}/")

dt_ap0 = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
hr_ap0 = "hr=*/*/"

readLoc0 = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap0 + hr_ap0
print(readLoc0)

pullSchema2 = T.StructType([       
    T.StructField('ap', T.StringType(), True),
    T.StructField('ev_name', T.StringType(), True),
    T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),
    T.StructField('timestamp', T.LongType(), True),
])


print('Reading from... %s' %readLoc0)
rdd_apevents0 = sc.sequenceFile(readLoc0).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample0 = rdd_apevents0.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list) and (r['source']['ev_name']=="AP_CONFIGURED") )
    
# print('\n',event_rdd_apevents_sampledSites.count())


# rdd_apevents_sample1 = rdd_apevents_sample0.map(lambda r: returnFields(r))



# pullSchema4 = T.StructType([T.StructField("ap" , T.StringType() , True), T.StructField("timestamp" , T.LongType() , True), \
#     T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),T.StructField("source_config_WLANs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_SVIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_LED" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SSH" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Stats" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_IrqBalance" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_OurOUIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_ID" , T.StringType() , True),T.StructField("source_config_UUID" , T.StringType() , True),T.StructField("source_config__errors" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Name" , T.StringType() , True),T.StructField("source_config_SiteName" , T.StringType() , True),T.StructField("source_config_HttpRedirector" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SiteString" , T.StringType() , True),T.StructField("source_config_ProxyArp" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Steer" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_WIDS" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_CnaHostnames" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Dnsmasq" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AppDef" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Monitor" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_ExtIO" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_PwrMgmt" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_PoePwrConstrained" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AVS" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SwitchPorts" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_L2TPTunnels" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_VLANNames" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_APRecovery" , T.MapType(T.StringType(),T.StringType()) , True)
# ])


# df_apevents_sample0 = rdd_apevents_sample1.toDF(pullSchema4)
# df_i1 = df_apevents_sample0



df_apevents_sample0 = rdd_apevents_sample0.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)

dfm0 = df_select_scope_events_apscope.select('row_key', 'start_time', 'ap')\
                                        .join(df_apevents_sample0, on='ap', how='inner')\
                                        .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                        .filter('time_delta>0 and time_delta <' + str(6*60*60*1000))

ap_list_positive = [x.ap for x in dfm0.select('ap').distinct().dropna().collect()]
print(len(ap_list_positive))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dt=2022-01-{07,06}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{07,06}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{07,06}/hr=*/*/
5

In [9]:
def returnFields(r):
    fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]','r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]','r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]','r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]','r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']
    retval = list()
    for x_ in fields:
        try:
            retval.append(eval(x_))
        except:
            retval.append(None)
    return retval


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
strval = "dt=2022-01-{"
for x in range(var_date, var_date-8, -1):
    strval= strval + str(x).zfill(2) + ","
print(strval[:-1] +"}/")

dt_ap = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
hr_ap = "hr=*/*/"

readLoc = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap + hr_ap
print(readLoc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dt=2022-01-{07,06,05,04,03,02,01,00}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{07,06,05,04,03,02,01,00}/hr=*/*/

In [18]:
# pullSchema2 = T.StructType([       
#     T.StructField('ap', T.StringType(), True),
#     T.StructField('ev_name', T.StringType(), True),
#     T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),
#     T.StructField('timestamp', T.LongType(), True),
# ])


print('Reading from... %s' %readLoc)
rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list_positive) and (r['source']['ev_name']=="AP_CONFIGURED") )
rdd_apevents_sample1 = rdd_apevents_sample.map(lambda r: returnFields(r))

pullSchema4 = T.StructType([T.StructField("ap" , T.StringType() , True), T.StructField("timestamp" , T.LongType() , True), \
    T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),T.StructField("source_config_WLANs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_SVIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_LED" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SSH" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Stats" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_IrqBalance" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_OurOUIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_ID" , T.StringType() , True),T.StructField("source_config_UUID" , T.StringType() , True),T.StructField("source_config__errors" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Name" , T.StringType() , True),T.StructField("source_config_SiteName" , T.StringType() , True),T.StructField("source_config_HttpRedirector" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SiteString" , T.StringType() , True),T.StructField("source_config_ProxyArp" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Steer" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_WIDS" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_CnaHostnames" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Dnsmasq" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AppDef" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Monitor" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_ExtIO" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_PwrMgmt" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_PoePwrConstrained" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AVS" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SwitchPorts" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_L2TPTunnels" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_VLANNames" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_APRecovery" , T.MapType(T.StringType(),T.StringType()) , True)
])

df_apevents_sample = rdd_apevents_sample1.toDF(pullSchema4)



# print('\n',event_rdd_apevents_sampledSites.count())
# df_apevents_sample = rdd_apevents_sample.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)
df_apevents_sample.printSchema()
df_apevents_sample.persist()
print(df_apevents_sample.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{07,06,05,04,03,02,01,00}/hr=*/*/
root
 |-- ap: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- source_config_Radios: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- source_config_Hostname: string (nullable = true)
 |-- source_config_WLANs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- source_config_SVIs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- source_config_LED: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- source_config_SSH: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- source_config_Stats: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- source_config_IrqBalance: map (nullable = true)
 |    |-- key: string
 |    |-- value: s

In [19]:
fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]','r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]','r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]','r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]','r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']

L1 = [x_.replace("r[","").replace("\"][\"", "_").replace("]","").replace("\"","")+ '_step' for x_ in fields[2:]]
print(L1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_config_Radios_step', 'source_config_Hostname_step', 'source_config_WLANs_step', 'source_config_SVIs_step', 'source_config_LED_step', 'source_config_SSH_step', 'source_config_Stats_step', 'source_config_IrqBalance_step', 'source_config_OurOUIs_step', 'source_config_ID_step', 'source_config_UUID_step', 'source_config__errors_step', 'source_config_Name_step', 'source_config_SiteName_step', 'source_config_HttpRedirector_step', 'source_config_SiteString_step', 'source_config_ProxyArp_step', 'source_config_Steer_step', 'source_config_WIDS_step', 'source_config_CnaHostnames_step', 'source_config_Dnsmasq_step', 'source_config_AppDef_step', 'source_config_Monitor_step', 'source_config_ExtIO_step', 'source_config_PwrMgmt_step', 'source_config_PoePwrConstrained_step', 'source_config_AVS_step', 'source_config_SwitchPorts_step', 'source_config_L2TPTunnels_step', 'source_config_VLANNames_step', 'source_config_APRecovery_step']

In [20]:
ap_list_positive

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['5c5b35aeec86', 'd4dc09212b24', '5c5b35af127e', '5c5b350e3f84', '5c5b352eeaa6']

In [21]:
df_select_scope_events_apscope.columns, df_apevents_sample.columns


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(['row_key', 'start_time', 'ap'], ['ap', 'timestamp', 'source_config_Radios', 'source_config_Hostname', 'source_config_WLANs', 'source_config_SVIs', 'source_config_LED', 'source_config_SSH', 'source_config_Stats', 'source_config_IrqBalance', 'source_config_OurOUIs', 'source_config_ID', 'source_config_UUID', 'source_config__errors', 'source_config_Name', 'source_config_SiteName', 'source_config_HttpRedirector', 'source_config_SiteString', 'source_config_ProxyArp', 'source_config_Steer', 'source_config_WIDS', 'source_config_CnaHostnames', 'source_config_Dnsmasq', 'source_config_AppDef', 'source_config_Monitor', 'source_config_ExtIO', 'source_config_PwrMgmt', 'source_config_PoePwrConstrained', 'source_config_AVS', 'source_config_SwitchPorts', 'source_config_L2TPTunnels', 'source_config_VLANNames', 'source_config_APRecovery'])

In [22]:
step_window = Window.partitionBy('ap').orderBy(F.col("row_number").desc())

df_apscope_withconfig = df_select_scope_events_apscope.join(df_apevents_sample, on ='ap', how='inner')\
                                .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                .filter('time_delta>0')\
                                .withColumn('row_number', F.row_number().over(Window.partitionBy('ap').orderBy(F.col('time_delta').asc())))

df_apscope_withconfig = df_apscope_withconfig.select('ap', 'timestamp','row_key', 'start_time','time_delta', 'row_number',\
                                                     *[c.replace("_step", "").replace("\"", "") for c in L1], \
                                                     *[F.lag((c.replace("_step", "").replace("\"", "")), offset=1).over(step_window).alias(c) for c in L1],\
                                                    F.lag('timestamp', offset=1).over(step_window).alias('timestamp_step'))


# df_apscope_withconfig.printSchema()
# df_apscope_withconfig.select('ap', 'timestamp','timestamp_step', 'row_key', 'start_time','time_delta', 'row_number',\
#                              'source_config_Monitor','source_config_Monitor_step')\
# .filter("ap == '5c5b35aeec86'").orderBy('timestamp').show(1000,)
# df_select_scope_events_apscope.unpersist()
# df_apevents_sample.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
L0 = df_apscope_withconfig.columns
print(L0)
print('--')
print(L1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ap', 'timestamp', 'row_key', 'start_time', 'time_delta', 'row_number', 'source_config_Radios', 'source_config_Hostname', 'source_config_WLANs', 'source_config_SVIs', 'source_config_LED', 'source_config_SSH', 'source_config_Stats', 'source_config_IrqBalance', 'source_config_OurOUIs', 'source_config_ID', 'source_config_UUID', 'source_config__errors', 'source_config_Name', 'source_config_SiteName', 'source_config_HttpRedirector', 'source_config_SiteString', 'source_config_ProxyArp', 'source_config_Steer', 'source_config_WIDS', 'source_config_CnaHostnames', 'source_config_Dnsmasq', 'source_config_AppDef', 'source_config_Monitor', 'source_config_ExtIO', 'source_config_PwrMgmt', 'source_config_PoePwrConstrained', 'source_config_AVS', 'source_config_SwitchPorts', 'source_config_L2TPTunnels', 'source_config_VLANNames', 'source_config_APRecovery', 'source_config_Radios_step', 'source_config_Hostname_step', 'source_config_WLANs_step', 'source_config_SVIs_step', 'source_config_LED_step', 'sourc

In [28]:
def getconfigdiff_def(x, y):
    if (x == "") or (y == ""):
        return "'config not found'"
    else:
        try:
            j1 = str(x)  # json.loads(x)
            j2 = str(y)  # json.loads(y)
            l1 = [x for x in difflib.unified_diff(j1, j2, n=100, lineterm='') if "@@" in x]
            retlist = list()
            for x_ in l1:
                sp = x_.replace("@", "").replace(" ", "").split("+")[1].split(",")
                s_ = int(sp[0])
                e_ = int(sp[1])
                if sp != []:
                    print(s_, e_)
                    retlist.append([{"new": j1[s_: s_ + e_ - 1], "old": j2[s_: s_ + e_ - 1]}])
            retval = str(retlist)
            return retval
        except:
            retval = "error"
        return retval

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
getconfigDiff = F.udf(getconfigdiff_def, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df_apscope_withconfigdiff = df_apscope_withconfig.select(*[c for c in L0], *[getconfigDiff(F.col(c.replace("_step","")), F.col(c) ).alias(c.replace("_step","_change")) for c in L1] )


# df_apscope_withconfigdiff.select('ap', 'timestamp','row_key', 'start_time','time_delta', 'row_number',\
#                                  'source_config_Radios','source_config_Radios_step','source_config_Radios_change')\
# .filter("ap == '5c5b35aeec86'").orderBy('timestamp').show(1000,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [198]:
df_apscope_withconfigdiff.select('ap', 'timestamp','row_key', 'start_time','time_delta', 'row_number',\
                                 'source_config_Radios','source_config_Radios_step','source_config_Radios_change')\
.filter("ap == '5c5b35aeec86' and row_number==1").select('source_config_Radios', 'source_config_Radios_step').take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(source_config_Radios=['{ProbeControl={ProbeReqSW=true, ProbeRespSW=true}, QOS={Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1}, {DSCP=8, AC=1}, {DSCP=16, AC=1}, {DSCP=9, AC=1}, {DSCP=11, AC=1}, {DSCP=13, AC=1}, {DSCP=15, AC=1}, {DSCP=17, AC=1}, {DSCP=19, AC=1}, {DSCP=21, AC=1}, {DSCP=23, AC=1}]}, PreambleLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=2.4, RRM={Width=20, Power=8, Channel=1}, Width=20, Power=8}', '{ProbeControl={ProbeReqSW=true, ProbeRespSW=true}, QOS={Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1}, {DSCP=8, AC=1}, {DSCP=16, AC=1}, {DSCP=9, AC=1}, {DSCP=11, AC=1}, {DSCP=13, AC=1}, {DSCP=15, AC=1}, {DSCP=17, AC=1}, {DSCP=19, AC=1}, {DSCP=21, AC=1}, {DSCP=23, AC=1}]}, PreambleLength=1, NonAggrRegulate={Disabled=false, Enabled=true}, Channel=0, Band=5, RRM={Width=40, Power=9, Channel=132}, Width=40, Power=9}'], source_c

In [207]:
[c for c in df_apscope_withconfigdiff.columns if "_change" in c ]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_config_Radios_change', 'source_config_Hostname_change', 'source_config_WLANs_change', 'source_config_SVIs_change', 'source_config_LED_change', 'source_config_SSH_change', 'source_config_Stats_change', 'source_config_IrqBalance_change', 'source_config_OurOUIs_change', 'source_config_ID_change', 'source_config_UUID_change', 'source_config__errors_change', 'source_config_Name_change', 'source_config_SiteName_change', 'source_config_HttpRedirector_change', 'source_config_SiteString_change', 'source_config_ProxyArp_change', 'source_config_Steer_change', 'source_config_WIDS_change', 'source_config_CnaHostnames_change', 'source_config_Dnsmasq_change', 'source_config_AppDef_change', 'source_config_Monitor_change', 'source_config_ExtIO_change', 'source_config_PwrMgmt_change', 'source_config_PoePwrConstrained_change', 'source_config_AVS_change', 'source_config_SwitchPorts_change', 'source_config_L2TPTunnels_change', 'source_config_VLANNames_change', 'source_config_APRecovery_change']

In [32]:
L2 = df_apscope_withconfigdiff.columns

df_apscope_withconfigdiff_2 = df_apscope_withconfigdiff.select(*[c for c in L2], *[(~F.col(c).contains('[]')).alias(c.replace("_change","_hasChanged")) for c in L2 if "_change" in c ] )


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
df_apscope_withconfigdiff_2.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ap', 'timestamp', 'row_key', 'start_time', 'time_delta', 'row_number', 'source_config_Radios', 'source_config_Hostname', 'source_config_WLANs', 'source_config_SVIs', 'source_config_LED', 'source_config_SSH', 'source_config_Stats', 'source_config_IrqBalance', 'source_config_OurOUIs', 'source_config_ID', 'source_config_UUID', 'source_config__errors', 'source_config_Name', 'source_config_SiteName', 'source_config_HttpRedirector', 'source_config_SiteString', 'source_config_ProxyArp', 'source_config_Steer', 'source_config_WIDS', 'source_config_CnaHostnames', 'source_config_Dnsmasq', 'source_config_AppDef', 'source_config_Monitor', 'source_config_ExtIO', 'source_config_PwrMgmt', 'source_config_PoePwrConstrained', 'source_config_AVS', 'source_config_SwitchPorts', 'source_config_L2TPTunnels', 'source_config_VLANNames', 'source_config_APRecovery', 'source_config_Radios_step', 'source_config_Hostname_step', 'source_config_WLANs_step', 'source_config_SVIs_step', 'source_config_LED_step', 'sourc

In [35]:
df_apscope_withconfigdiff_2.filter("ap == '5c5b35aeec86' and row_number==1")\
.select('ap', 'timestamp', 'row_key',"source_config_WLANs_hasChanged").show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+---------------------------------------------------------------------------+------------------------------+
|ap          |timestamp       |row_key                                                                    |source_config_WLANs_hasChanged|
+------------+----------------+---------------------------------------------------------------------------+------------------------------+
|5c5b35aeec86|1641553219000000|5c5b35aeec86&connectivity_failure&failed_scope_authentication&1641560762570|true                          |
+------------+----------------+---------------------------------------------------------------------------+------------------------------+

In [36]:
df_apscope_withconfigdiff_2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

70

In [37]:
df_apscope_withconfigdiff_2.filter("ap == '5c5b35aeec86' and row_number>0")\
.select('ap', 'timestamp', 'row_number', 'row_key',"source_config_WLANs_hasChanged").orderBy('row_number').show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+----------+---------------------------------------------------------------------------+------------------------------+
|ap          |timestamp       |row_number|row_key                                                                    |source_config_WLANs_hasChanged|
+------------+----------------+----------+---------------------------------------------------------------------------+------------------------------+
|5c5b35aeec86|1641553219000000|1         |5c5b35aeec86&connectivity_failure&failed_scope_authentication&1641560762570|true                          |
|5c5b35aeec86|1641539261000000|2         |5c5b35aeec86&connectivity_failure&failed_scope_authentication&1641560762570|false                         |
|5c5b35aeec86|1641531617000000|3         |5c5b35aeec86&connectivity_failure&failed_scope_authentication&1641560762570|false                         |
|5c5b35aeec86|1641502807000000|4         |5c5b35aeec86&connectivity_failure&failed_scope_authenticat

In [38]:
t2 = time.time()
print(t2 - start_time)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2314.8951964378357

In [39]:
rdd_config = df_apscope_withconfigdiff_2.rdd.map(lambda x: x.asDict())
rdd_config.persist()
# print(rdd_config.count())
out_filename = "s3://mist-data-science-dev/aditya/scope_batch_v2/" + dt_ee
rdd_config.map(lambda x: (None, json.dumps(x))).saveAsSequenceFile(out_filename)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
t2 = time.time()
print(t2 - start_time)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2432.2308564186096

In [41]:
#check

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
out_filename

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://mist-data-science-dev/aditya/scope_batch_v2/dt=2022-01-07/'

In [43]:
sloc = "s3://mist-data-science-dev/aditya/scope_batch_v2/dt=2022-01-07/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
rdd0 = sc.sequenceFile(sloc).map(lambda r: json.loads(r[1])) #.persist()
print('\n',rdd0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 70

In [45]:
r1 = rdd0.take(1)
print(r1[0].keys())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['ap', 'timestamp', 'row_key', 'start_time', 'time_delta', 'row_number', 'source_config_Radios', 'source_config_Hostname', 'source_config_WLANs', 'source_config_SVIs', 'source_config_LED', 'source_config_SSH', 'source_config_Stats', 'source_config_IrqBalance', 'source_config_OurOUIs', 'source_config_ID', 'source_config_UUID', 'source_config__errors', 'source_config_Name', 'source_config_SiteName', 'source_config_HttpRedirector', 'source_config_SiteString', 'source_config_ProxyArp', 'source_config_Steer', 'source_config_WIDS', 'source_config_CnaHostnames', 'source_config_Dnsmasq', 'source_config_AppDef', 'source_config_Monitor', 'source_config_ExtIO', 'source_config_PwrMgmt', 'source_config_PoePwrConstrained', 'source_config_AVS', 'source_config_SwitchPorts', 'source_config_L2TPTunnels', 'source_config_VLANNames', 'source_config_APRecovery', 'source_config_Radios_step', 'source_config_Hostname_step', 'source_config_WLANs_step', 'source_config_SVIs_step', 'source_config_LED_ste

In [47]:
r1 = rdd0.filter(lambda r: r['row_number']==1).map(lambda r: (r['ap'], r['start_time'], r['row_key'], r['source_config_VLANNames_hasChanged']))
r1.take(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('5c5b35aeec86', 1641560762570, '5c5b35aeec86&connectivity_failure&failed_scope_authentication&1641560762570', False), ('d4dc09212b24', 1641463366231, 'd4dc09212b24--0b940631-3d96-4e88-8021-9a37709db8db&connectivity_failure&failed_scope_authentication&1641463366231', True), ('5c5b35af127e', 1641563451455, '5c5b35af127e--f5d57efa-4596-475d-99ea-bf93b464d6a6&connectivity_failure&failed_scope_authentication&1641563451455', False), ('5c5b350e3f84', 1641560757919, '5c5b350e3f84&connectivity_failure&failed_scope_authentication&1641560757919', False), ('5c5b352eeaa6', 1641553586227, '5c5b352eeaa6--b4d1b405-14a6-4846-9d16-fcb6ec2b7829&connectivity_failure&failed_scope_authentication&1641553586227', False)]

In [232]:
df_apscope_withconfigdiff\
.filter("ap == '5c5b35aeec86' and row_number==1")\
.select("ap", "row_number", *[(~F.col(c).contains('[]')).alias(c.replace("_change","_hasChanged")) for c in df_apscope_withconfigdiff.columns if "_change" in c ]).show(1,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+-------------------------------+---------------------------------+------------------------------+-----------------------------+----------------------------+----------------------------+------------------------------+-----------------------------------+--------------------------------+---------------------------+-----------------------------+--------------------------------+-----------------------------+---------------------------------+---------------------------------------+-----------------------------------+---------------------------------+------------------------------+-----------------------------+-------------------------------------+--------------------------------+-------------------------------+--------------------------------+------------------------------+--------------------------------+------------------------------------------+----------------------------+------------------------------------+------------------------------------+------------------

In [235]:
df_apscope_withconfigdiff\
.filter("ap == '5c5b35aeec86' and row_number==1")\
.select("source_config_WLANs_change").take(1)[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(source_config_WLANs_change='[[{\'new\': \'n=false, ID=03c3b584-f30e-4fff-8b33-e35f01f66822, SSID=WCS-Guest, DisabledIEs={DisableHeIes=true}, roamMode=none, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=true, Ssdp_allow=false, \', \'old\': \'n=false, ID=03c3b584-f30e-4fff-8b33-e35f01f66822, SSID=WCS-Guest, DisabledIEs={DisableHeIes=true}, BSS={Disabled=true}, roamMode=none, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=tru\'}], [{\'new\': \'-33cd-4552-8d4d-56b7d359c1f1, SSID=WCS-Open, DisabledIEs={DisableHeIes=true}, BSS={Disabled=true}, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=true, Ssdp_allow=f\', \'old\': \'en=false, ID=195c9f15-33cd-4552-8d4d-56b7d359c1f1, SSID=WCS-Open, DisabledIEs={DisableHeIes=true}, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=true, Ssdp_allow=f\'}], [{\'new\': "TimeoutSec=1800, SilentDeauthTimeout=true, Reclai

In [236]:
a=_

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [250]:
eval(a.source_config_WLANs_change)[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'new': 'n=false, ID=03c3b584-f30e-4fff-8b33-e35f01f66822, SSID=WCS-Guest, DisabledIEs={DisableHeIes=true}, roamMode=none, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=true, Ssdp_allow=false, ', 'old': 'n=false, ID=03c3b584-f30e-4fff-8b33-e35f01f66822, SSID=WCS-Guest, DisabledIEs={DisableHeIes=true}, BSS={Disabled=true}, roamMode=none, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=tru'}]

In [255]:
for x_ in eval(a.source_config_WLANs_change):
    print('new: %s' %x_[0]['new'])
    print('old: %s' %x_[0]['old'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

new: n=false, ID=03c3b584-f30e-4fff-8b33-e35f01f66822, SSID=WCS-Guest, DisabledIEs={DisableHeIes=true}, roamMode=none, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=true, Ssdp_allow=false, 
old: n=false, ID=03c3b584-f30e-4fff-8b33-e35f01f66822, SSID=WCS-Guest, DisabledIEs={DisableHeIes=true}, BSS={Disabled=true}, roamMode=none, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=tru
new: -33cd-4552-8d4d-56b7d359c1f1, SSID=WCS-Open, DisabledIEs={DisableHeIes=true}, BSS={Disabled=true}, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=true, Ssdp_allow=f
old: en=false, ID=195c9f15-33cd-4552-8d4d-56b7d359c1f1, SSID=WCS-Open, DisabledIEs={DisableHeIes=true}, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=true, Ssdp_allow=f
new: TimeoutSec=1800, SilentDeauthTimeout=true, ReclaimTimeoutSec=300}, Vlans=[200], ForMesh=false}', '{DisabledIEs={DisableHeIes=true}, A

<h3>Bulk #2.1

In [3]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
var_date = 20
var_hr = 14
start_time = time.time()
print(start_time)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1643261939.2777507

In [5]:
s3locRoot_ee = 's3://mist-aggregated-stats-production/entity_event/'
s3locChild_ee = 'entity_event-production/'
dt_ee = "dt=2022-01-%s/" % str(var_date).zfill(2)
hr_ee = "hr=*/*/" #  "hr=14/" 
s3loc_ee = s3locRoot_ee + s3locChild_ee + dt_ee + hr_ee
print('\n--- s3loc_ee ---')
print(s3loc_ee)

event_rdd_ee = sc.sequenceFile(s3loc_ee).map(lambda r: json.loads(r[1]))#.persist()

event2consider = ['connectivity_failure']

event_rdd_ee_scope = event_rdd_ee.filter(lambda r: (('event_name' in r.keys() ) and (r['event_name'] in event2consider) \
                                                    and ('ap' in r['reason'].keys()) and (r['severity'] > 50) ) )
# print('\n',event_rdd_ee_scope.count()) 
# typically 250403 in 3 days for ['connectivity_failure', 'saturn_anomaly', 'preconnection_anomaly'], 1262 for 1 day in Dec 2021 for scope events with ap & severity >50

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-21/hr=*/*/

In [6]:
pull_schema = T.StructType([
    T.StructField('row_key', T.StringType(), True),
    T.StructField('start_time', T.LongType(), True),
    T.StructField('ap', T.StringType(), True)
])

df_select_scope_events_apscope = event_rdd_ee_scope.map(
    lambda r: (r['row_key'], r['start_time'], r['reason']['ap'])).distinct().toDF(pull_schema)

df_select_scope_events_apscope.persist()
print(df_select_scope_events_apscope.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

130

In [7]:
ap_list = [x.ap for x in df_select_scope_events_apscope.select('ap').distinct().dropna().collect()]
print(len(ap_list))
print('Some ap ids: %s', str(ap_list[:10]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

124
Some ap ids: %s ['5c5b350e21f7', 'd4dc09acf918', '5c5b35d24185', '5c5b358ecb8e', '5c5b358eb842', 'd4dc09e4e791', '5c5b355261f6', '5c5b352ed64c', 'd420b0c12fa7', 'd420b000b324']

In [8]:
strval = "dt=2022-01-{"
for x in range(var_date, var_date-2, -1):
    strval= strval + str(x).zfill(2) + ","
print(strval[:-1] +"}/")

dt_ap0 = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
hr_ap0 = "hr=*/*/"

readLoc0 = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap0 + hr_ap0
print(readLoc0)

pullSchema2 = T.StructType([       
    T.StructField('ap', T.StringType(), True),
    T.StructField('ev_name', T.StringType(), True),
    T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),
    T.StructField('timestamp', T.LongType(), True),
])


print('Reading from... %s' %readLoc0)
rdd_apevents0 = sc.sequenceFile(readLoc0).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample0 = rdd_apevents0.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list) and (r['source']['ev_name']=="AP_CONFIGURED") )
    

df_apevents_sample0 = rdd_apevents_sample0.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)

dfm0 = df_select_scope_events_apscope.select('row_key', 'start_time', 'ap')\
                                        .join(df_apevents_sample0, on='ap', how='inner')\
                                        .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                        .filter('time_delta>0 and time_delta <' + str(6*60*60*1000))

ap_list_positive = [x.ap for x in dfm0.select('ap').distinct().dropna().collect()]
print(len(ap_list_positive))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dt=2022-01-{21,20}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{21,20}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{21,20}/hr=*/*/
20

In [9]:
def returnFields(r):
    fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]','r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]','r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]','r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]','r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']
    retval = list()
    for x_ in fields:
        try:
            retval.append(eval(x_))
        except:
            retval.append(None)
    return retval


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
strval = "dt=2022-01-{"
for x in range(var_date, var_date-8, -1):
    strval= strval + str(x).zfill(2) + ","
print(strval[:-1] +"}/")

dt_ap = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
hr_ap = "hr=*/*/"

readLoc = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap + hr_ap
print(readLoc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dt=2022-01-{21,20,19,18,17,16,15,14}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{21,20,19,18,17,16,15,14}/hr=*/*/

In [11]:

print('Reading from... %s' %readLoc)
rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list_positive) and (r['source']['ev_name']=="AP_CONFIGURED") )
rdd_apevents_sample1 = rdd_apevents_sample.map(lambda r: returnFields(r))

pullSchema4 = T.StructType([T.StructField("ap" , T.StringType() , True), T.StructField("timestamp" , T.LongType() , True), \
    T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),T.StructField("source_config_WLANs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_SVIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_LED" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SSH" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Stats" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_IrqBalance" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_OurOUIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_ID" , T.StringType() , True),T.StructField("source_config_UUID" , T.StringType() , True),T.StructField("source_config__errors" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Name" , T.StringType() , True),T.StructField("source_config_SiteName" , T.StringType() , True),T.StructField("source_config_HttpRedirector" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SiteString" , T.StringType() , True),T.StructField("source_config_ProxyArp" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Steer" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_WIDS" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_CnaHostnames" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Dnsmasq" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AppDef" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Monitor" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_ExtIO" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_PwrMgmt" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_PoePwrConstrained" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AVS" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SwitchPorts" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_L2TPTunnels" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_VLANNames" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_APRecovery" , T.MapType(T.StringType(),T.StringType()) , True)
])

df_apevents_sample = rdd_apevents_sample1.toDF(pullSchema4)



# print('\n',event_rdd_apevents_sampledSites.count())
# df_apevents_sample = rdd_apevents_sample.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)
df_apevents_sample.printSchema()
df_apevents_sample.persist()
print(df_apevents_sample.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{21,20,19,18,17,16,15,14}/hr=*/*/
root
 |-- ap: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- source_config_Radios: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- source_config_Hostname: string (nullable = true)
 |-- source_config_WLANs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- source_config_SVIs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- source_config_LED: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- source_config_SSH: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- source_config_Stats: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- source_config_IrqBalance: map (nullable = true)
 |    |-- key: string
 |    |-- value: s

In [12]:
fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]','r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]','r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]','r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]','r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']

L1 = [x_.replace("r[","").replace("\"][\"", "_").replace("]","").replace("\"","")+ '_step' for x_ in fields[2:]]
print(L1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['source_config_Radios_step', 'source_config_Hostname_step', 'source_config_WLANs_step', 'source_config_SVIs_step', 'source_config_LED_step', 'source_config_SSH_step', 'source_config_Stats_step', 'source_config_IrqBalance_step', 'source_config_OurOUIs_step', 'source_config_ID_step', 'source_config_UUID_step', 'source_config__errors_step', 'source_config_Name_step', 'source_config_SiteName_step', 'source_config_HttpRedirector_step', 'source_config_SiteString_step', 'source_config_ProxyArp_step', 'source_config_Steer_step', 'source_config_WIDS_step', 'source_config_CnaHostnames_step', 'source_config_Dnsmasq_step', 'source_config_AppDef_step', 'source_config_Monitor_step', 'source_config_ExtIO_step', 'source_config_PwrMgmt_step', 'source_config_PoePwrConstrained_step', 'source_config_AVS_step', 'source_config_SwitchPorts_step', 'source_config_L2TPTunnels_step', 'source_config_VLANNames_step', 'source_config_APRecovery_step']

In [13]:
ap_list_positive

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['5c5b35d24185', 'd420b000b324', 'd420b000b1f3', '5c5b352e556e', 'd420b0022a06', '5c5b353e155e', '5c5b350e3f84', 'd420b042f062', '5c5b35aebca7', 'd4dc09256315', 'd4dc09e4bded', 'd4dc09acfd14', '5c5b35cf68c3', '5c5b35505979', 'd4dc09246366', '5c5b352eefa6', 'd420b000b1b2', '5c5b35aebc98', 'd420b0c0f2cb', '5c5b35d37564']

In [14]:
df_select_scope_events_apscope.columns, df_apevents_sample.columns


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(['row_key', 'start_time', 'ap'], ['ap', 'timestamp', 'source_config_Radios', 'source_config_Hostname', 'source_config_WLANs', 'source_config_SVIs', 'source_config_LED', 'source_config_SSH', 'source_config_Stats', 'source_config_IrqBalance', 'source_config_OurOUIs', 'source_config_ID', 'source_config_UUID', 'source_config__errors', 'source_config_Name', 'source_config_SiteName', 'source_config_HttpRedirector', 'source_config_SiteString', 'source_config_ProxyArp', 'source_config_Steer', 'source_config_WIDS', 'source_config_CnaHostnames', 'source_config_Dnsmasq', 'source_config_AppDef', 'source_config_Monitor', 'source_config_ExtIO', 'source_config_PwrMgmt', 'source_config_PoePwrConstrained', 'source_config_AVS', 'source_config_SwitchPorts', 'source_config_L2TPTunnels', 'source_config_VLANNames', 'source_config_APRecovery'])

In [15]:
step_window = Window.partitionBy('ap').orderBy(F.col("row_number").desc())

df_apscope_withconfig = df_select_scope_events_apscope.join(df_apevents_sample, on ='ap', how='inner')\
                                .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                .filter('time_delta>0')\
                                .withColumn('row_number', F.row_number().over(Window.partitionBy('ap').orderBy(F.col('time_delta').asc())))

df_apscope_withconfig = df_apscope_withconfig.select('ap', 'timestamp','row_key', 'start_time','time_delta', 'row_number',\
                                                     *[c.replace("_step", "").replace("\"", "") for c in L1], \
                                                     *[F.lag((c.replace("_step", "").replace("\"", "")), offset=1).over(step_window).alias(c) for c in L1],\
                                                    F.lag('timestamp', offset=1).over(step_window).alias('timestamp_step'))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
def getconfigdiff_def(x, y):
    if (x == "") or (y == ""):
        return "'config not found'"
    else:
        try:
            j1 = str(x)  # json.loads(x)
            j2 = str(y)  # json.loads(y)
            l1 = [x for x in difflib.unified_diff(j1, j2, n=100, lineterm='') if "@@" in x]
            retlist = list()
            for x_ in l1:
                sp = x_.replace("@", "").replace(" ", "").split("+")[1].split(",")
                s_ = int(sp[0])
                e_ = int(sp[1])
                if sp != []:
                    print(s_, e_)
                    retlist.append([{"new": j1[s_: s_ + e_ - 1], "old": j2[s_: s_ + e_ - 1]}])
            retval = str(retlist)
            return retval
        except:
            retval = "error"
        return retval

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
getconfigDiff = F.udf(getconfigdiff_def, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
L0 = df_apscope_withconfig.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_apscope_withconfigdiff = df_apscope_withconfig.select(*[c for c in L0], *[getconfigDiff(F.col(c.replace("_step","")), F.col(c) ).alias(c.replace("_step","_change")) for c in L1] )



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
L2 = df_apscope_withconfigdiff.columns

df_apscope_withconfigdiff_2 = df_apscope_withconfigdiff.select(*[c for c in L2], *[(~F.col(c).contains('[]')).alias(c.replace("_change","_hasChanged")) for c in L2 if "_change" in c ] )


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# df_select_scope_events_apscope.unpersist()
# df_apevents_sample.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
t2 = time.time()
print(t2 - start_time)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

947.8242650032043

In [23]:
rdd_config = df_apscope_withconfigdiff_2.rdd.map(lambda x: x.asDict())
rdd_config.persist()
# print(rdd_config.count())
out_filename = "s3://mist-data-science-dev/aditya/scope_batch_v2/" + dt_ee
rdd_config.map(lambda x: (None, json.dumps(x))).saveAsSequenceFile(out_filename)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
t2 = time.time()
print(t2 - start_time)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

991.2588894367218

In [25]:
rdd_config.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[86] at RDD at PythonRDD.scala:53

In [41]:
#check

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
out_filename

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://mist-data-science-dev/aditya/scope_batch_v2/dt=2022-01-07/'

In [76]:
sloc = "s3://mist-data-science-dev/aditya/scope_batch_v2/dt=2022-01-07/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
rdd0 = sc.sequenceFile(sloc).map(lambda r: json.loads(r[1])) #.persist()
print('\n',rdd0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 329

In [78]:
r1 = rdd0.take(1)
print(r1[0].keys())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_keys(['ap', 'timestamp', 'row_key', 'start_time', 'time_delta', 'row_number', 'source_config_Radios', 'source_config_Hostname', 'source_config_WLANs', 'source_config_SVIs', 'source_config_LED', 'source_config_SSH', 'source_config_Stats', 'source_config_IrqBalance', 'source_config_OurOUIs', 'source_config_ID', 'source_config_UUID', 'source_config__errors', 'source_config_Name', 'source_config_SiteName', 'source_config_HttpRedirector', 'source_config_SiteString', 'source_config_ProxyArp', 'source_config_Steer', 'source_config_WIDS', 'source_config_CnaHostnames', 'source_config_Dnsmasq', 'source_config_AppDef', 'source_config_Monitor', 'source_config_ExtIO', 'source_config_PwrMgmt', 'source_config_PoePwrConstrained', 'source_config_AVS', 'source_config_SwitchPorts', 'source_config_L2TPTunnels', 'source_config_VLANNames', 'source_config_APRecovery', 'source_config_Radios_step', 'source_config_Hostname_step', 'source_config_WLANs_step', 'source_config_SVIs_step', 'source_config_LED_ste

In [80]:
r1 = rdd0.filter(lambda r: r['row_number']==1).map(lambda r: (r['ap'], r['start_time'],r['source_config_WLANs_hasChanged'], r['source_config_VLANNames_hasChanged']))
r2 = r1.take(5)

for x_ in r2:
    print(x_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('5c5b35aeec86', 1641560762570, True, False)
('d4dc09212b24', 1641463366231, True, True)
('5c5b353e14c3', 1641561983019, False, False)
('5c5b358e2ff8', 1641463205549, False, False)
('5c5b352e556e', 1641562011267, True, False)

<h3>Bulk #2.2

In [5]:
def returnFields(r):
    fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]','r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]','r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]','r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]','r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']
    retval = list()
    for x_ in fields:
        try:
            retval.append(eval(x_))
        except:
            retval.append(None)
    return retval

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def getconfigdiff_def(x, y):
    if (x == "") or (y == ""):
        return "'config not found'"
    else:
        try:
            j1 = str(x)  # json.loads(x)
            j2 = str(y)  # json.loads(y)
            l1 = [x for x in difflib.unified_diff(j1, j2, n=100, lineterm='') if "@@" in x]
            retlist = list()
            for x_ in l1:
                sp = x_.replace("@", "").replace(" ", "").split("+")[1].split(",")
                s_ = int(sp[0])
                e_ = int(sp[1])
                if sp != []:
                    print(s_, e_)
                    retlist.append([{"new": j1[s_: s_ + e_ - 1], "old": j2[s_: s_ + e_ - 1]}])
            retval = str(retlist)
            return retval
        except:
            retval = "error"
        return retval

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
getconfigDiff = F.udf(getconfigdiff_def, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
def ExtractConfigs(sc, var_date):
    # var_date = 20
    var_hr = 14
    start_time = time.time()
    logs = list()
    log1 = "Start time: %d" %start_time
    print(log1)
    logs.append(log1)
    # ---
    s3locRoot_ee = 's3://mist-aggregated-stats-production/entity_event/'
    s3locChild_ee = 'entity_event-production/'
    dt_ee = "dt=2022-01-%s/" % str(var_date).zfill(2)
    dt_ee = "dt=2021-12-%s/" % str(var_date).zfill(2)
    hr_ee = "hr=*/*/" #  "hr=14/" 
    s3loc_ee = s3locRoot_ee + s3locChild_ee + dt_ee + hr_ee
    print('\n--- s3loc_ee ---')
    print(s3loc_ee)
    logs.append(s3loc_ee)

    event_rdd_ee = sc.sequenceFile(s3loc_ee).map(lambda r: json.loads(r[1]))#.persist()
    event2consider = ['connectivity_failure']
    event_rdd_ee_scope = event_rdd_ee.filter(lambda r: (('event_name' in r.keys() ) and (r['event_name'] in event2consider) \
                                                        and ('ap' in r['reason'].keys()) and (r['severity'] > 50) ) )
    # ---
    pull_schema = T.StructType([
    T.StructField('row_key', T.StringType(), True),
    T.StructField('start_time', T.LongType(), True),
    T.StructField('ap', T.StringType(), True)
    ])
    df_select_scope_events_apscope = event_rdd_ee_scope.map(
        lambda r: (r['row_key'], r['start_time'], r['reason']['ap'])).distinct().toDF(pull_schema)

    df_select_scope_events_apscope.persist()
    print('\n--- Scope events with AP scope ---')
    print(df_select_scope_events_apscope.count())
    #---
    ap_list = [x.ap for x in df_select_scope_events_apscope.select('ap').distinct().dropna().collect()]
    log2 = '\n--- APs involved --- \n %s' %str(len(ap_list))
    print(log2)
    logs.append(log2)
    log3  = 'Some ap ids: %s' %str(ap_list[:10])
    logs.append(log3)
    print(log3)
    # ---
    strval = "dt=2022-01-{"
    strval = "dt=2021-12-{"
    for x in range(var_date, var_date-2, -1):
        strval= strval + str(x).zfill(2) + ","
    print(strval[:-1] +"}/")
    dt_ap0 = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
    hr_ap0 = "hr=*/*/"
    readLoc0 = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap0 + hr_ap0
    print(readLoc0)
    pullSchema2 = T.StructType([       
        T.StructField('ap', T.StringType(), True),
        T.StructField('ev_name', T.StringType(), True),
        T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),
        T.StructField('timestamp', T.LongType(), True),
    ])
    print('Reading from... %s' %readLoc0)
    rdd_apevents0 = sc.sequenceFile(readLoc0).map(lambda r: json.loads(r[1])) #.persist()
    rdd_apevents_sample0 = rdd_apevents0.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                         and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list) and (r['source']['ev_name']=="AP_CONFIGURED") )


    df_apevents_sample0 = rdd_apevents_sample0.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)
    dfm0 = df_select_scope_events_apscope.select('row_key', 'start_time', 'ap')\
                                            .join(df_apevents_sample0, on='ap', how='inner')\
                                            .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                            .filter('time_delta>0 and time_delta <' + str(6*60*60*1000))
    ap_list_positive = [x.ap for x in dfm0.select('ap').distinct().dropna().collect()]
    log4 = '\n---APs that had a config event within 6 hours ---\n %s' %str(len(ap_list_positive))
    print(log4)
    logs.append(log4)
    #---
    strval = "dt=2022-01-{"
    strval = "dt=2021-12-{"
    for x in range(var_date, var_date-8, -1):
        strval= strval + str(x).zfill(2) + ","
    print(strval[:-1] +"}/")

    dt_ap = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
    hr_ap = "hr=*/*/"

    readLoc = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap + hr_ap
    print(readLoc)
    #---
    print('Reading from... %s' %readLoc)
    rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
    rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                         and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list_positive) and (r['source']['ev_name']=="AP_CONFIGURED") )
    rdd_apevents_sample1 = rdd_apevents_sample.map(lambda r: returnFields(r))

    pullSchema4 = T.StructType([T.StructField("ap" , T.StringType() , True), T.StructField("timestamp" , T.LongType() , True), \
        T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),T.StructField("source_config_WLANs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_SVIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_LED" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SSH" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Stats" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_IrqBalance" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_OurOUIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_ID" , T.StringType() , True),T.StructField("source_config_UUID" , T.StringType() , True),T.StructField("source_config__errors" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Name" , T.StringType() , True),T.StructField("source_config_SiteName" , T.StringType() , True),T.StructField("source_config_HttpRedirector" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SiteString" , T.StringType() , True),T.StructField("source_config_ProxyArp" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Steer" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_WIDS" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_CnaHostnames" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Dnsmasq" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AppDef" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Monitor" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_ExtIO" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_PwrMgmt" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_PoePwrConstrained" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AVS" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SwitchPorts" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_L2TPTunnels" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_VLANNames" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_APRecovery" , T.MapType(T.StringType(),T.StringType()) , True)
    ])

    df_apevents_sample = rdd_apevents_sample1.toDF(pullSchema4)
    # df_apevents_sample.printSchema()
    df_apevents_sample.persist()
    log5 = '\n--- AP configured events pulled for the last 7 days ---\n %s' %str(df_apevents_sample.count())
    print(log5)
    logs.append(log5)
   #---
    fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]','r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]','r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]','r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]','r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']

    L1 = [x_.replace("r[","").replace("\"][\"", "_").replace("]","").replace("\"","")+ '_step' for x_ in fields[2:]]
    #---
    step_window = Window.partitionBy('ap').orderBy(F.col("row_number").desc())

    df_apscope_withconfig = df_select_scope_events_apscope.join(df_apevents_sample, on ='ap', how='inner')\
                                    .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                    .filter('time_delta>0')\
                                    .withColumn('row_number', F.row_number().over(Window.partitionBy('ap').orderBy(F.col('time_delta').asc())))

    df_apscope_withconfig = df_apscope_withconfig.select('ap', 'timestamp','row_key', 'start_time','time_delta', 'row_number',\
                                                         *[c.replace("_step", "").replace("\"", "") for c in L1], \
                                                         *[F.lag((c.replace("_step", "").replace("\"", "")), offset=1).over(step_window).alias(c) for c in L1],\
                                                        F.lag('timestamp', offset=1).over(step_window).alias('timestamp_step'))
    #---
    L0 = df_apscope_withconfig.columns
    df_apscope_withconfigdiff = df_apscope_withconfig.select(*[c for c in L0], *[getconfigDiff(F.col(c.replace("_step","")), F.col(c) ).alias(c.replace("_step","_change")) for c in L1] )
    L2 = df_apscope_withconfigdiff.columns

    df_apscope_withconfigdiff_2 = df_apscope_withconfigdiff.select(*[c for c in L2], *[(~F.col(c).contains('[]')).alias(c.replace("_change","_hasChanged")) for c in L2 if "_change" in c ] )
    t2 = time.time()
    log6 = '\n--- Data pulled ---\n %s' %str(t2 - start_time)
    print(log6)
    logs.append(log6)

    rdd_config = df_apscope_withconfigdiff_2.rdd.map(lambda x: x.asDict())
    rdd_config.persist()
    out_filename = "s3://mist-data-science-dev/aditya/scope_batch_v2/" + dt_ee
    rdd_config.map(lambda x: (None, json.dumps(x))).saveAsSequenceFile(out_filename)
    t2 = time.time()
    log7 = '\n--- Data written ---\n %s' %str(t2 - start_time)
    print(log7)
    logs.append(log7)
    
    #---
    df_select_scope_events_apscope.unpersist()
    df_apevents_sample.unpersist()
    rdd_config.unpersist()
    return logs
#     #-- write logs
#     fid = open("s3://mist-data-science-dev/aditya/scope_batch_v2_logs.txt","wa")
#     for x_ in logs:
#         fid.write

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
ExtractConfigs(sc, 19)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1643277811.1194637

--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-19/hr=*/*/
221
192
Some ap ids: %s ['d4dc09acf918', 'd420b084a6a0', 'd420b083ce1a', 'd420b0008601', 'd420b000b37e', 'd420b02027d3', '5c5b35d3359f', '5c5b35d24185', '5c5b358eabf9', 'd4dc09225a7b']
dt=2022-01-{19,18}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{19,18}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{19,18}/hr=*/*/
68
dt=2022-01-{19,18,17,16,15,14,13,12}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{19,18,17,16,15,14,13,12}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{19,18,17,16,15,14,13,12}/hr=*/*/
root
 |-- ap: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- source_config_Radios: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- source_config_Host

In [15]:
ExtractConfigs(sc, 18)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Start time: 1643282046

--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-18/hr=*/*/

--- Scope events with AP scope ---
154

--- APs involved --- 
 135
Some ap ids: ['5c5b35f24438', '5c5b350e21f7', '5c5b35d24185', '5c5b353e19c8', '5c5b35d07f35', '5c5b35506973', 'd4dc09259443', 'd420b08022ec', 'd4dc09244f2a', '5c5b352ed64c']
dt=2022-01-{18,17}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{18,17}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{18,17}/hr=*/*/

---APs that had a config event within 6 hours ---
 27
dt=2022-01-{18,17,16,15,14,13,12,11}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{18,17,16,15,14,13,12,11}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{18,17,16,15,14,13,12,11}/hr=*/*/

--- AP configured events pulled for the last 7 days ---
 389

--- Data pulled ---
 117

In [10]:
ExtractConfigs(sc, 20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Start time: 1643279762.364057

--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-20/hr=*/*/

--- Scope events with AP scope ---
193

--- APs involved ---
168
Some ap ids: %s ['5c5b35f1e968', '5c5b35f24438', '5c5b35d24185', '5c5b350e21f7', 'd420b08013bf', '5c5b35d07f35', '5c5b353e150e', '5c5b3552ca3d', '5c5b353e939e', 'd4dc092b5482']
dt=2022-01-{20,19}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{20,19}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{20,19}/hr=*/*/

---APs that had a config event within 6 hours ---
41
dt=2022-01-{20,19,18,17,16,15,14,13}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{20,19,18,17,16,15,14,13}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{20,19,18,17,16,15,14,13}/hr=*/*/

--- AP configured events pulled for the last 7 days ---
476
Data pulled 1174.11

In [20]:
logs=['logs entry', 'run outputs', 'log line 3']

s3 = boto3.resource('s3')
s3object = s3.Object('mist-data-science-dev', 'aditya/scope_batch_v2/scope_batch_v2_logs_15.txt')

s3object.put(
    Body=(bytes(json.dumps(logs).encode('UTF-8')))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '0V4E0XB202RERTZW', 'HostId': '8Yl9RN8C/3obb+qF4Dsqk0DTJ8e4Rb0fE091x080VwZOcqltTTLcHfMejikyXFP38uvokY8MkwY=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '8Yl9RN8C/3obb+qF4Dsqk0DTJ8e4Rb0fE091x080VwZOcqltTTLcHfMejikyXFP38uvokY8MkwY=', 'x-amz-request-id': '0V4E0XB202RERTZW', 'date': 'Thu, 27 Jan 2022 12:11:00 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"9c9d63acbc1a96f837d6d45d86c82192"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}, 'ETag': '"9c9d63acbc1a96f837d6d45d86c82192"', 'ServerSideEncryption': 'AES256'}

In [10]:
mainlogs = list()
s3 = boto3.resource('s3')

for x_ in range(15,18):
    logs = ExtractConfigs(sc, x_)
    outfilename = 'aditya/scope_batch_v2/scope_batch_v2_logs_'+str(x_)+'.txt'
    s3object = s3.Object('mist-data-science-dev', outfilename)
    print(outfilename)
    s3object.put(Body=(bytes(json.dumps(logs).encode('UTF-8'))))
    mainlogs.append(logs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Start time: 1643286292

--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-15/hr=*/*/

--- Scope events with AP scope ---
221

--- APs involved --- 
 187
Some ap ids: ['d420b0855960', '5c5b35d17e1c', '5c5b350e21f7', 'd420b085582a', 'd4dc09259439', '5c5b350eea7b', 'd420b002b994', 'd420b08013bf', 'd420b08018e2', '5c5b35ce5f55']
dt=2022-01-{15,14}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{15,14}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{15,14}/hr=*/*/

---APs that had a config event within 6 hours ---
 33
dt=2022-01-{15,14,13,12,11,10,09,08}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{15,14,13,12,11,10,09,08}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{15,14,13,12,11,10,09,08}/hr=*/*/

--- AP configured events pulled for the last 7 days ---
 400

--- Data pulled ---
 944

In [11]:
mainlogs = list()
s3 = boto3.resource('s3')

for x_ in range(12,15):
    logs = ExtractConfigs(sc, x_)
    outfilename = 'aditya/scope_batch_v2/scope_batch_v2_logs_'+str(x_)+'.txt'
    s3object = s3.Object('mist-data-science-dev', outfilename)
    print(outfilename)
    s3object.put(Body=(bytes(json.dumps(logs).encode('UTF-8'))))
    mainlogs.append(logs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Start time: 1643288993

--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-12/hr=*/*/

--- Scope events with AP scope ---
178

--- APs involved --- 
 157
Some ap ids: ['5c5b353e939e', '5c5b35f24438', 'd420b00276cd', '5c5b358eff00', '5c5b358ecfad', '5c5b35ae94ac', 'd4dc09e4e791', 'd4dc09212b24', '5c5b35be1086', '5c5b35ae7bed']
dt=2022-01-{12,11}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{12,11}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{12,11}/hr=*/*/

---APs that had a config event within 6 hours ---
 30
dt=2022-01-{12,11,10,09,08,07,06,05}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{12,11,10,09,08,07,06,05}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{12,11,10,09,08,07,06,05}/hr=*/*/

--- AP configured events pulled for the last 7 days ---
 477

--- Data pulled ---
 771

In [13]:
mainlogs = list()
s3 = boto3.resource('s3')

for x_ in range(8,12):
    logs = ExtractConfigs(sc, x_)
    outfilename = 'aditya/scope_batch_v2/scope_batch_v2_logs_'+str(x_)+'.txt'
    s3object = s3.Object('mist-data-science-dev', outfilename)
    print(outfilename)
    s3object.put(Body=(bytes(json.dumps(logs).encode('UTF-8'))))
    mainlogs.append(logs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://ip-172-31-63-99.ec2.internal:8998/sessions/109/statements/13 with error payload: {"msg":"requirement failed: Session isn't active."}


In [13]:
s3 = boto3.resource('s3')

for x_ in range(1,18):
    filename = "aditya/scope_batch_v2/scope_batch_v2_logs_%d.txt" %x_
    try:
        s3object = s3.Object('mist-data-science-dev', filename)
        l0 = json.load(s3object.get()["Body"])
        print("\n> %s ---" %filename)
        print(l0[-4])
    except:
        continue

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


> aditya/scope_batch_v2/scope_batch_v2_logs_1.txt ---

---APs that had a config event within 6 hours ---
 1

> aditya/scope_batch_v2/scope_batch_v2_logs_2.txt ---

---APs that had a config event within 6 hours ---
 3

> aditya/scope_batch_v2/scope_batch_v2_logs_3.txt ---

---APs that had a config event within 6 hours ---
 18

> aditya/scope_batch_v2/scope_batch_v2_logs_4.txt ---

---APs that had a config event within 6 hours ---
 18

> aditya/scope_batch_v2/scope_batch_v2_logs_5.txt ---

---APs that had a config event within 6 hours ---
 82

> aditya/scope_batch_v2/scope_batch_v2_logs_6.txt ---

---APs that had a config event within 6 hours ---
 92

> aditya/scope_batch_v2/scope_batch_v2_logs_8.txt ---

---APs that had a config event within 6 hours ---
 3

> aditya/scope_batch_v2/scope_batch_v2_logs_9.txt ---

---APs that had a config event within 6 hours ---
 2

> aditya/scope_batch_v2/scope_batch_v2_logs_10.txt ---

---APs that had a config event within 6 hours ---
 34

> aditya/sco

In [12]:
l0

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Start time: 1643287899', 's3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-17/hr=*/*/', '\n--- APs involved --- \n 133', "Some ap ids: ['5c5b35d07f35', '5c5b358f0ff9', '5c5b353e19c8', 'd420b08013bf', 'd420b0c0b833', 'd420b000b324', '5c5b358e1c39', 'd420b08022ec', 'd4dc09212b24', '5c5b352ed64c']", '\n---APs that had a config event within 6 hours ---\n 28', '\n--- AP configured events pulled for the last 7 days ---\n 368', '\n--- Data pulled ---\n 557.2381687164307', '\n--- Data written ---\n 602.3569478988647']

<h5> run 2021

In [20]:
ExtractConfigs(sc, 31)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Start time: 1643360680

--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2021-12-31/hr=*/*/

--- Scope events with AP scope ---
60

--- APs involved --- 
 53
Some ap ids: ['d4dc09212b24', '5c5b358eb842', 'd4dc09acfe77', '5c5b353e91d2', '5c5b35be1400', '5c5b350e6d89', '5c5b35aeb013', '5c5b358f5289', 'd420b085029e', '5c5b35d364a7']
dt=2021-12-{31,30}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-12-{31,30}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-12-{31,30}/hr=*/*/

---APs that had a config event within 6 hours ---
 9
dt=2021-12-{31,30,29,28,27,26,25,24}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-12-{31,30,29,28,27,26,25,24}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2021-12-{31,30,29,28,27,26,25,24}/hr=*/*/

--- AP configured events pulled for the last 7 days ---
 105

--- Data pulled ---
 645.54

In [3]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
mainlogs = list()
s3 = boto3.resource('s3')

for x_ in range(27,31):
    logs = ExtractConfigs(sc, x_)
    outfilename = 'aditya/scope_batch_v2/scope_batch_v2_logs_2021_'+str(x_)+'.txt'
    s3object = s3.Object('mist-data-science-dev', outfilename)
    print(outfilename)
    s3object.put(Body=(bytes(json.dumps(logs).encode('UTF-8'))))
    mainlogs.append(logs)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'ExtractConfigs' is not defined
Traceback (most recent call last):
NameError: name 'ExtractConfigs' is not defined



<h4>Prep validation

In [41]:
newlist0 = list()
for x_ in list(np.arange(1,32)):
    newlist0.append(str(x_))
",".join(newlist0)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31'

In [42]:
sloc = "s3://mist-data-science-dev/aditya/scope_batch_v2/dt=202{1,2}-{01,12}-{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31}/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
rdd0 = sc.sequenceFile(sloc).map(lambda r: json.loads(r[1])).filter(lambda r: r['row_number']==1).persist()
print('\n',rdd0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 646

In [45]:
# r1 = rdd0.take(1)
# print(r1[0].keys())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
# pullSchema4 = T.StructType([T.StructField("ap" , T.StringType() , True), T.StructField("timestamp" , T.LongType() , True), \
#     T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),T.StructField("source_config_WLANs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_SVIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_LED" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SSH" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Stats" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_IrqBalance" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_OurOUIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_ID" , T.StringType() , True),T.StructField("source_config_UUID" , T.StringType() , True),T.StructField("source_config__errors" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Name" , T.StringType() , True),T.StructField("source_config_SiteName" , T.StringType() , True),T.StructField("source_config_HttpRedirector" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SiteString" , T.StringType() , True),T.StructField("source_config_ProxyArp" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Steer" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_WIDS" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_CnaHostnames" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Dnsmasq" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AppDef" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Monitor" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_ExtIO" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_PwrMgmt" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_PoePwrConstrained" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AVS" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SwitchPorts" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_L2TPTunnels" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_VLANNames" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_APRecovery" , T.MapType(T.StringType(),T.StringType()) , True)
# ])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
l0 = ['ap', 'timestamp', 'row_key', 'start_time', 'time_delta', 'row_number', 'source_config_Radios', 'source_config_Hostname', 'source_config_WLANs', 'source_config_SVIs', 'source_config_LED', 'source_config_SSH', 'source_config_Stats', 'source_config_IrqBalance', 'source_config_OurOUIs', 'source_config_ID', 'source_config_UUID', 'source_config__errors', 'source_config_Name', 'source_config_SiteName', 'source_config_HttpRedirector', 'source_config_SiteString', 'source_config_ProxyArp', 'source_config_Steer', 'source_config_WIDS', 'source_config_CnaHostnames', 'source_config_Dnsmasq', 'source_config_AppDef', 'source_config_Monitor', 'source_config_ExtIO', 'source_config_PwrMgmt', 'source_config_PoePwrConstrained', 'source_config_AVS', 'source_config_SwitchPorts', 'source_config_L2TPTunnels', 'source_config_VLANNames', 'source_config_APRecovery', 'source_config_Radios_step', 'source_config_Hostname_step', 'source_config_WLANs_step', 'source_config_SVIs_step', 'source_config_LED_step', 'source_config_SSH_step', 'source_config_Stats_step', 'source_config_IrqBalance_step', 'source_config_OurOUIs_step', 'source_config_ID_step', 'source_config_UUID_step', 'source_config__errors_step', 'source_config_Name_step', 'source_config_SiteName_step', 'source_config_HttpRedirector_step', 'source_config_SiteString_step', 'source_config_ProxyArp_step', 'source_config_Steer_step', 'source_config_WIDS_step', 'source_config_CnaHostnames_step', 'source_config_Dnsmasq_step', 'source_config_AppDef_step', 'source_config_Monitor_step', 'source_config_ExtIO_step', 'source_config_PwrMgmt_step', 'source_config_PoePwrConstrained_step', 'source_config_AVS_step', 'source_config_SwitchPorts_step', 'source_config_L2TPTunnels_step', 'source_config_VLANNames_step', 'source_config_APRecovery_step', 'timestamp_step', 'source_config_Radios_change', 'source_config_Hostname_change', 'source_config_WLANs_change', 'source_config_SVIs_change', 'source_config_LED_change', 'source_config_SSH_change', 'source_config_Stats_change', 'source_config_IrqBalance_change', 'source_config_OurOUIs_change', 'source_config_ID_change', 'source_config_UUID_change', 'source_config__errors_change', 'source_config_Name_change', 'source_config_SiteName_change', 'source_config_HttpRedirector_change', 'source_config_SiteString_change', 'source_config_ProxyArp_change', 'source_config_Steer_change', 'source_config_WIDS_change', 'source_config_CnaHostnames_change', 'source_config_Dnsmasq_change', 'source_config_AppDef_change', 'source_config_Monitor_change', 'source_config_ExtIO_change', 'source_config_PwrMgmt_change', 'source_config_PoePwrConstrained_change', 'source_config_AVS_change', 'source_config_SwitchPorts_change', 'source_config_L2TPTunnels_change', 'source_config_VLANNames_change', 'source_config_APRecovery_change', 'source_config_Radios_hasChanged', 'source_config_Hostname_hasChanged', 'source_config_WLANs_hasChanged', 'source_config_SVIs_hasChanged', 'source_config_LED_hasChanged', 'source_config_SSH_hasChanged', 'source_config_Stats_hasChanged', 'source_config_IrqBalance_hasChanged', 'source_config_OurOUIs_hasChanged', 'source_config_ID_hasChanged', 'source_config_UUID_hasChanged', 'source_config__errors_hasChanged', 'source_config_Name_hasChanged', 'source_config_SiteName_hasChanged', 'source_config_HttpRedirector_hasChanged', 'source_config_SiteString_hasChanged', 'source_config_ProxyArp_hasChanged', 'source_config_Steer_hasChanged', 'source_config_WIDS_hasChanged', 'source_config_CnaHostnames_hasChanged', 'source_config_Dnsmasq_hasChanged', 'source_config_AppDef_hasChanged', 'source_config_Monitor_hasChanged', 'source_config_ExtIO_hasChanged', 'source_config_PwrMgmt_hasChanged', 'source_config_PoePwrConstrained_hasChanged', 'source_config_AVS_hasChanged', 'source_config_SwitchPorts_hasChanged', 'source_config_L2TPTunnels_hasChanged', 'source_config_VLANNames_hasChanged', 'source_config_APRecovery_hasChanged']
l1 = ['ap', 'timestamp', 'row_key', 'start_time', 'time_delta', 'row_number']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
pickCols = l1 + [x_ for x_ in l0 if "_hasChanged" in x_]
print(pickCols)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ap', 'timestamp', 'row_key', 'start_time', 'time_delta', 'row_number', 'source_config_Radios_hasChanged', 'source_config_Hostname_hasChanged', 'source_config_WLANs_hasChanged', 'source_config_SVIs_hasChanged', 'source_config_LED_hasChanged', 'source_config_SSH_hasChanged', 'source_config_Stats_hasChanged', 'source_config_IrqBalance_hasChanged', 'source_config_OurOUIs_hasChanged', 'source_config_ID_hasChanged', 'source_config_UUID_hasChanged', 'source_config__errors_hasChanged', 'source_config_Name_hasChanged', 'source_config_SiteName_hasChanged', 'source_config_HttpRedirector_hasChanged', 'source_config_SiteString_hasChanged', 'source_config_ProxyArp_hasChanged', 'source_config_Steer_hasChanged', 'source_config_WIDS_hasChanged', 'source_config_CnaHostnames_hasChanged', 'source_config_Dnsmasq_hasChanged', 'source_config_AppDef_hasChanged', 'source_config_Monitor_hasChanged', 'source_config_ExtIO_hasChanged', 'source_config_PwrMgmt_hasChanged', 'source_config_PoePwrConstrained_hasChang

In [49]:
c1 = ["T.StructField(\"%s\" , T.StringType() , True)"%x_ for x_ in l1]
c2 = ["T.StructField(\"%s\" , T.BooleanType() , True)"%x_ for x_ in pickCols[6:]]

print(c1)
print(c2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['T.StructField("ap" , T.StringType() , True)', 'T.StructField("timestamp" , T.StringType() , True)', 'T.StructField("row_key" , T.StringType() , True)', 'T.StructField("start_time" , T.StringType() , True)', 'T.StructField("time_delta" , T.StringType() , True)', 'T.StructField("row_number" , T.StringType() , True)']
['T.StructField("source_config_Radios_hasChanged" , T.BooleanType() , True)', 'T.StructField("source_config_Hostname_hasChanged" , T.BooleanType() , True)', 'T.StructField("source_config_WLANs_hasChanged" , T.BooleanType() , True)', 'T.StructField("source_config_SVIs_hasChanged" , T.BooleanType() , True)', 'T.StructField("source_config_LED_hasChanged" , T.BooleanType() , True)', 'T.StructField("source_config_SSH_hasChanged" , T.BooleanType() , True)', 'T.StructField("source_config_Stats_hasChanged" , T.BooleanType() , True)', 'T.StructField("source_config_IrqBalance_hasChanged" , T.BooleanType() , True)', 'T.StructField("source_config_OurOUIs_hasChanged" , T.BooleanType() 

In [50]:
for x_ in c1:
    print(x_+",")
for x_ in c2:
    print(x_+",")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

T.StructField("ap" , T.StringType() , True),
T.StructField("timestamp" , T.StringType() , True),
T.StructField("row_key" , T.StringType() , True),
T.StructField("start_time" , T.StringType() , True),
T.StructField("time_delta" , T.StringType() , True),
T.StructField("row_number" , T.StringType() , True),
T.StructField("source_config_Radios_hasChanged" , T.BooleanType() , True),
T.StructField("source_config_Hostname_hasChanged" , T.BooleanType() , True),
T.StructField("source_config_WLANs_hasChanged" , T.BooleanType() , True),
T.StructField("source_config_SVIs_hasChanged" , T.BooleanType() , True),
T.StructField("source_config_LED_hasChanged" , T.BooleanType() , True),
T.StructField("source_config_SSH_hasChanged" , T.BooleanType() , True),
T.StructField("source_config_Stats_hasChanged" , T.BooleanType() , True),
T.StructField("source_config_IrqBalance_hasChanged" , T.BooleanType() , True),
T.StructField("source_config_OurOUIs_hasChanged" , T.BooleanType() , True),
T.StructField("source_

In [51]:
pullSchema_new = T.StructType([
    T.StructField("ap" , T.StringType() , True),
    T.StructField("timestamp" , T.StringType() , True),
    T.StructField("row_key" , T.StringType() , True),
    T.StructField("start_time" , T.StringType() , True),
    T.StructField("time_delta" , T.StringType() , True),
    T.StructField("row_number" , T.StringType() , True),
    T.StructField("source_config_Radios_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_Hostname_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_WLANs_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_SVIs_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_LED_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_SSH_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_Stats_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_IrqBalance_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_OurOUIs_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_ID_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_UUID_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config__errors_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_Name_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_SiteName_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_HttpRedirector_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_SiteString_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_ProxyArp_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_Steer_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_WIDS_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_CnaHostnames_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_Dnsmasq_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_AppDef_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_Monitor_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_ExtIO_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_PwrMgmt_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_PoePwrConstrained_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_AVS_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_SwitchPorts_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_L2TPTunnels_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_VLANNames_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_APRecovery_hasChanged" , T.BooleanType() , True),
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
df0 = rdd0.map(lambda r: tuple([r[x_] for x_ in pickCols])).toDF(pullSchema_new)

df0.show(5,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+--------------------+-------------+-----------+----------+-------------------------------+---------------------------------+------------------------------+-----------------------------+----------------------------+----------------------------+------------------------------+-----------------------------------+--------------------------------+---------------------------+-----------------------------+--------------------------------+-----------------------------+---------------------------------+---------------------------------------+-----------------------------------+---------------------------------+------------------------------+-----------------------------+-------------------------------------+--------------------------------+-------------------------------+--------------------------------+------------------------------+--------------------------------+------------------------------------------+----------------------------+----------------------------

In [53]:
out_filename = "s3://mist-data-science-dev/aditya/scope_batch_aws_stg_dev_v2/set1/"
df0.coalesce(1).write.mode("overwrite").option('header', 'True').option('delimiter', '\t').option("quote",'"').option("escape",',').csv(out_filename)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h3>Bulk #2.3

- extra code to extract nested Radio and WLAN info

In [3]:
def returnFields(r):
    fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]','r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]','r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]','r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]','r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']
    retval = list()
    for x_ in fields:
        try:
            retval.append(eval(x_))
        except:
            retval.append(None)
    return retval

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
dt_ap = "dt=2022-01-24/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
hr_ap = "hr=14"

readLoc = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap + hr_ap
print(readLoc)
#---
print('Reading from... %s' %readLoc)
rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ev_name']=="AP_CONFIGURED") )
rdd_apevents_sample1 = rdd_apevents_sample.map(lambda r: returnFields(r)).persist()

pullSchema4 = T.StructType([T.StructField("ap" , T.StringType() , True), T.StructField("timestamp" , T.LongType() , True), \
    T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),T.StructField("source_config_WLANs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_SVIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_LED" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SSH" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Stats" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_IrqBalance" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_OurOUIs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_ID" , T.StringType() , True),T.StructField("source_config_UUID" , T.StringType() , True),T.StructField("source_config__errors" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Name" , T.StringType() , True),T.StructField("source_config_SiteName" , T.StringType() , True),T.StructField("source_config_HttpRedirector" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SiteString" , T.StringType() , True),T.StructField("source_config_ProxyArp" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_Steer" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_WIDS" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_CnaHostnames" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Dnsmasq" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AppDef" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Monitor" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_ExtIO" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_PwrMgmt" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_PoePwrConstrained" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AVS" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SwitchPorts" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_L2TPTunnels" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_VLANNames" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_APRecovery" , T.MapType(T.StringType(),T.StringType()) , True)
])

df_apevents_sample = rdd_apevents_sample1.toDF(pullSchema4)
# df_apevents_sample.printSchema()
df_apevents_sample.persist()
print(df_apevents_sample.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-24/hr=14
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-24/hr=14
9998

In [13]:
df_apevents_sample.select(F.size(F.col('source_config_WLANs'))).distinct().show(100,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+
|size(source_config_WLANs)|
+-------------------------+
|                       12|
|                        1|
|                        6|
|                        3|
|                        4|
|                        8|
|                       11|
|                        5|
|                        2|
|                       13|
|                        7|
|                        0|
|                        9|
+-------------------------+

In [ ]:
# +--------------------------+
# |size(source_config_Radios)|
# +--------------------------+
# |                         1|
# |                         3|
# |                         2|
# |                         0|
# +--------------------------+

In [14]:
sloc = "s3://mist-data-science-dev/aditya/scope_batch_v2/dt=202{1,2}-{01,12}-{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31}/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
rdd0 = sc.sequenceFile(sloc).map(lambda r: json.loads(r[1])).filter(lambda r: r['row_number']==1).persist()
print('\n',rdd0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 669

In [27]:
# T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),T.StructField("source_config_WLANs"

pullSchema_new = T.StructType([
    T.StructField("ap" , T.StringType() , True),
    T.StructField("timestamp" , T.StringType() , True),
    T.StructField("row_key" , T.StringType() , True),
    T.StructField("start_time" , T.StringType() , True),
    T.StructField("time_delta" , T.StringType() , True),
    T.StructField("row_number" , T.StringType() , True),
    T.StructField("source_config_Radios_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_WLANs_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_Radios_change" , T.StringType() , True),
    T.StructField("source_config_WLANs_change" , T.StringType() , True)
])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
pickCols = ['ap', 'timestamp', 'row_key', 'start_time', 'time_delta', 'row_number','source_config_Radios_hasChanged','source_config_WLANs_hasChanged',\
            'source_config_Radios_change','source_config_WLANs_change']
df0 = rdd0.map(lambda r: tuple([r[x_] for x_ in pickCols])).toDF(pullSchema_new)

df0.filter("source_config_Radios_hasChanged is true").show(100,)#.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+--------------------+-------------+-----------+----------+-------------------------------+------------------------------+---------------------------+--------------------------+
|          ap|       timestamp|             row_key|   start_time| time_delta|row_number|source_config_Radios_hasChanged|source_config_WLANs_hasChanged|source_config_Radios_change|source_config_WLANs_change|
+------------+----------------+--------------------+-------------+-----------+----------+-------------------------------+------------------------------+---------------------------+--------------------------+
|d4dc09ad276c|1640087453000000|d4dc09ad276c--9cd...|1640102769098|1.5316098E7|         1|                           true|                         false|       [[{'new': "ngth=1...|                        []|
|d420b0440042|1640158795000000|d420b0440042--183...|1640178872222|2.0077222E7|         1|                           true|                         false|       [[{'new':

In [33]:
# T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),T.StructField("source_config_WLANs"

pullSchema_new = T.StructType([
    T.StructField("ap" , T.StringType() , True),
    T.StructField("timestamp" , T.StringType() , True),
    T.StructField("row_key" , T.StringType() , True),
    T.StructField("start_time" , T.StringType() , True),
    T.StructField("time_delta" , T.StringType() , True),
    T.StructField("row_number" , T.StringType() , True),
    T.StructField("source_config_Radios_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_WLANs_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_Radios_change" , T.StringType() , True),
    T.StructField("source_config_WLANs_change" , T.StringType() , True),
    T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),
    T.StructField("source_config_Radios_step" , T.ArrayType(T.StringType()) , True),
    T.StructField("source_config_WLANs" , T.ArrayType(T.StringType()) , True),
    T.StructField("source_config_WLANs_step" , T.ArrayType(T.StringType()) , True)

])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
pickCols = ['ap', 'timestamp', 'row_key', 'start_time', 'time_delta', 'row_number','source_config_Radios_hasChanged','source_config_WLANs_hasChanged',\
            'source_config_Radios_change','source_config_WLANs_change','source_config_Radios','source_config_Radios_step',
           'source_config_WLANs','source_config_WLANs_step']
df0 = rdd0.map(lambda r: tuple([r[x_] for x_ in pickCols])).toDF(pullSchema_new)

# df0.filter("source_config_Radios_hasChanged is true").select('source_config_Radios','source_config_Radios_step').show(100,)#.count()
df0.filter("source_config_Radios_hasChanged is true").select(F.size(F.col('source_config_Radios')).alias('col1'),
                                                             F.size(F.col('source_config_Radios_step')).alias('col2'))\
.filter('col1 != col2').show(100,)#.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+
|col1|col2|
+----+----+
|   2|  -1|
|   2|  -1|
|   2|  -1|
|   2|  -1|
|   2|  -1|
|   2|  -1|
|   2|  -1|
+----+----+

In [36]:
df0.filter("source_config_WLANs_hasChanged is true").select(F.size(F.col('source_config_WLANs')).alias('col1'),
                                                             F.size(F.col('source_config_WLANs_step')).alias('col2'))\
.filter('col1 != col2').show(100,)#.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----+
|col1|col2|
+----+----+
|   2|   1|
|   4|  -1|
|   4|  -1|
|   5|   4|
|   5|   4|
|   5|   4|
|   5|  -1|
|   4|   5|
|   5|   4|
|   5|   4|
|   4|   5|
|   5|   4|
|   5|  -1|
|   4|  -1|
|   5|  -1|
|   6|   7|
|   3|  -1|
|   3|   2|
|   3|   2|
|   3|   2|
|   3|   2|
|   3|   2|
|   3|   2|
+----+----+

In [39]:
df0.filter("source_config_WLANs_hasChanged is true").select('source_config_WLANs','source_config_WLANs_step',
                                                            F.size(F.col('source_config_WLANs')).alias('col1'),
                                                             F.size(F.col('source_config_WLANs_step')).alias('col2')).take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(source_config_WLANs=['{Acct_Interim_Interval=0, BandConfigs=[{BSSIDIndex=5, Band=5}], AMPDU={RST=true}, RateLimit={WlanDown=0, WlanUp=0, ClientUp=0, ClientDown=0}, WNM={BSSTransitionManagement={Passive=true, Active=false}, BSSLoad=true, NeighborReport=true}, Auth={Type=psk, AKM={WPA2_PSK=true}, WEP=null, GroupRekeyIntervalSec=86400, GroupMasterRekeyIntervalSec=86400, PSK=21122021, Pairwise={WPA1={CCMP=false, TKIP=false}, TKIP=false, CCMP=true}, MultiPSK=null}, Hidden=false, ID=c7638ac1-9c7f-469f-afc1-0f66e35e6b74, SSID=OEM_Flash, DisabledIEs={}, roamMode=none, Enabled=true, BcastFilter={Mdns_allow=false, Dhcp=true, DHCPBcast2UnicastResponse=true, Ssdp_allow=false, Mcast=true, DropDHCPUnknownResponse=true, AllowICMPv6NeighborDiscovery=false, Arp=true}, NoStaticIP=false, MacAuth={Enabled=false}, RestrictDNS={FromDHCP=false}, Bands=[5], template_name=OEM Flash, QOS={Enabled=true, AccessClass=4}, RateSets=[{Band=5, LegacyRates=[36, 48, 54], HTMCSRateMask=00f800f800f800f8, VHTMCSRateMa

<h4>subcategory on radios

In [40]:
df0.filter("source_config_Radios_hasChanged is true").select('source_config_Radios','source_config_Radios_step',
                                                            F.size(F.col('source_config_Radios')).alias('col1'),
                                                             F.size(F.col('source_config_Radios_step')).alias('col2')).show(10,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------------+----+----+
|source_config_Radios|source_config_Radios_step|col1|col2|
+--------------------+-------------------------+----+----+
|[{ProbeControl={P...|     [{ProbeControl={P...|   2|   2|
|[{ProbeControl={P...|     [{ProbeControl={P...|   2|   2|
|[{ProbeControl={P...|     [{ProbeControl={P...|   2|   2|
|[{Mitigation={TxP...|     [{Mitigation={TxP...|   2|   2|
|[{ProbeControl={P...|     [{ProbeControl={P...|   2|   2|
|[{ProbeControl={P...|     [{ProbeControl={P...|   2|   2|
|[{ProbeControl={P...|     [{ProbeControl={P...|   2|   2|
|[{ProbeControl={P...|                     null|   2|  -1|
|[{ProbeControl={P...|     [{ProbeControl={P...|   2|   2|
|[{ProbeControl={P...|     [{ProbeControl={P...|   2|   2|
+--------------------+-------------------------+----+----+
only showing top 10 rows

In [66]:
df0.select(F.from_json(F.col("source_config_Radios").cast(T.StringType()), 
           schema=T.StructType([
    T.StructField("ProbeControl" , T.StringType() , True),
    T.StructField("timestamp" , T.StringType() , True)]))).show(2,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------+
|from_json(CAST(source_config_Radios AS STRING))|
+-----------------------------------------------+
|                                   {null, null}|
|                                   {null, null}|
+-----------------------------------------------+
only showing top 2 rows

In [56]:
def RadioChange(x1, x2):
    k1 = eval(x1[0])['Band']
    if x2 is not None:
        k2 = len(x2)
    else:
        k2 = ""
    return str([k1,k2])

udf_radio_change = F.udf(RadioChange, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
df0.filter("source_config_Radios_hasChanged is true").select('source_config_Radios','source_config_Radios_step',
                                                            F.size(F.col('source_config_Radios')).alias('col1'),F.size(F.col('source_config_Radios_step')).alias('col2'),
                                                            udf_radio_change(F.col('source_config_Radios'),F.col('source_config_Radios_step'))).show(10,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:

  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1642782761878_0151/container_1642782761878_0151_01_000081/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/mnt/yarn/usercache/livy/appcache/application_1642782761878_0151/container_1642782761878_0151_01_000081/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/mnt/yarn/usercache/livy/appcache/application_1642782761878_0151/container_1642782761878_0151_01_000081/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/mnt/yarn/usercache/livy/appcache/application_1642782761878_0151/container_1642782761878_0151_01_000081/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/mnt/yarn/usercache/l

In [187]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h4>Extract Radios

In [188]:
def returnFields(r):
    fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["WLANs"]']
    retval = list()
    for x_ in fields:
        try:
            retval.append(eval(x_))
        except:
            retval.append(None)
    return retval

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [189]:
dt_ap = "dt=2022-01-24/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
hr_ap = "hr=14"

readLoc = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap + hr_ap
print(readLoc)
#---
print('Reading from... %s' %readLoc)
rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ev_name']=="AP_CONFIGURED") )
rdd_apevents_sample1 = rdd_apevents_sample.map(lambda r: returnFields(r)).persist()



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-24/hr=14
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-24/hr=14

In [197]:
wlan_columns = ["roamMode","Nas_ID","auth_servers","acct_servers","dynamic_vlan","RadiusTimeout","Acct_Interim_Interval","Bands","RateSets","BandConfigs","SSID","Enabled","Auth","MacAuth","Vlans","Isolation","BcastFilter","Hidden","NoStaticIP","RestrictDNS","DisabledIEs","ForMesh","ID","Wxlan","WxlanRule","QOS","AMPDU","RateLimit","WNM","WCInactivity"]
start_str = "T.ArrayType(T.StructType(["
for x_ in wlan_columns:
    start_str = start_str + "T.StructField(\"" + x_ + "\", T.StringType() , True),\n"
start_str = start_str[:-2] + "]))"
print(start_str)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

T.ArrayType(T.StructType([T.StructField("roamMode", T.StringType() , True),
T.StructField("Nas_ID", T.StringType() , True),
T.StructField("auth_servers", T.StringType() , True),
T.StructField("acct_servers", T.StringType() , True),
T.StructField("dynamic_vlan", T.StringType() , True),
T.StructField("RadiusTimeout", T.StringType() , True),
T.StructField("Acct_Interim_Interval", T.StringType() , True),
T.StructField("Bands", T.StringType() , True),
T.StructField("RateSets", T.StringType() , True),
T.StructField("BandConfigs", T.StringType() , True),
T.StructField("SSID", T.StringType() , True),
T.StructField("Enabled", T.StringType() , True),
T.StructField("Auth", T.StringType() , True),
T.StructField("MacAuth", T.StringType() , True),
T.StructField("Vlans", T.StringType() , True),
T.StructField("Isolation", T.StringType() , True),
T.StructField("BcastFilter", T.StringType() , True),
T.StructField("Hidden", T.StringType() , True),
T.StructField("NoStaticIP", T.StringType() , True),
T.Str

In [202]:
schema_wlan = T.ArrayType(T.StructType([T.StructField("roamMode", T.StringType() , True),
T.StructField("Nas_ID", T.StringType() , True),
T.StructField("auth_servers", T.StringType() , True),
T.StructField("acct_servers", T.StringType() , True),
T.StructField("dynamic_vlan", T.StringType() , True),
T.StructField("RadiusTimeout", T.StringType() , True),
T.StructField("Acct_Interim_Interval", T.StringType() , True),
T.StructField("Bands", T.StringType() , True),
T.StructField("RateSets", T.StringType() , True),
T.StructField("BandConfigs", T.StringType() , True),
T.StructField("SSID", T.StringType() , True),
T.StructField("Enabled", T.StringType() , True),
T.StructField("Auth", T.StringType() , True),
T.StructField("MacAuth", T.StringType() , True),
T.StructField("Vlans", T.StringType() , True),
T.StructField("Isolation", T.StringType() , True),
T.StructField("BcastFilter", T.StringType() , True),
T.StructField("Hidden", T.StringType() , True),
T.StructField("NoStaticIP", T.StringType() , True),
T.StructField("RestrictDNS", T.StringType() , True),
T.StructField("DisabledIEs", T.StringType() , True),
T.StructField("ForMesh", T.StringType() , True),
T.StructField("ID", T.StringType() , True),
T.StructField("Wxlan", T.StringType() , True),
T.StructField("WxlanRule", T.StringType() , True),
T.StructField("QOS", T.StringType() , True),
T.StructField("AMPDU", T.StringType() , True),
T.StructField("RateLimit", T.StringType() , True),
T.StructField("WNM", T.StringType() , True),
T.StructField("WCInactivity", T.StringType() , True)]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [201]:
schema_radios = T.ArrayType(T.StructType([
    T.StructField("Band" , T.StringType() , True),
    T.StructField("Width" , T.StringType() , True),
    T.StructField("Power" , T.StringType() , True),
    T.StructField("PreambleLength" , T.StringType() , True),
    T.StructField("QOS" , T.StringType() , True),
    T.StructField("Channel" , T.StringType() , True),
    T.StructField("RRM" , T.StringType() , True),
    T.StructField("NonAggrRegulate" , T.StringType() , True),
    T.StructField("ProbeControl" , T.StringType() , True), 
    ]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [205]:
pullSchema6 = T.StructType([T.StructField("ap" , T.StringType() , True), T.StructField("timestamp" , T.LongType() , True), \
    T.StructField("source_config_Radios" , schema_radios, True),
    T.StructField("source_config_WLANs", schema_wlan, True)])

df_apevents_sample = rdd_apevents_sample1.toDF(pullSchema6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [206]:
df_apevents_sample.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(ap='5c5b353eab63', timestamp=1643032804000000, source_config_Radios=[Row(Band='2.4', Width='20', Power='8', PreambleLength='1', QOS='{Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1}, {DSCP=8, AC=1}, {DSCP=16, AC=1}, {DSCP=9, AC=1}, {DSCP=11, AC=1}, {DSCP=13, AC=1}, {DSCP=15, AC=1}, {DSCP=17, AC=1}, {DSCP=19, AC=1}, {DSCP=21, AC=1}, {DSCP=23, AC=1}]}', Channel='0', RRM='{Width=20, Power=8, Channel=1}', NonAggrRegulate='{Disabled=false, Enabled=true}', ProbeControl='{ProbeReqSW=true, ProbeRespSW=true}'), Row(Band='5', Width='40', Power='16', PreambleLength='1', QOS='{Dscp2Ac=[{DSCP=10, AC=1}, {DSCP=12, AC=1}, {DSCP=14, AC=1}, {DSCP=18, AC=1}, {DSCP=20, AC=1}, {DSCP=22, AC=1}, {DSCP=8, AC=1}, {DSCP=16, AC=1}, {DSCP=9, AC=1}, {DSCP=11, AC=1}, {DSCP=13, AC=1}, {DSCP=15, AC=1}, {DSCP=17, AC=1}, {DSCP=19, AC=1}, {DSCP=21, AC=1}, {DSCP=23, AC=1}]}', Channel='0', RRM='{Width=40, Power=16, Channel=100}', NonAggrRegulate='{Disable

In [101]:
df_apevents_sample.select('ap','timestamp', F.to_json('source_config_Radios')).show(10,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [185]:
def RadioChange(x1, x2):
    k1 = eval(x1)
    if x2 is not None:
        k2 = eval(x2)
        if len(k1)==2 and len(k2)==2:
            diff_fields = list()
            for counter in range(len(k1)):
                elem = k1[counter]
                prefix = str(elem['Band'])+"_"
                elem_n = k2[counter]
                k0_0 = set(sorted(elem.keys()))
                k0_1 = set(sorted(elem_n.keys()))
                k_overlap = set.intersection(k0_0,k0_1)
                k0_add = list(set.difference(k0_0, k0_1))
                k1_add = list(set.difference(k0_1, k0_0))
                for x__ in k0_add:
                    diff_fields.append(prefix+"add_"+str(x__))
                for x__ in k1_add:
                    diff_fields.append(prefix+"add_"+str(x__))
                
                for elem_k0 in list(k_overlap):
                    if elem[elem_k0]!=elem_n[elem_k0]:
                        diff_fields.append(prefix+elem_k0)
            r1 = sorted(diff_fields)
        else:
            r1 = "unk"
    else:
        r1="unk"
    return str(r1)

udf_radio_change = F.udf(RadioChange, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [186]:
step_window = Window.partitionBy('ap').orderBy(F.col("timestamp").desc())

df_apevents_sample.select('ap','timestamp', F.to_json('source_config_Radios').alias("source_config_Radios"))\
.select('ap','timestamp','source_config_Radios', F.lag('source_config_Radios', offset=1).over(step_window).alias('source_config_Radios_step'))\
.select('ap','timestamp','source_config_Radios','source_config_Radios_step', udf_radio_change(F.col('source_config_Radios'),F.col('source_config_Radios_step')).alias('newcol'))\
.select('newcol').filter('newcol not in ("[]", "unk")').distinct().show(100,False)

# df_apevents_sample.select('ap','timestamp', F.to_json('source_config_Radios').alias("source_config_Radios"))\
# .select('ap','timestamp','source_config_Radios', F.lag('source_config_Radios', offset=1).over(step_window).alias('source_config_Radios_step'))\
# .select(udf_radio_change(F.col('source_config_Radios'),F.col('source_config_Radios_step')))\
# .distinct().show(10,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------+
|newcol                                          |
+------------------------------------------------+
|['2.4_RRM', '2.4_Width']                        |
|['5_Power', '5_RRM']                            |
|['5_Channel', '5_Width', '5_add_RRM']           |
|['2.4_RRM']                                     |
|['2.4_Power', '2.4_RRM']                        |
|['5_RRM']                                       |
|['5_Channel', '5_Power', '5_add_RRM']           |
|['2.4_Power', '2.4_RRM', '5_Power', '5_RRM']    |
|['2.4_RRM', '5_RRM']                            |
|['5_Channel', '5_Power', '5_Width', '5_add_RRM']|
|['2.4_RRM', '5_Power', '5_RRM']                 |
|['2.4_Power', '2.4_RRM', '5_RRM']               |
+------------------------------------------------+

- wlan drill

In [212]:
df_apevents_sample.select('ap','timestamp', F.size(F.col('source_config_WLANs')).alias('newcol2')).groupBy('newcol2').agg(F.count('ap')).show(10,)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+
|newcol2|count(ap)|
+-------+---------+
|     12|       23|
|      1|      299|
|      6|      312|
|      3|     1521|
|      4|      955|
|      8|      117|
|     11|      638|
|      5|     1324|
|      2|     1123|
|     13|        2|
+-------+---------+
only showing top 10 rows

- there is no standardized arraysize to wlan : this is expected because every AP can define this independently

<h4>Extract WLANs

In [217]:
def WlanChange(x1, x2):
    k1 = eval(x1)
    if x2 is not None:
        k2 = eval(x2)
        if len(k1)==len(k2):
            diff_fields = list()
            for counter in range(len(k1)):
                elem = k1[counter]
                #prefix = str(elem['ID'])+"_"
                prefix=""
                elem_n = k2[counter]
                k0_0 = set(sorted(elem.keys()))
                k0_1 = set(sorted(elem_n.keys()))
                k_overlap = set.intersection(k0_0,k0_1)
                k0_add = list(set.difference(k0_0, k0_1))
                k1_add = list(set.difference(k0_1, k0_0))
                for x__ in k0_add:
                    diff_fields.append(prefix+"add_"+str(x__))
                for x__ in k1_add:
                    diff_fields.append(prefix+"add_"+str(x__))
                
                for elem_k0 in list(k_overlap):
                    if elem[elem_k0]!=elem_n[elem_k0]:
                        diff_fields.append(prefix+elem_k0)
            r1 = sorted(diff_fields)
        else:
            r1 = "unk"
    else:
        r1="unk"
    return str(r1)

udf_wlan_change = F.udf(WlanChange, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [219]:
step_window = Window.partitionBy('ap').orderBy(F.col("timestamp").desc())

df_apevents_sample.select('ap','timestamp',F.size(F.col('source_config_WLANs')).alias('countcol'), F.to_json('source_config_WLANs').alias("source_config_WLANs"))\
.select('ap','timestamp','source_config_WLANs','countcol', F.lag('source_config_WLANs', offset=1).over(step_window).alias('source_config_WLANs_step'))\
.select('ap','timestamp','source_config_WLANs','source_config_WLANs_step','countcol', udf_wlan_change(F.col('source_config_WLANs'),F.col('source_config_WLANs_step')).alias('newcol2'))\
.select('newcol2').filter('newcol2 not in ("[]", "unk")').distinct().show(100,False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------------------------------------+
|newcol2                                                                                        |
+-----------------------------------------------------------------------------------------------+
|['auth_servers']                                                                               |
|['Vlans']                                                                                      |
|['SSID']                                                                                       |
|['BandConfigs', 'BandConfigs', 'ID', 'ID', 'Vlans', 'Vlans', 'Wxlan']                          |
|['Auth']                                                                                       |
|['SSID', 'Wxlan']                                                                              |
|['Wxlan']                                                                                      |
|['Acct_Interim_Inte

<h4>pull for existing

In [31]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# sloc = "s3://mist-data-science-dev/aditya/scope_batch_v2/dt=202{1,2}-{01,12}-{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31}/"
sloc = "s3://mist-data-science-dev/aditya/scope_batch_v2/dt=2022-01-{24}/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
rdd0 = sc.sequenceFile(sloc).map(lambda r: json.loads(r[1])).filter(lambda r: r['row_number']==1).persist()
print('\n',rdd0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 56

In [34]:
"timestamp_step" in list(rdd0.take(1)[0].keys())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [39]:
# T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),T.StructField("source_config_WLANs"

pullSchema_new = T.StructType([
    T.StructField("ap" , T.StringType() , True),
    T.StructField("timestamp" , T.StringType() , True),
    T.StructField("row_key" , T.StringType() , True),
    T.StructField("start_time" , T.StringType() , True),
    T.StructField("time_delta" , T.StringType() , True),
    T.StructField("row_number" , T.StringType() , True),
    T.StructField("source_config_Radios_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_WLANs_hasChanged" , T.BooleanType() , True),
    T.StructField("timestamp_step" , T.StringType() , True),
    T.StructField("source_config_Radios_step" , T.StringType() , True),
    T.StructField("source_config_WLANs_step" , T.StringType() , True)])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
pickCols = ['ap', 'timestamp', 'row_key', 'start_time', 'time_delta', 'row_number','source_config_Radios_hasChanged','source_config_WLANs_hasChanged',\
            'timestamp_step','source_config_Radios_step','source_config_WLANs_step']
df0 = rdd0.map(lambda r: tuple([r[x_] for x_ in pickCols])).toDF(pullSchema_new)

df0.filter("source_config_Radios_hasChanged is true").show(100,)#.count()

df0.filter("source_config_WLANs_hasChanged is true").show(100,)#.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+--------------------+-------------+-----------+----------+-------------------------------+------------------------------+----------------+-------------------------+------------------------+
|          ap|       timestamp|             row_key|   start_time| time_delta|row_number|source_config_Radios_hasChanged|source_config_WLANs_hasChanged|  timestamp_step|source_config_Radios_step|source_config_WLANs_step|
+------------+----------------+--------------------+-------------+-----------+----------+-------------------------------+------------------------------+----------------+-------------------------+------------------------+
|d420b0442072|1642925486000000|d420b0442072&conn...|1642932003926|  6517926.0|         1|                           true|                         false|1642789697000000|     [{ProbeControl={P...|    [{mdid=32283, Acc...|
|d420b0845461|1643019242000000|d420b0845461--176...|1643034458987|1.5216987E7|         1|                           

In [230]:
df0.filter("source_config_Radios_hasChanged is true").count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

292

In [231]:
df0.filter("source_config_Radios_hasChanged is true and row_number==1").count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

292

In [232]:
df0.filter("source_config_Radios_hasChanged is true").select("ap").distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

177

In [238]:
ap_list_filter = [x_.ap for x_ in df0.filter("source_config_Radios_hasChanged is true").select("ap").distinct().collect()]
# [x.ap for x in df_select_scope_events_apscope.select('ap').distinct().dropna().collect()]
print(ap_list_filter[:4])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['d420b0442072', '5c5b350e21f7', '5c5b35f24438', 'd420b084a6a0']

In [42]:
# sloc = "s3://mist-data-science-dev/aditya/scope_batch_v2/dt=202{1,2}-{01,12}-{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31}/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [241]:
dt_ap = "dt=2022-01-{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
hr_ap = "hr=*/*/"

readLoc = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap + hr_ap
print(readLoc)
#---
print('Reading from... %s' %readLoc)
rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list_filter) and (r['source']['ev_name']=="AP_CONFIGURED") )
rdd_apevents_sample1 = rdd_apevents_sample.map(lambda r: returnFields(r)).persist()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24}/hr=*/*/

In [242]:
rdd_apevents_sample1.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interrupted by user


- this is too time consuming 

<h4>bulk update

In [9]:
def returnFields(r):
    fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]',
              'r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]',
              'r["source"]["config"]["OurOUIs"]','r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]',
              'r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]',
              'r["source"]["config"]["Steer"]','r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]',
              'r["source"]["config"]["Monitor"]','r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]',
              'r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]',
              'r["source"]["config"]["Radios"]','r["source"]["config"]["WLANs"]']
    retval = list()
    for x_ in fields:
        try:
            retval.append(eval(x_))
        except:
            retval.append(None)
    return retval

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
def getconfigdiff_def(x, y):
    if (x == "") or (y == ""):
        return "'config not found'"
    else:
        try:
            j1 = str(x)  # json.loads(x)
            j2 = str(y)  # json.loads(y)
            l1 = [x for x in difflib.unified_diff(j1, j2, n=100, lineterm='') if "@@" in x]
            retlist = list()
            for x_ in l1:
                sp = x_.replace("@", "").replace(" ", "").split("+")[1].split(",")
                s_ = int(sp[0])
                e_ = int(sp[1])
                if sp != []:
                    print(s_, e_)
                    retlist.append([{"new": j1[s_: s_ + e_ - 1], "old": j2[s_: s_ + e_ - 1]}])
            retval = str(retlist)
            return retval
        except:
            retval = "error"
        return retval

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
getconfigDiff = F.udf(getconfigdiff_def, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
wlan_columns = ["roamMode","Nas_ID","auth_servers","acct_servers","dynamic_vlan","RadiusTimeout","Acct_Interim_Interval","Bands","RateSets","BandConfigs","SSID","Enabled","Auth","MacAuth","Vlans","Isolation","BcastFilter","Hidden","NoStaticIP","RestrictDNS","DisabledIEs","ForMesh","ID","Wxlan","WxlanRule","QOS","AMPDU","RateLimit","WNM","WCInactivity"]
start_str = "T.ArrayType(T.StructType(["
for x_ in wlan_columns:
    start_str = start_str + "T.StructField(\"" + x_ + "\", T.StringType() , True),\n"
start_str = start_str[:-2] + "]))"
# print(start_str)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
schema_wlan = T.ArrayType(T.StructType([T.StructField("roamMode", T.StringType() , True),
T.StructField("Nas_ID", T.StringType() , True),
T.StructField("auth_servers", T.StringType() , True),
T.StructField("acct_servers", T.StringType() , True),
T.StructField("dynamic_vlan", T.StringType() , True),
T.StructField("RadiusTimeout", T.StringType() , True),
T.StructField("Acct_Interim_Interval", T.StringType() , True),
T.StructField("Bands", T.StringType() , True),
T.StructField("RateSets", T.StringType() , True),
T.StructField("BandConfigs", T.StringType() , True),
T.StructField("SSID", T.StringType() , True),
T.StructField("Enabled", T.StringType() , True),
T.StructField("Auth", T.StringType() , True),
T.StructField("MacAuth", T.StringType() , True),
T.StructField("Vlans", T.StringType() , True),
T.StructField("Isolation", T.StringType() , True),
T.StructField("BcastFilter", T.StringType() , True),
T.StructField("Hidden", T.StringType() , True),
T.StructField("NoStaticIP", T.StringType() , True),
T.StructField("RestrictDNS", T.StringType() , True),
T.StructField("DisabledIEs", T.StringType() , True),
T.StructField("ForMesh", T.StringType() , True),
T.StructField("ID", T.StringType() , True),
T.StructField("Wxlan", T.StringType() , True),
T.StructField("WxlanRule", T.StringType() , True),
T.StructField("QOS", T.StringType() , True),
T.StructField("AMPDU", T.StringType() , True),
T.StructField("RateLimit", T.StringType() , True),
T.StructField("WNM", T.StringType() , True),
T.StructField("WCInactivity", T.StringType() , True)]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
schema_radios = T.ArrayType(T.StructType([
    T.StructField("Band" , T.StringType() , True),
    T.StructField("Width" , T.StringType() , True),
    T.StructField("Power" , T.StringType() , True),
    T.StructField("PreambleLength" , T.StringType() , True),
    T.StructField("QOS" , T.StringType() , True),
    T.StructField("Channel" , T.StringType() , True),
    T.StructField("RRM" , T.StringType() , True),
    T.StructField("NonAggrRegulate" , T.StringType() , True),
    T.StructField("ProbeControl" , T.StringType() , True), 
    ]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# pullSchema6 = T.StructType([T.StructField("ap" , T.StringType() , True), T.StructField("timestamp" , T.LongType() , True), \
#     T.StructField("source_config_Radios" , schema_radios, True),
#     T.StructField("source_config_WLANs", schema_wlan, True)])

# df_apevents_sample = rdd_apevents_sample1.toDF(pullSchema6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
def RadioChange(x1, x2):
    k1 = eval(x1)
    if x2 is not None:
        k2 = eval(x2)
        if len(k1)==2 and len(k2)==2:
            diff_fields = list()
            for counter in range(len(k1)):
                elem = k1[counter]
                prefix = str(elem['Band'])+"_"
                elem_n = k2[counter]
                k0_0 = set(sorted(elem.keys()))
                k0_1 = set(sorted(elem_n.keys()))
                k_overlap = set.intersection(k0_0,k0_1)
                k0_add = list(set.difference(k0_0, k0_1))
                k1_add = list(set.difference(k0_1, k0_0))
                for x__ in k0_add:
                    diff_fields.append(prefix+"add_"+str(x__))
                for x__ in k1_add:
                    diff_fields.append(prefix+"add_"+str(x__))
                
                for elem_k0 in list(k_overlap):
                    if elem[elem_k0]!=elem_n[elem_k0]:
                        diff_fields.append(prefix+elem_k0)
            r1 = sorted(diff_fields)
        else:
            r1 = "unk"
    else:
        r1="unk"
    return str(r1)

udf_radio_change = F.udf(RadioChange, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
def WlanChange(x1, x2):
    k1 = eval(x1)
    if x2 is not None:
        k2 = eval(x2)
        if len(k1)==len(k2):
            diff_fields = list()
            for counter in range(len(k1)):
                elem = k1[counter]
                #prefix = str(elem['ID'])+"_"
                prefix=""
                elem_n = k2[counter]
                k0_0 = set(sorted(elem.keys()))
                k0_1 = set(sorted(elem_n.keys()))
                k_overlap = set.intersection(k0_0,k0_1)
                k0_add = list(set.difference(k0_0, k0_1))
                k1_add = list(set.difference(k0_1, k0_0))
                for x__ in k0_add:
                    diff_fields.append(prefix+"add_"+str(x__))
                for x__ in k1_add:
                    diff_fields.append(prefix+"add_"+str(x__))
                
                for elem_k0 in list(k_overlap):
                    if elem[elem_k0]!=elem_n[elem_k0]:
                        diff_fields.append(prefix+elem_k0)
            r1 = sorted(diff_fields)
        else:
            r1 = "unk"
    else:
        r1="unk"
    return str(r1)

udf_wlan_change = F.udf(WlanChange, T.StringType())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# def ExtractConfigs(sc, var_year, var_month, var_date):
    # var_date = 20
var_year=2022
var_month=1
var_date=24
var_hr = 14
start_time = time.time()
logs = list()
log1 = "Start time: %d" %start_time
print(log1)
logs.append(log1)
# ---
s3locRoot_ee = 's3://mist-aggregated-stats-production/entity_event/'
s3locChild_ee = 'entity_event-production/'
dt_ee = "dt=%s-%s-%s/" % (str(var_year), str(var_month).zfill(2), str(var_date).zfill(2))
hr_ee = "hr=*/*/" #  "hr=14/" 
s3loc_ee = s3locRoot_ee + s3locChild_ee + dt_ee + hr_ee
print('\n--- s3loc_ee ---')
print(s3loc_ee)
logs.append(s3loc_ee)

event_rdd_ee = sc.sequenceFile(s3loc_ee).map(lambda r: json.loads(r[1]))#.persist()
event2consider = ['connectivity_failure']
event_rdd_ee_scope = event_rdd_ee.filter(lambda r: (('event_name' in r.keys() ) and (r['event_name'] in event2consider) \
                                                    and ('ap' in r['reason'].keys()) and (r['severity'] > 50) and r['reason']['ap']=="d420b0c0b833") ) # d420b0c0b833 d420b0845461
# ---
pull_schema = T.StructType([
T.StructField('row_key', T.StringType(), True),
T.StructField('start_time', T.LongType(), True),
T.StructField('ap', T.StringType(), True)
])
df_select_scope_events_apscope = event_rdd_ee_scope.map(
    lambda r: (r['row_key'], r['start_time'], r['reason']['ap'])).distinct().toDF(pull_schema)

df_select_scope_events_apscope.persist()
print('\n--- Scope events with AP scope ---')
print(df_select_scope_events_apscope.count())
#---
# ap_list = [x.ap for x in df_select_scope_events_apscope.select('ap').distinct().dropna().collect()]
ap_list = ["d420b0c0b833"]
log2 = '\n--- APs involved --- \n %s' %str(len(ap_list))
print(log2)
logs.append(log2)
log3  = 'Some ap ids: %s' %str(ap_list[:10])
logs.append(log3)
print(log3)
# ---
strval = "dt=%s-%s-{" %(str(var_year), str(var_month).zfill(2))
for x in range(var_date, var_date-2, -1): #this wont work for 1st
    strval= strval + str(x).zfill(2) + ","
print(strval[:-1] +"}/")
dt_ap0 = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
hr_ap0 = "hr=*/*/"
readLoc0 = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap0 + hr_ap0
print(readLoc0)
pullSchema2 = T.StructType([       
    T.StructField('ap', T.StringType(), True),
    T.StructField('ev_name', T.StringType(), True),
    T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),
    T.StructField('timestamp', T.LongType(), True),
])
print('Reading from... %s' %readLoc0)
rdd_apevents0 = sc.sequenceFile(readLoc0).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample0 = rdd_apevents0.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list) and (r['source']['ev_name']=="AP_CONFIGURED") )


df_apevents_sample0 = rdd_apevents_sample0.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)
dfm0 = df_select_scope_events_apscope.select('row_key', 'start_time', 'ap')\
                                        .join(df_apevents_sample0, on='ap', how='inner')\
                                        .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                        .filter('time_delta>0 and time_delta <' + str(6*60*60*1000))
# ap_list_positive = [x.ap for x in dfm0.select('ap').distinct().dropna().collect()]
ap_list_positive = ["d420b0c0b833"]
log4 = '\n---APs that had a config event within 6 hours ---\n %s' %str(len(ap_list_positive))
print(log4)
logs.append(log4)
#---
strval = "dt=%s-%s-{" %(str(var_year), str(var_month).zfill(2))
# strval = "dt=2021-12-{"
for x in range(var_date, var_date-8, -1): #this wont work for dates before 8th
    strval= strval + str(x).zfill(2) + ","
print(strval[:-1] +"}/")

dt_ap = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
hr_ap = "hr=*/*/"

readLoc = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap + hr_ap
print(readLoc)
#---
print('Reading from... %s' %readLoc)
rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                     and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list_positive) and (r['source']['ev_name']=="AP_CONFIGURED") )
rdd_apevents_sample1 = rdd_apevents_sample.map(lambda r: returnFields(r))

pullSchema4 = T.StructType([T.StructField("ap" , T.StringType() , True), T.StructField("timestamp" , T.LongType() , True), 
                            T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),
                            T.StructField("source_config_WLANs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_SVIs" , T.ArrayType(T.StringType()) , True),
                            T.StructField("source_config_LED" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SSH" , T.MapType(T.StringType(),T.StringType()), True),
                            T.StructField("source_config_Stats" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_IrqBalance" , T.MapType(T.StringType(),T.StringType()) , True),
                            T.StructField("source_config_OurOUIs" , T.ArrayType(T.StringType()) , True),
                            T.StructField("source_config_ID" , T.StringType() , True),T.StructField("source_config_UUID" , T.StringType() , True),
                            T.StructField("source_config__errors" , T.ArrayType(T.StringType()) , True),
                            T.StructField("source_config_Name" , T.StringType() , True),T.StructField("source_config_SiteName" , T.StringType() , True),
                            T.StructField("source_config_HttpRedirector" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SiteString" , T.StringType() , True),
                            T.StructField("source_config_ProxyArp" , T.MapType(T.StringType(),T.StringType()) , True),
                            T.StructField("source_config_Steer" , T.MapType(T.StringType(),T.StringType()) , True),
                            T.StructField("source_config_WIDS" , T.ArrayType(T.StringType()) , True),
                            T.StructField("source_config_CnaHostnames" , T.ArrayType(T.StringType()) , True),
                            T.StructField("source_config_Dnsmasq" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AppDef" , T.ArrayType(T.StringType()) , True),
                            T.StructField("source_config_Monitor" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_ExtIO" , T.ArrayType(T.StringType()) , True),
                            T.StructField("source_config_PwrMgmt" , T.MapType(T.StringType(),T.StringType()) , True),
                            T.StructField("source_config_PoePwrConstrained" , T.MapType(T.StringType(),T.StringType()) , True),
                            T.StructField("source_config_AVS" , T.MapType(T.StringType(),T.StringType()) , True),
                            T.StructField("source_config_SwitchPorts" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_L2TPTunnels" , T.ArrayType(T.StringType()) , True),
                            T.StructField("source_config_VLANNames" , T.MapType(T.StringType(),T.StringType()) , True),
                            T.StructField("source_config_APRecovery" , T.MapType(T.StringType(),T.StringType()) , True),
                            T.StructField("source_config_Radios_e" , schema_radios, True),T.StructField("source_config_WLANs_e", schema_wlan, True)
])

df_apevents_sample = rdd_apevents_sample1.toDF(pullSchema4)
# df_apevents_sample.printSchema()
df_apevents_sample.persist()
log5 = '\n--- AP configured events pulled for the last 7 days ---\n %s' %str(df_apevents_sample.count())
print(log5)
logs.append(log5)
# #---
fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]',
          'r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]',
          'r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]',
          'r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]',
          'r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']

L1 = [x_.replace("r[","").replace("\"][\"", "_").replace("]","").replace("\"","")+ '_step' for x_ in fields[2:]]
#---
step_window = Window.partitionBy('ap').orderBy(F.col("row_number").desc())

df_apscope_withconfig = df_select_scope_events_apscope.join(df_apevents_sample, on ='ap', how='inner')\
                                .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                .filter('time_delta>0')\
                                .withColumn('row_number', F.row_number().over(Window.partitionBy('ap').orderBy(F.col('time_delta').asc())))

df_apscope_withconfig = df_apscope_withconfig.select('ap', 'timestamp','row_key', 'start_time','time_delta', 'row_number',\
                                                     *[c.replace("_step", "").replace("\"", "") for c in L1], \
                                                     *[F.lag((c.replace("_step", "").replace("\"", "")), offset=1).over(step_window).alias(c) for c in L1],\
                                                    F.lag('timestamp', offset=1).over(step_window).alias('timestamp_step'),
                                                     'source_config_Radios_e',F.lag('source_config_Radios_e', offset=1).over(step_window).alias('source_config_Radios_e_step'),
                                                     'source_config_WLANs_e', F.lag('source_config_WLANs_e', offset=1).over(step_window).alias('source_config_WLANs_e_step')
                                                    )
#---
L0 = df_apscope_withconfig.columns
df_apscope_withconfigdiff = df_apscope_withconfig.select(*[c for c in L0],
                                                         *[getconfigDiff(F.col(c.replace("_step","")), F.col(c) ).alias(c.replace("_step","_change")) for c in L1],
                                                        udf_radio_change(F.to_json(F.col('source_config_Radios_e')),F.to_json(F.col('source_config_Radios_e_step'))).alias('source_config_Radios_e_change'),
                                                        udf_wlan_change(F.to_json(F.col('source_config_WLANs_e')),F.to_json(F.col('source_config_WLANs_e_step'))).alias('source_config_WLANs_e_change'))
L2 = df_apscope_withconfigdiff.columns

df_apscope_withconfigdiff_2 = df_apscope_withconfigdiff.select(*[c for c in L2], *[(~F.col(c).contains('[]')).alias(c.replace("_change","_hasChanged")) for c in L2 if "_change" in c ] )
t2 = time.time()
log6 = '\n--- Data pulled ---\n %s' %str(t2 - start_time)
print(log6)
logs.append(log6)

    

#     rdd_config = df_apscope_withconfigdiff_2.rdd.map(lambda x: x.asDict())
#     rdd_config.persist()
#     out_filename = "s3://mist-data-science-dev/aditya/scope_batch_v2_1/" + dt_ee
#     rdd_config.map(lambda x: (None, json.dumps(x))).saveAsSequenceFile(out_filename)
#     t2 = time.time()
#     log7 = '\n--- Data written ---\n %s' %str(t2 - start_time)
#     print(log7)
#     logs.append(log7)
    
#     #---
#     df_select_scope_events_apscope.unpersist()
#     df_apevents_sample.unpersist()
#     rdd_config.unpersist()
#     return logs


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Start time: 1643650768

--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-24/hr=*/*/

--- Scope events with AP scope ---
1

--- APs involved --- 
 1
Some ap ids: ['d420b0c0b833']
dt=2022-01-{24,23}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{24,23}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{24,23}/hr=*/*/

---APs that had a config event within 6 hours ---
 1
dt=2022-01-{24,23,22,21,20,19,18,17}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{24,23,22,21,20,19,18,17}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{24,23,22,21,20,19,18,17}/hr=*/*/

--- AP configured events pulled for the last 7 days ---
 21

--- Data pulled ---
 617.5585215091705

In [20]:
df_apevents_sample.select('ap', 'source_config_Radios_e','source_config_WLANs_e').show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [29]:
fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]',
          'r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]',
          'r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]',
          'r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]',
          'r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']

L1 = [x_.replace("r[","").replace("\"][\"", "_").replace("]","").replace("\"","")+ '_step' for x_ in fields[2:]]
#---
step_window = Window.partitionBy('ap').orderBy(F.col("row_number").desc())

df_apscope_withconfig = df_select_scope_events_apscope.join(df_apevents_sample, on ='ap', how='inner')\
                                .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                .filter('time_delta>0')\
                                .withColumn('row_number', F.row_number().over(Window.partitionBy('ap').orderBy(F.col('time_delta').asc())))

df_apscope_withconfig = df_apscope_withconfig.select('ap', 'timestamp','row_key', 'start_time','time_delta', 'row_number',\
                                                     *[c.replace("_step", "").replace("\"", "") for c in L1], \
                                                     *[F.lag((c.replace("_step", "").replace("\"", "")), offset=1).over(step_window).alias(c) for c in L1],\
                                                    F.lag('timestamp', offset=1).over(step_window).alias('timestamp_step'),
                                                     'source_config_Radios_e',F.lag('source_config_Radios_e', offset=1).over(step_window).alias('source_config_Radios_e_step'),
                                                     'source_config_WLANs_e', F.lag('source_config_WLANs_e', offset=1).over(step_window).alias('source_config_WLANs_e_step')
                                                    )
#---
L0 = df_apscope_withconfig.columns
df_apscope_withconfigdiff = df_apscope_withconfig.select(*[c for c in L0],
                                                         *[getconfigDiff(F.col(c.replace("_step","")), F.col(c) ).alias(c.replace("_step","_change")) for c in L1],
                                                        udf_radio_change(F.to_json(F.col('source_config_Radios_e')),F.to_json(F.col('source_config_Radios_e_step'))).alias('source_config_Radios_e_change'),
                                                        udf_wlan_change(F.to_json(F.col('source_config_WLANs_e')),F.to_json(F.col('source_config_WLANs_e_step'))).alias('source_config_WLANs_e_change'))
L2 = df_apscope_withconfigdiff.columns

# df_apscope_withconfigdiff.select('ap', 'source_config_Radios_e','source_config_Radios_e_step','source_config_Radios_e_change').show(10, False)

# df_apscope_withconfigdiff_2 = df_apscope_withconfigdiff.select(*[c for c in L2], *[(~F.col(c).contains('[]')).alias(c.replace("_change","_hasChanged")) for c in L2 if "_change" in c ] )
# t2 = time.time()
# log6 = '\n--- Data pulled ---\n %s' %str(t2 - start_time)
# print(log6)
# logs.append(log6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
df_apscope_withconfigdiff.select('ap','row_number','source_config_Radios_e_change').orderBy('row_number').show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+-----------------------------+
|ap          |row_number|source_config_Radios_e_change|
+------------+----------+-----------------------------+
|d420b0c0b833|1         |[]                           |
|d420b0c0b833|2         |['2.4_Power', '2.4_RRM']     |
|d420b0c0b833|3         |['2.4_RRM']                  |
|d420b0c0b833|4         |['2.4_Power', '2.4_RRM']     |
|d420b0c0b833|5         |['2.4_RRM']                  |
|d420b0c0b833|6         |['2.4_RRM']                  |
|d420b0c0b833|7         |['2.4_RRM']                  |
|d420b0c0b833|8         |['2.4_RRM']                  |
|d420b0c0b833|9         |[]                           |
|d420b0c0b833|10        |[]                           |
+------------+----------+-----------------------------+
only showing top 10 rows

In [44]:
df_apscope_withconfigdiff.select('ap','row_number','source_config_WLANs_e_change').orderBy('row_number').show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+----------------------------+
|ap          |row_number|source_config_WLANs_e_change|
+------------+----------+----------------------------+
|d420b0c0b833|1         |[]                          |
|d420b0c0b833|2         |[]                          |
|d420b0c0b833|3         |[]                          |
|d420b0c0b833|4         |[]                          |
|d420b0c0b833|5         |[]                          |
|d420b0c0b833|6         |[]                          |
|d420b0c0b833|7         |[]                          |
|d420b0c0b833|8         |[]                          |
|d420b0c0b833|9         |[]                          |
|d420b0c0b833|10        |[]                          |
+------------+----------+----------------------------+
only showing top 10 rows

In [47]:
def ExtractConfigs(sc, var_year, var_month, var_date):
    start_time = time.time()
    logs = list()
    log1 = "Start time: %d" %start_time
    print(log1)
    logs.append(log1)
    # ---
    s3locRoot_ee = 's3://mist-aggregated-stats-production/entity_event/'
    s3locChild_ee = 'entity_event-production/'
    dt_ee = "dt=%s-%s-%s/" % (str(var_year), str(var_month).zfill(2), str(var_date).zfill(2))
    hr_ee = "hr=*/*/" #  "hr=14/" 
    s3loc_ee = s3locRoot_ee + s3locChild_ee + dt_ee + hr_ee
    print('\n--- s3loc_ee ---')
    print(s3loc_ee)
    logs.append(s3loc_ee)

    event_rdd_ee = sc.sequenceFile(s3loc_ee).map(lambda r: json.loads(r[1]))#.persist()
    event2consider = ['connectivity_failure']
    event_rdd_ee_scope = event_rdd_ee.filter(lambda r: (('event_name' in r.keys() ) and (r['event_name'] in event2consider) \
                                                        and ('ap' in r['reason'].keys()) and (r['severity'] > 50) ) ) # d420b0c0b833 d420b0845461
    # ---
    pull_schema = T.StructType([
    T.StructField('row_key', T.StringType(), True),
    T.StructField('start_time', T.LongType(), True),
    T.StructField('ap', T.StringType(), True)
    ])
    df_select_scope_events_apscope = event_rdd_ee_scope.map(
        lambda r: (r['row_key'], r['start_time'], r['reason']['ap'])).distinct().toDF(pull_schema)

    df_select_scope_events_apscope.persist()
    print('\n--- Scope events with AP scope ---')
    print(df_select_scope_events_apscope.count())
    #---
    ap_list = [x.ap for x in df_select_scope_events_apscope.select('ap').distinct().dropna().collect()]
    log2 = '\n--- APs involved --- \n %s' %str(len(ap_list))
    print(log2)
    logs.append(log2)
    log3  = 'Some ap ids: %s' %str(ap_list[:10])
    logs.append(log3)
    print(log3)
    # ---
    strval = "dt=%s-%s-{" %(str(var_year), str(var_month).zfill(2))
    for x in range(var_date, var_date-2, -1): #this wont work for 1st
        strval= strval + str(x).zfill(2) + ","
    print(strval[:-1] +"}/")
    dt_ap0 = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
    hr_ap0 = "hr=*/*/"
    readLoc0 = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap0 + hr_ap0
    print(readLoc0)
    pullSchema2 = T.StructType([       
        T.StructField('ap', T.StringType(), True),
        T.StructField('ev_name', T.StringType(), True),
        T.StructField('config', T.MapType(T.StringType(), T.StringType()), True),
        T.StructField('timestamp', T.LongType(), True),
    ])
    print('Reading from... %s' %readLoc0)
    rdd_apevents0 = sc.sequenceFile(readLoc0).map(lambda r: json.loads(r[1])) #.persist()
    rdd_apevents_sample0 = rdd_apevents0.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                         and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list) and (r['source']['ev_name']=="AP_CONFIGURED") )


    df_apevents_sample0 = rdd_apevents_sample0.map(lambda r: (r['source']['ap'],r['source']['ev_name'], r['source']['config'], r['timestamp'] )).toDF(pullSchema2)
    dfm0 = df_select_scope_events_apscope.select('row_key', 'start_time', 'ap')\
                                            .join(df_apevents_sample0, on='ap', how='inner')\
                                            .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                            .filter('time_delta>0 and time_delta <' + str(6*60*60*1000))
    ap_list_positive = [x.ap for x in dfm0.select('ap').distinct().dropna().collect()]
    log4 = '\n---APs that had a config event within 6 hours ---\n %s' %str(len(ap_list_positive))
    print(log4)
    logs.append(log4)
    #---
    strval = "dt=%s-%s-{" %(str(var_year), str(var_month).zfill(2))
    # strval = "dt=2021-12-{"
    for x in range(var_date, var_date-8, -1): #this wont work for dates before 8th
        strval= strval + str(x).zfill(2) + ","
    print(strval[:-1] +"}/")

    dt_ap = strval[:-1] +"}/" #"dt=2022-01-{%d,%d}/" %(var_date-1, var_date)
    hr_ap = "hr=*/*/"

    readLoc = "s3://mist-secorapp-production/ap-events/ap-events-production/" + dt_ap + hr_ap
    print(readLoc)
    #---
    print('Reading from... %s' %readLoc)
    rdd_apevents = sc.sequenceFile(readLoc).map(lambda r: json.loads(r[1])) #.persist()
    rdd_apevents_sample = rdd_apevents.filter(lambda r: ('event_type' in r.keys()) and (r['event_type'] =='device_events') \
                                         and ('ap' in r['source'].keys()) and (r['source']['ap'] in ap_list_positive) and (r['source']['ev_name']=="AP_CONFIGURED") )
    rdd_apevents_sample1 = rdd_apevents_sample.map(lambda r: returnFields(r))

    pullSchema4 = T.StructType([T.StructField("ap" , T.StringType() , True), T.StructField("timestamp" , T.LongType() , True), 
                                T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),
                                T.StructField("source_config_WLANs" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_SVIs" , T.ArrayType(T.StringType()) , True),
                                T.StructField("source_config_LED" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SSH" , T.MapType(T.StringType(),T.StringType()), True),
                                T.StructField("source_config_Stats" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_IrqBalance" , T.MapType(T.StringType(),T.StringType()) , True),
                                T.StructField("source_config_OurOUIs" , T.ArrayType(T.StringType()) , True),
                                T.StructField("source_config_ID" , T.StringType() , True),T.StructField("source_config_UUID" , T.StringType() , True),
                                T.StructField("source_config__errors" , T.ArrayType(T.StringType()) , True),
                                T.StructField("source_config_Name" , T.StringType() , True),T.StructField("source_config_SiteName" , T.StringType() , True),
                                T.StructField("source_config_HttpRedirector" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_SiteString" , T.StringType() , True),
                                T.StructField("source_config_ProxyArp" , T.MapType(T.StringType(),T.StringType()) , True),
                                T.StructField("source_config_Steer" , T.MapType(T.StringType(),T.StringType()) , True),
                                T.StructField("source_config_WIDS" , T.ArrayType(T.StringType()) , True),
                                T.StructField("source_config_CnaHostnames" , T.ArrayType(T.StringType()) , True),
                                T.StructField("source_config_Dnsmasq" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_AppDef" , T.ArrayType(T.StringType()) , True),
                                T.StructField("source_config_Monitor" , T.MapType(T.StringType(),T.StringType()) , True),T.StructField("source_config_ExtIO" , T.ArrayType(T.StringType()) , True),
                                T.StructField("source_config_PwrMgmt" , T.MapType(T.StringType(),T.StringType()) , True),
                                T.StructField("source_config_PoePwrConstrained" , T.MapType(T.StringType(),T.StringType()) , True),
                                T.StructField("source_config_AVS" , T.MapType(T.StringType(),T.StringType()) , True),
                                T.StructField("source_config_SwitchPorts" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_L2TPTunnels" , T.ArrayType(T.StringType()) , True),
                                T.StructField("source_config_VLANNames" , T.MapType(T.StringType(),T.StringType()) , True),
                                T.StructField("source_config_APRecovery" , T.MapType(T.StringType(),T.StringType()) , True),
                                T.StructField("source_config_Radios_e" , schema_radios, True),T.StructField("source_config_WLANs_e", schema_wlan, True)
    ])

    df_apevents_sample = rdd_apevents_sample1.toDF(pullSchema4)
    # df_apevents_sample.printSchema()
    df_apevents_sample.persist()
    log5 = '\n--- AP configured events pulled for the last 7 days ---\n %s' %str(df_apevents_sample.count())
    print(log5)
    logs.append(log5)
    # #---
    fields = ['r["source"]["ap"]','r["timestamp"]','r["source"]["config"]["Radios"]','r["source"]["config"]["Hostname"]','r["source"]["config"]["WLANs"]','r["source"]["config"]["SVIs"]','r["source"]["config"]["LED"]','r["source"]["config"]["SSH"]','r["source"]["config"]["Stats"]','r["source"]["config"]["IrqBalance"]','r["source"]["config"]["OurOUIs"]',
              'r["source"]["config"]["ID"]','r["source"]["config"]["UUID"]','r["source"]["config"]["_errors"]','r["source"]["config"]["Name"]','r["source"]["config"]["SiteName"]','r["source"]["config"]["HttpRedirector"]','r["source"]["config"]["SiteString"]','r["source"]["config"]["ProxyArp"]','r["source"]["config"]["Steer"]',
              'r["source"]["config"]["WIDS"]','r["source"]["config"]["CnaHostnames"]','r["source"]["config"]["Dnsmasq"]','r["source"]["config"]["AppDef"]','r["source"]["config"]["Monitor"]',
              'r["source"]["config"]["ExtIO"]','r["source"]["config"]["PwrMgmt"]','r["source"]["config"]["PoePwrConstrained"]','r["source"]["config"]["AVS"]',
              'r["source"]["config"]["SwitchPorts"]','r["source"]["config"]["L2TPTunnels"]','r["source"]["config"]["VLANNames"]','r["source"]["config"]["APRecovery"]']

    L1 = [x_.replace("r[","").replace("\"][\"", "_").replace("]","").replace("\"","")+ '_step' for x_ in fields[2:]]
    #---
    step_window = Window.partitionBy('ap').orderBy(F.col("row_number").desc())

    df_apscope_withconfig = df_select_scope_events_apscope.join(df_apevents_sample, on ='ap', how='inner')\
                                    .withColumn('time_delta', F.col('start_time')-F.col('timestamp')/1000)\
                                    .filter('time_delta>0')\
                                    .withColumn('row_number', F.row_number().over(Window.partitionBy('ap').orderBy(F.col('time_delta').asc())))

    df_apscope_withconfig = df_apscope_withconfig.select('ap', 'timestamp','row_key', 'start_time','time_delta', 'row_number',\
                                                         *[c.replace("_step", "").replace("\"", "") for c in L1], \
                                                         *[F.lag((c.replace("_step", "").replace("\"", "")), offset=1).over(step_window).alias(c) for c in L1],\
                                                        F.lag('timestamp', offset=1).over(step_window).alias('timestamp_step'),
                                                         'source_config_Radios_e',F.lag('source_config_Radios_e', offset=1).over(step_window).alias('source_config_Radios_e_step'),
                                                         'source_config_WLANs_e', F.lag('source_config_WLANs_e', offset=1).over(step_window).alias('source_config_WLANs_e_step')
                                                        )
    #---
    L0 = df_apscope_withconfig.columns
    df_apscope_withconfigdiff = df_apscope_withconfig.select(*[c for c in L0],
                                                             *[getconfigDiff(F.col(c.replace("_step","")), F.col(c) ).alias(c.replace("_step","_change")) for c in L1],
                                                            udf_radio_change(F.to_json(F.col('source_config_Radios_e')),F.to_json(F.col('source_config_Radios_e_step'))).alias('source_config_Radios_e_change'),
                                                            udf_wlan_change(F.to_json(F.col('source_config_WLANs_e')),F.to_json(F.col('source_config_WLANs_e_step'))).alias('source_config_WLANs_e_change'))
    L2 = df_apscope_withconfigdiff.columns

    df_apscope_withconfigdiff_2 = df_apscope_withconfigdiff.select(*[c for c in L2], *[(~F.col(c).contains('[]')).alias(c.replace("_change","_hasChanged")) for c in L2 if "_change" in c ] )
    t2 = time.time()
    log6 = '\n--- Data pulled ---\n %s' %str(t2 - start_time)
    print(log6)
    logs.append(log6)
    
    #---
    rdd_config = df_apscope_withconfigdiff_2.rdd.map(lambda x: x.asDict())
    rdd_config.persist()
    out_filename = "s3://mist-data-science-dev/aditya/scope_batch_v2_1/" + dt_ee
    rdd_config.map(lambda x: (None, json.dumps(x))).saveAsSequenceFile(out_filename)
    t2 = time.time()
    log7 = '\n--- Data written ---\n %s' %str(t2 - start_time)
    print(log7)
    logs.append(log7)
    
    #---
    df_select_scope_events_apscope.unpersist()
    df_apevents_sample.unpersist()
    rdd_config.unpersist()
    return logs


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
a = ExtractConfigs(sc, 2022, 1, 24)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Start time: 1643652901

--- s3loc_ee ---
s3://mist-aggregated-stats-production/entity_event/entity_event-production/dt=2022-01-24/hr=*/*/

--- Scope events with AP scope ---
260

--- APs involved --- 
 228
Some ap ids: ['d4dc092617ab', 'd420b0442072', '5c5b35505db6', '5c5b353e150e', '5c5b35f1e968', '5c5b35d24185', '5c5b35505e60', '5c5b35506bb7', '5c5b35d3359f', 'd420b000b324']
dt=2022-01-{24,23}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{24,23}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{24,23}/hr=*/*/

---APs that had a config event within 6 hours ---
 56
dt=2022-01-{24,23,22,21,20,19,18,17}/
s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{24,23,22,21,20,19,18,17}/hr=*/*/
Reading from... s3://mist-secorapp-production/ap-events/ap-events-production/dt=2022-01-{24,23,22,21,20,19,18,17}/hr=*/*/

--- AP configured events pulled for the last 7 days ---
 808

--- Data pulled ---
 608

<h4>check1

In [50]:
spark.catalog.clearCache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
sloc = "s3://mist-data-science-dev/aditya/scope_batch_v2_1/dt=2022-01-{24}/"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
rdd0 = sc.sequenceFile(sloc).map(lambda r: json.loads(r[1])).filter(lambda r: r['row_number']==1).persist()
print('\n',rdd0.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


 56

In [53]:
sorted(list(rdd0.take(1)[0].keys()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ap', 'row_key', 'row_number', 'source_config_APRecovery', 'source_config_APRecovery_change', 'source_config_APRecovery_hasChanged', 'source_config_APRecovery_step', 'source_config_AVS', 'source_config_AVS_change', 'source_config_AVS_hasChanged', 'source_config_AVS_step', 'source_config_AppDef', 'source_config_AppDef_change', 'source_config_AppDef_hasChanged', 'source_config_AppDef_step', 'source_config_CnaHostnames', 'source_config_CnaHostnames_change', 'source_config_CnaHostnames_hasChanged', 'source_config_CnaHostnames_step', 'source_config_Dnsmasq', 'source_config_Dnsmasq_change', 'source_config_Dnsmasq_hasChanged', 'source_config_Dnsmasq_step', 'source_config_ExtIO', 'source_config_ExtIO_change', 'source_config_ExtIO_hasChanged', 'source_config_ExtIO_step', 'source_config_Hostname', 'source_config_Hostname_change', 'source_config_Hostname_hasChanged', 'source_config_Hostname_step', 'source_config_HttpRedirector', 'source_config_HttpRedirector_change', 'source_config_HttpRedirecto

In [57]:
# T.StructField("source_config_Radios" , T.ArrayType(T.StringType()) , True),T.StructField("source_config_Hostname" , T.StringType() , True),T.StructField("source_config_WLANs"

pullSchema_new = T.StructType([
    T.StructField("ap" , T.StringType() , True),
    T.StructField("timestamp" , T.StringType() , True),
    T.StructField("source_config_Radios_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_WLANs_hasChanged" , T.BooleanType() , True),
    T.StructField("source_config_Radios_e_change" , T.StringType() , True),
    T.StructField("source_config_WLANs_e_change" , T.StringType() , True)])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
pickCols = ['ap', 'timestamp','source_config_Radios_hasChanged','source_config_WLANs_hasChanged',\
            'source_config_Radios_e_change','source_config_WLANs_e_change']
df0 = rdd0.map(lambda r: tuple([r[x_] for x_ in pickCols])).toDF(pullSchema_new)

df0.filter("source_config_Radios_hasChanged is true").show(100,)#.count()

df0.filter("source_config_WLANs_hasChanged is true").show(100,)#.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------------+-------------------------------+------------------------------+-----------------------------+----------------------------+
|          ap|       timestamp|source_config_Radios_hasChanged|source_config_WLANs_hasChanged|source_config_Radios_e_change|source_config_WLANs_e_change|
+------------+----------------+-------------------------------+------------------------------+-----------------------------+----------------------------+
|d420b0442072|1642925486000000|                           true|                         false|                          unk|                          []|
|d420b0845461|1643019242000000|                           true|                         false|         ['2.4_Power', '2....|                          []|
|5c5b35506973|1643012657000000|                           true|                         false|         ['5_Power', '5_RRM']|                          []|
|5c5b35bf3f38|1643018783000000|                           true|             